# แปลงเป็น Bag of words ข่าวจริง - จริง

In [1]:
import pythainlp
import numpy
import math
from pythainlp import word_tokenize
from pythainlp import corpus
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

In [2]:
import nltk
nltk.download('words')
th_stop = thai_stopwords()
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='icu')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if not i in th_stop and not i in punctuation and not i in ['“','”','‘','’']] 
    
    # Thai transform to root word
    # tokens_temp=[]
    # for i in tokens:
    #     w_syn = wordnet.synsets(i)
    #     if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
    #         tokens_temp.append(w_syn[0].lemma_names('tha')[0])
    #     else:
    #         tokens_temp.append(i)
    
    # tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [4]:
def find_cosine_similarity(news_array, first_part_index, second_part_index):
    sum = 0 #upper part of cosine similarity
    sum_a = 0
    sum_b = 0
    total_words_number = news_array.shape[1]

    for index in range(total_words_number):
        if (news_array[first_part_index, index] == 0) and (news_array[second_part_index, index] ==0):
            continue
        sum_a += news_array[first_part_index, index]**2
        sum_b += news_array[second_part_index, index]**2

        sum += news_array[first_part_index, index]*news_array[second_part_index, index]

    print('This is sum (upperpart) :',sum)
    
    squareroot_sum_a = math.sqrt(sum_a)
    squareroot_sum_b = math.sqrt(sum_b)
    lowerpart = squareroot_sum_a * squareroot_sum_b

    print('This is lowerpart :',lowerpart)
    return (sum/lowerpart)

In [5]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

# ข่าวที่ 1

In [6]:
clean_text = ["น้ำจะท่วมกรุงเทพฯ ในเดือนตุลา 64","โดยสรุปก็คึอ ถ้าดูจากตัวเลขปริมาณการไหลของน้ำ การระบายน้ำ ปริมาณน้ำในเขื่อนใหญ่ที่ยังน้อยมาก และยังมีพื้นที่รับน้ำได้อีกมาก รวมทั้งพื้นที่รับน้ำที่ท่วมอยู่ในเวลานี้ด้วย ก็จะเห็นว่า กรุงเทพมหานคร และนนทบุรี ยังคงไม่มีแนวโน้มจะถูกน้ำท่วม (ที่ไม่ใช่น้ำท่วมขังเวลาฝนตก)", "น้ำท่วมจะไปไหน ข่าวลือเริ่มแรง กรุงเทพฯ-นนทบุรี จะจมน้ำ?", "แหล่งข่าวผู้เชี่ยวชาญด้านทรัพยากรน้ำระบุ สถานการณ์น้ำในปัจจุบัน จากข้อมูลพบว่ายังคงแตกต่างจากปี 2554 หลังเกิดความกังวล ส่งต่อข้อมูลและเปรียบเทียบว่า น้ำจะท่วมกรุงเทพมหานครและนนทบุรีแบบมหาอุทกภัย"]

**Step 2: Tokenize**

In [7]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['น้ำ', 'ท่วม', 'กรุงเทพฯ', 'เดือน', 'ตุลา']
tokenized text:
 ['สรุป', 'คึอ', 'ดู', 'ตัวเลข', 'ปริมาณ', 'ไหล', 'น้ำ', 'ระบาย', 'น้ำ', 'ปริมาณ', 'น้ำ', 'เขื่อน', 'พื้นที่', 'น้ำ', 'พื้นที่', 'น้ำ', 'ท่วม', 'เวลา', 'กรุงเทพมหานคร', 'นนทบุรี', 'ไม่มี', 'แนว', 'โน้ม', 'น้ำ', 'ท่วม', 'น้ำ', 'ท่วม', 'ขัง', 'เวลา', 'ฝน', 'ตก']
tokenized text:
 ['น้ำ', 'ท่วม', 'ข่าว', 'ลือ', 'แรง', 'กรุงเทพฯ', 'นนทบุรี', 'จม', 'น้ำ']
tokenized text:
 ['แหล่ง', 'ข่าว', 'เชี่ยวชาญ', 'ทรัพยากร', 'น้ำ', 'ระบุ', 'สถานการณ์', 'น้ำ', 'ข้อมูล', 'แตก', 'ปี', 'กังวล', 'ข้อมูล', 'เปรียบ', 'เทียบ', 'น้ำ', 'ท่วม', 'กรุงเทพมหานคร', 'นนทบุรี', 'มหา', 'อุทกภัย']


In [8]:
tokens_list = [split_word(txt) for txt in clean_text]

In [9]:
tokens_list

[['น้ำ', 'ท่วม', 'กรุงเทพฯ', 'เดือน', 'ตุลา'],
 ['สรุป',
  'คึอ',
  'ดู',
  'ตัวเลข',
  'ปริมาณ',
  'ไหล',
  'น้ำ',
  'ระบาย',
  'น้ำ',
  'ปริมาณ',
  'น้ำ',
  'เขื่อน',
  'พื้นที่',
  'น้ำ',
  'พื้นที่',
  'น้ำ',
  'ท่วม',
  'เวลา',
  'กรุงเทพมหานคร',
  'นนทบุรี',
  'ไม่มี',
  'แนว',
  'โน้ม',
  'น้ำ',
  'ท่วม',
  'น้ำ',
  'ท่วม',
  'ขัง',
  'เวลา',
  'ฝน',
  'ตก'],
 ['น้ำ', 'ท่วม', 'ข่าว', 'ลือ', 'แรง', 'กรุงเทพฯ', 'นนทบุรี', 'จม', 'น้ำ'],
 ['แหล่ง',
  'ข่าว',
  'เชี่ยวชาญ',
  'ทรัพยากร',
  'น้ำ',
  'ระบุ',
  'สถานการณ์',
  'น้ำ',
  'ข้อมูล',
  'แตก',
  'ปี',
  'กังวล',
  'ข้อมูล',
  'เปรียบ',
  'เทียบ',
  'น้ำ',
  'ท่วม',
  'กรุงเทพมหานคร',
  'นนทบุรี',
  'มหา',
  'อุทกภัย']]

**Step 3 + 4a: Bag of words + count word**

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x0000027AA3187C10>)


In [11]:
cvec.vocabulary_

{'น้ำ': 15,
 'ท่วม': 13,
 'กรุงเทพฯ': 1,
 'เดือน': 29,
 'ตุลา': 11,
 'สรุป': 25,
 'คึอ': 6,
 'ดู': 8,
 'ตัวเลข': 10,
 'ปริมาณ': 16,
 'ไหล': 39,
 'ระบาย': 21,
 'เขื่อน': 27,
 'พื้นที่': 19,
 'เวลา': 32,
 'กรุงเทพมหานคร': 0,
 'นนทบุรี': 14,
 'ไม่มี': 38,
 'แนว': 34,
 'โน้ม': 37,
 'ขัง': 3,
 'ฝน': 18,
 'ตก': 9,
 'ข่าว': 4,
 'ลือ': 23,
 'แรง': 35,
 'จม': 7,
 'แหล่ง': 36,
 'เชี่ยวชาญ': 28,
 'ทรัพยากร': 12,
 'ระบุ': 22,
 'สถานการณ์': 24,
 'ข้อมูล': 5,
 'แตก': 33,
 'ปี': 17,
 'กังวล': 2,
 'เปรียบ': 31,
 'เทียบ': 30,
 'มหา': 20,
 'อุทกภัย': 26}

In [12]:
c_feat[:,:].todense()

matrix([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 3, 1, 7, 2, 0, 1, 2, 0,
         1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 0, 1, 0, 0, 1,
         0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]],
       dtype=int64)

In [13]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0]
 [1 0 0 1 0 0 1 0 1 1 1 0 0 3 1 7 2 0 1 2 0 1 0 0 0 1 0 1 0 0 0 0 2 0 1 0
  0 1 1 1]
 [0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0]
 [1 0 1 0 1 2 0 0 0 0 0 0 1 1 1 3 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0
  1 0 0 0]]


In [14]:
first_news.shape[1]

40

In [15]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 10
This is lowerpart : 20.615528128088304


0.48507125007266594

In [16]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 4
This is lowerpart : 7.416198487095663


0.5393598899705937

In [17]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 4
This is lowerpart : 12.041594578792296


0.3321819194149599

# -------------- ข่าวที่ 2 ----------------------------

In [18]:
clean_text = ["ขณะนี้ คิวในการยื่นขอ VISA ประเทศสหรัฐอเมริกา คิวอยู่ที่เดือนสิงหาคม พ.ศ. 2565 แล้ว จริงหรือไม่","ระยะเวลารอคิวการยื่นวีซ่าสามารถเช็กได้ที่เว็บไซต์https://travel.state.gov/content/travel/⋯sa-information-resources/wait-times.htmlจากการเช็กล่าสุดพบว่าระยะเวลาการรอวีซ่าคือVisitor Visa 35 Calendar DaysStudent/Exchange Visitor Visas 2 Calendar DaysAll Other Nonimmigrant Visas 2 Calendar Daysไม่ถึงปี 65 แต่อย่างใด"]

**Step 2: Tokenize**

In [19]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
# print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
# print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['คิว', 'ยื่น', 'VISA', 'ประเทศ', 'สหรัฐอเมริกา', 'คิว', 'เดือน', 'สิงหาคม', 'พ', 'ศ']
tokenized text:
 ['เวลา', 'รอ', 'คิว', 'ยื่น', 'วีซ่า', 'สา', 'รถ', 'เช็ก', 'เว็บไซต์', 'https', 'travel.state.gov', 'content', 'travel', '⋯', 'sa', 'information', 'resources', 'wait', 'times.html', 'จา', 'กการเช็', 'กล่า', 'เวลา', 'รอ', 'วีซ่า', 'Visitor', 'Visa', 'Calendar', 'DaysStudent', 'Exchange', 'Visitor', 'Visas', 'Calendar', 'DaysAll', 'Other', 'Nonimmigrant', 'Visas', 'Calendar', 'Days', 'ปี', 'ใด']


In [20]:
tokens_list = [split_word(txt) for txt in clean_text]

In [21]:
tokens_list

[['คิว',
  'ยื่น',
  'VISA',
  'ประเทศ',
  'สหรัฐอเมริกา',
  'คิว',
  'เดือน',
  'สิงหาคม',
  'พ',
  'ศ'],
 ['เวลา',
  'รอ',
  'คิว',
  'ยื่น',
  'วีซ่า',
  'สา',
  'รถ',
  'เช็ก',
  'เว็บไซต์',
  'https',
  'travel.state.gov',
  'content',
  'travel',
  '⋯',
  'sa',
  'information',
  'resources',
  'wait',
  'times.html',
  'จา',
  'กการเช็',
  'กล่า',
  'เวลา',
  'รอ',
  'วีซ่า',
  'Visitor',
  'Visa',
  'Calendar',
  'DaysStudent',
  'Exchange',
  'Visitor',
  'Visas',
  'Calendar',
  'DaysAll',
  'Other',
  'Nonimmigrant',
  'Visas',
  'Calendar',
  'Days',
  'ปี',
  'ใด']]

**Step 3 + 4a: Bag of words + count word**

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [23]:
cvec.vocabulary_

{'คิว': 22,
 'ยื่น': 27,
 'VISA': 7,
 'ประเทศ': 24,
 'สหรัฐอเมริกา': 32,
 'เดือน': 36,
 'สิงหาคม': 34,
 'พ': 26,
 'ศ': 31,
 'เวลา': 37,
 'รอ': 29,
 'วีซ่า': 30,
 'สา': 33,
 'รถ': 28,
 'เช็ก': 35,
 'เว็บไซต์': 38,
 'https': 12,
 'travel.state.gov': 18,
 'content': 11,
 'travel': 17,
 '⋯': 40,
 'sa': 15,
 'information': 13,
 'resources': 14,
 'wait': 19,
 'times.html': 16,
 'จา': 23,
 'กการเช็': 20,
 'กล่า': 21,
 'Visitor': 10,
 'Visa': 8,
 'Calendar': 0,
 'DaysStudent': 3,
 'Exchange': 4,
 'Visas': 9,
 'DaysAll': 2,
 'Other': 6,
 'Nonimmigrant': 5,
 'Days': 1,
 'ปี': 25,
 'ใด': 39}

In [24]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0],
        [3, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 1, 0, 1, 1, 2, 2, 0, 0, 1, 0, 1, 0, 2, 1, 1, 1]],
       dtype=int64)

In [25]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 1 1 0 0 0 1 1 0 1 0
  1 0 0 0 0]
 [3 1 1 1 1 1 1 0 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 2 2 0 0 1 0 1
  0 2 1 1 1]]


In [26]:
first_news.shape[1]

41

In [27]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 3
This is lowerpart : 26.15339366124404


0.1147078669352809

# ข่าวที่ 3

In [28]:
clean_text = ["ประเทศไทย มีจังหวัดเพิ่มอีก 6 จังหวัด จริงหรือ","ผู้สื่อข่าวรายงานว่า กรมการปกครอง กระทรวงมหาดไทย เผยแพร่ข้อมูลว่าตามที่ปรากฏข่าวสารผ่านสื่อออนไลน์ว่า “ประกาศชาวไทย จะมี 83 จังหวัดแล้ว” นั้น กรมการปกครอง กระทรวงมหาดไทย ชี้แจงว่า ข้อมูลข่าวดังกล่าวไม่เป็นความจริงแต่ประการใด และเป็นการแชร์ข่าวที่เคยเป็นข่าวปลอม มาตั้งแต่ปี พ.ศ.2558 แล้ว","มท.ยืนยันประเทศไทยมีแค่ 76 จังหวัด","กรมการปกครอง ร่อนหนังสือชี้แจงข่าวปลอมในโซเชียล อ้างประกาศชาวไทยจะมี 83 จังหวัด ชี้เป็นข่าวปลอมตั้งแต่ปี 2558 ยืนยันไทยมี 76 จังหวัด"]

**Step 2: Tokenize**

In [29]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ประเทศไทย', 'จังหวัด', 'จังหวัด']
tokenized text:
 ['สื่อ', 'ข่าว', 'รายงาน', 'กรม', 'ปกครอง', 'กระทรวง', 'มหาดไทย', 'เผย', 'แพร่', 'ข้อมูล', 'ข่าวสาร', 'สื่อ', 'ออนไลน์', 'ประกาศ', 'ไทย', 'จะมี', 'จังหวัด', 'กรม', 'ปกครอง', 'กระทรวง', 'มหาดไทย', 'ชี้แจง', 'ข้อมูล', 'ข่าว', 'ใด', 'แชร์', 'ข่าว', 'ข่าว', 'ปลอม', 'ปี', 'พ', 'ศ']
tokenized text:
 ['มท', 'ประเทศไทย', 'จังหวัด']
tokenized text:
 ['กรม', 'ปกครอง', 'ร่อน', 'หนังสือ', 'ชี้แจง', 'ข่าว', 'ปลอม', 'โซ', 'เชีย', 'ล', 'อ้าง', 'ประกาศ', 'ไทย', 'จังหวัด', 'ชี้', 'ข่าว', 'ปลอม', 'ปี', 'ไทย', 'จังหวัด']


In [30]:
tokens_list = [split_word(txt) for txt in clean_text]

In [31]:
tokens_list

[['ประเทศไทย', 'จังหวัด', 'จังหวัด'],
 ['สื่อ',
  'ข่าว',
  'รายงาน',
  'กรม',
  'ปกครอง',
  'กระทรวง',
  'มหาดไทย',
  'เผย',
  'แพร่',
  'ข้อมูล',
  'ข่าวสาร',
  'สื่อ',
  'ออนไลน์',
  'ประกาศ',
  'ไทย',
  'จะมี',
  'จังหวัด',
  'กรม',
  'ปกครอง',
  'กระทรวง',
  'มหาดไทย',
  'ชี้แจง',
  'ข้อมูล',
  'ข่าว',
  'ใด',
  'แชร์',
  'ข่าว',
  'ข่าว',
  'ปลอม',
  'ปี',
  'พ',
  'ศ'],
 ['มท', 'ประเทศไทย', 'จังหวัด'],
 ['กรม',
  'ปกครอง',
  'ร่อน',
  'หนังสือ',
  'ชี้แจง',
  'ข่าว',
  'ปลอม',
  'โซ',
  'เชีย',
  'ล',
  'อ้าง',
  'ประกาศ',
  'ไทย',
  'จังหวัด',
  'ชี้',
  'ข่าว',
  'ปลอม',
  'ปี',
  'ไทย',
  'จังหวัด']]

**Step 3 + 4a: Bag of words + count word**

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x0000027AA3242D30>)


In [33]:
cvec.vocabulary_

{'ประเทศไทย': 11,
 'จังหวัด': 6,
 'สื่อ': 21,
 'ข่าว': 2,
 'รายงาน': 17,
 'กรม': 0,
 'ปกครอง': 9,
 'กระทรวง': 1,
 'มหาดไทย': 16,
 'เผย': 26,
 'แพร่': 28,
 'ข้อมูล': 4,
 'ข่าวสาร': 3,
 'ออนไลน์': 23,
 'ประกาศ': 10,
 'ไทย': 31,
 'จะมี': 5,
 'ชี้แจง': 8,
 'ใด': 30,
 'แชร์': 27,
 'ปลอม': 12,
 'ปี': 13,
 'พ': 14,
 'ศ': 20,
 'มท': 15,
 'ร่อน': 18,
 'หนังสือ': 22,
 'โซ': 29,
 'เชีย': 25,
 'ล': 19,
 'อ้าง': 24,
 'ชี้': 7}

In [34]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 2, 4, 1, 2, 1, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1,
         2, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 2, 0, 0, 0, 2, 1, 1, 1, 1, 0, 2, 1, 0, 0, 0, 0, 1, 1, 0,
         0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 2]], dtype=int64)

In [35]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 2 4 1 2 1 1 0 1 2 1 0 1 1 1 0 2 1 0 0 1 2 0 1 0 0 1 1 1 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 2 0 0 0 2 1 1 1 1 0 2 1 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 1 0 2]]


In [36]:
first_news.shape[1]

32

In [37]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 2
This is lowerpart : 16.73320053068151


0.11952286093343936

In [38]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 3
This is lowerpart : 3.872983346207417


0.7745966692414834

In [39]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 4
This is lowerpart : 11.832159566199234


0.33806170189140655

# ข่าวที่ 4

In [40]:
clean_text = ["ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม", "นายอำเภอเมืองยะลา นำทีมลงพื้นที่ กำชับมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่มเคร่งครัด เพื่อลดความเสี่ยงแพร่เชื้อโควิด-19 ที่แพร่ระบาดอย่างรวดเร็ว รวมถึง คลัสเตอร์ใหญ่ในพื้นที่จังหวัดยะลา (โรงเรียน โรงงานยาง ยะลา เบตง) มียอดผู้ติดเชื้อเพิ่มสูงขึ้นอย่างต่อเนื่อง", "ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม ลดเสี่ยงโควิด-19","นายอำเภอเมืองยะลา นำทีมลงพื้นที่ กำชับมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่มเคร่งครัด เพื่อลดความเสี่ยงแพร่เชื้อโควิด-19"]

**Step 2: Tokenize**

In [41]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ยะลา', 'คุม', 'เข้ม', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่อง', 'ดื่ม', 'ทุ่ม']
tokenized text:
 ['อำเภอ', 'เมือง', 'ยะลา', 'ทีม', 'ลงพื้น', 'กำชับ', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่อง', 'ดื่ม', 'ทุ่ม', 'เคร่งครัด', 'ลด', 'เสี่ยง', 'แพร่', 'เชื้อ', 'โค', 'วิด', 'แพร่', 'ระบาด', 'คลัสเตอร์', 'พื้นที่', 'จังหวัด', 'ยะลา', 'โรงเรียน', 'โรงงาน', 'ยาง', 'ยะลา', 'เบตง', 'ยอด', 'ติด', 'เชื้อ', 'เนื่อง']
tokenized text:
 ['ยะลา', 'คุม', 'เข้ม', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่อง', 'ดื่ม', 'ทุ่ม', 'ลด', 'เสี่ยง', 'โค', 'วิด']
tokenized text:
 ['อำเภอ', 'เมือง', 'ยะลา', 'ทีม', 'ลงพื้น', 'กำชับ', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่อง', 'ดื่ม', 'ทุ่ม', 'เคร่งครัด', 'ลด', 'เสี่ยง', 'แพร่', 'เชื้อ', 'โค', 'วิด']


In [42]:
tokens_list = [split_word(txt) for txt in clean_text]

In [43]:
tokens_list

[['ยะลา',
  'คุม',
  'เข้ม',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่อง',
  'ดื่ม',
  'ทุ่ม'],
 ['อำเภอ',
  'เมือง',
  'ยะลา',
  'ทีม',
  'ลงพื้น',
  'กำชับ',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่อง',
  'ดื่ม',
  'ทุ่ม',
  'เคร่งครัด',
  'ลด',
  'เสี่ยง',
  'แพร่',
  'เชื้อ',
  'โค',
  'วิด',
  'แพร่',
  'ระบาด',
  'คลัสเตอร์',
  'พื้นที่',
  'จังหวัด',
  'ยะลา',
  'โรงเรียน',
  'โรงงาน',
  'ยาง',
  'ยะลา',
  'เบตง',
  'ยอด',
  'ติด',
  'เชื้อ',
  'เนื่อง'],
 ['ยะลา',
  'คุม',
  'เข้ม',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่อง',
  'ดื่ม',
  'ทุ่ม',
  'ลด',
  'เสี่ยง',
  'โค',
  'วิด'],
 ['อำเภอ',
  'เมือง',
  'ยะลา',
  'ทีม',
  'ลงพื้น',
  'กำชับ',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่อง',
  'ดื่ม',
  'ทุ่ม',
  'เคร่งครัด',
  'ลด',
  'เสี่ยง',
  'แพร่',
  'เชื้อ',
  'โค',
  'วิด']]

**Step 3 + 4a: Bag of words + count word**

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x0000027AA3242670>)


In [45]:
cvec.vocabulary_

{'ยะลา': 12,
 'คุม': 3,
 'เข้ม': 21,
 'มาตรการ': 10,
 'เคอร์ฟิว': 24,
 'ร้าน': 15,
 'ขาย': 1,
 'อาหาร': 19,
 'เครื่อง': 22,
 'ดื่ม': 5,
 'ทุ่ม': 8,
 'อำเภอ': 20,
 'เมือง': 28,
 'ทีม': 7,
 'ลงพื้น': 16,
 'กำชับ': 0,
 'เคร่งครัด': 23,
 'ลด': 17,
 'เสี่ยง': 29,
 'แพร่': 30,
 'เชื้อ': 25,
 'โค': 31,
 'วิด': 18,
 'ระบาด': 14,
 'คลัสเตอร์': 2,
 'พื้นที่': 9,
 'จังหวัด': 4,
 'โรงเรียน': 33,
 'โรงงาน': 32,
 'ยาง': 13,
 'เบตง': 27,
 'ยอด': 11,
 'ติด': 6,
 'เนื่อง': 26}

In [46]:
c_feat[:,:].todense()

matrix([[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
         1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1],
        [0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
         1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0]], dtype=int64)

In [47]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 0 1 1 1 2 1 1 1 1 2 1 1 1]
 [0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 0 0]
 [1 1 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 0]]


In [48]:
first_news.shape[1]

34

In [49]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 11
This is lowerpart : 22.494443758403985


0.48900964692182575

In [50]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 11
This is lowerpart : 12.84523257866513


0.8563488385776753

In [51]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 9
This is lowerpart : 15.198684153570662


0.5921565254637922

# ข่าวที่ 5

In [52]:
clean_text = ["ถ้าใครมีเส้นเลือดในสมองเริ่มตีบแล้ว จะไม่สามารถยืนด้วยขาข้างเดียวถึง 20 วินาที ซึ่งน่าจะจริง เพราะ ได้ไปทำ MRI ที่สมองแล้ว แพทย์พบว่า มีเส้นเลือดในสมองตีบ และ ได้มาพบคลิบนี้ ลองยืนขาเดียว ปรากฏว่า ยืนได้เพียง 3 วินาทีเท่านั้นจึงคิดว่า มีความเชื่อมโยงกันจริง ขอเชิญแต่ละท่านทดสอบว่า เส้นเลือดในสมองของท่านยังปกติ ดีหรือไม่", "ศูนย์ต่อต้านข่าวปลอม ประเทศไทย เปิดเผยว่า ตามที่มีคลิปวิดีโอแนะนำ เกี่ยวกับประเด็นเรื่อง ยืนขาเดียว ทดสอบการเป็นโรคหลอดเลือดสมองตีบ ด้วยตนเอง ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา กรมการแพทย์ กระทรวงสาธารณสุข", "ข่าวบิดเบือน! ยืนขาเดียว ทดสอบการเป็นโรคหลอดเลือดสมองตีบ ด้วยตนเอง","จากประเด็นยืนขาเดียว ทดสอบการเป็นโรคหลอดเลือดสมองตีบ ด้วยตนเอง ทางสถาบันประสาทวิทยา กรมการแพทย์ ชี้แจงว่าการยืนขาเดียวได้ไม่เกิน 20 วินาที ไม่ได้เป็นการยืนยันว่าเป็นโรคหลอดเลือดสมอง ซึ่งมีโรคอื่นที่เกิดจากการทรงตัวผิดปกติได้อีก เช่นโรคของการรับความรู้สึกของข้อผิดปกติ โรคข้อต่างๆ ก็อาจจะให้ผลเช่นกัน โดยโรคหลอดเลือดสมอง มักจะมีอาการเป็นฉับพลันทันใด ต้องได้รับการรักษาที่ถูกต้องเหมาะสมอย่างทันท่วงที"]

**Step 2: Tokenize**

In [53]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เส้นเลือด', 'สมอง', 'ตีบ', 'ยืน', 'ขา', 'วินาที', 'ทำ', 'MRI', 'สมอง', 'แพทย์', 'เส้นเลือด', 'สมอง', 'ตีบ', 'ค', 'ลิบ', 'ลอง', 'ยืน', 'ขา', 'ยืน', 'วินาที', 'เชื่อม', 'โยง', 'เชิญ', 'ท่าน', 'ทดสอบ', 'เส้นเลือด', 'สมอง', 'ท่าน', 'ปกติ', 'ดี']
tokenized text:
 ['ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ประเทศไทย', 'เผย', 'คลิป', 'วิดีโอ', 'แนะนำ', 'เกี่ยว', 'ประเด็น', 'เรื่อง', 'ยืน', 'ขา', 'ทดสอบ', 'โรค', 'หลอด', 'เลือด', 'สมอง', 'ตีบ', 'ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ดำเนิน', 'ตรวจ', 'สอบ', 'ข้อ', 'เท็จ', 'สถาบัน', 'ประสาท', 'วิทยา', 'กรม', 'แพทย์', 'กระทรวง', 'สาธารณสุข']
tokenized text:
 ['ข่าว', 'บิดเบือน', 'ยืน', 'ขา', 'ทดสอบ', 'โรค', 'หลอด', 'เลือด', 'สมอง', 'ตีบ']
tokenized text:
 ['ประเด็น', 'ยืน', 'ขา', 'ทดสอบ', 'โรค', 'หลอด', 'เลือด', 'สมอง', 'ตีบ', 'สถาบัน', 'ประสาท', 'วิทยา', 'กรม', 'แพทย์', 'ชี้แจง', 'ว่าการ', 'ยืน', 'ขา', 'วินาที', 'โรค', 'หลอด', 'เลือด', 'สมอง', 'โรค', 'ทรงตัว', 'ปกติ', 'โรค', 'รู้สึก', 'ข้อ', 'ปกติ', 'โรค', 'ข้อ', 'โรค', 'หลอด', 'เลือด', 'สม

In [54]:
tokens_list = [split_word(txt) for txt in clean_text]

In [55]:
tokens_list

[['เส้นเลือด',
  'สมอง',
  'ตีบ',
  'ยืน',
  'ขา',
  'วินาที',
  'ทำ',
  'MRI',
  'สมอง',
  'แพทย์',
  'เส้นเลือด',
  'สมอง',
  'ตีบ',
  'ค',
  'ลิบ',
  'ลอง',
  'ยืน',
  'ขา',
  'ยืน',
  'วินาที',
  'เชื่อม',
  'โยง',
  'เชิญ',
  'ท่าน',
  'ทดสอบ',
  'เส้นเลือด',
  'สมอง',
  'ท่าน',
  'ปกติ',
  'ดี'],
 ['ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ประเทศไทย',
  'เผย',
  'คลิป',
  'วิดีโอ',
  'แนะนำ',
  'เกี่ยว',
  'ประเด็น',
  'เรื่อง',
  'ยืน',
  'ขา',
  'ทดสอบ',
  'โรค',
  'หลอด',
  'เลือด',
  'สมอง',
  'ตีบ',
  'ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ดำเนิน',
  'ตรวจ',
  'สอบ',
  'ข้อ',
  'เท็จ',
  'สถาบัน',
  'ประสาท',
  'วิทยา',
  'กรม',
  'แพทย์',
  'กระทรวง',
  'สาธารณสุข'],
 ['ข่าว',
  'บิดเบือน',
  'ยืน',
  'ขา',
  'ทดสอบ',
  'โรค',
  'หลอด',
  'เลือด',
  'สมอง',
  'ตีบ'],
 ['ประเด็น',
  'ยืน',
  'ขา',
  'ทดสอบ',
  'โรค',
  'หลอด',
  'เลือด',
  'สมอง',
  'ตีบ',
  'สถาบัน',
  'ประสาท',
  'วิทยา',
  'กรม',
  'แพทย์',
  'ชี้แจง',
  'ว่าการ',
  'ยืน',
  'ขา',
  'วินาที',
  'โรค',


**Step 3 + 4a: Bag of words + count word**

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x0000027AA3242160>)


In [57]:
cvec.vocabulary_

{'เส้นเลือด': 53,
 'สมอง': 41,
 'ตีบ': 13,
 'ยืน': 28,
 'ขา': 3,
 'วินาที': 36,
 'ทำ': 18,
 'MRI': 0,
 'แพทย์': 56,
 'ค': 6,
 'ลิบ': 33,
 'ลอง': 32,
 'เชื่อม': 48,
 'โยง': 57,
 'เชิญ': 47,
 'ท่าน': 20,
 'ทดสอบ': 15,
 'ปกติ': 22,
 'ดี': 11,
 'ศูนย์': 38,
 'ต้าน': 14,
 'ข่าว': 4,
 'ปลอม': 26,
 'ประเทศไทย': 25,
 'เผย': 50,
 'คลิป': 7,
 'วิดีโอ': 34,
 'แนะนำ': 55,
 'เกี่ยว': 46,
 'ประเด็น': 24,
 'เรื่อง': 51,
 'โรค': 58,
 'หลอด': 44,
 'เลือด': 52,
 'ดำเนิน': 10,
 'ตรวจ': 12,
 'สอบ': 42,
 'ข้อ': 5,
 'เท็จ': 49,
 'สถาบัน': 39,
 'ประสาท': 23,
 'วิทยา': 35,
 'กรม': 1,
 'กระทรวง': 2,
 'สาธารณสุข': 43,
 'บิดเบือน': 21,
 'ชี้แจง': 9,
 'ว่าการ': 37,
 'ทรงตัว': 16,
 'รู้สึก': 31,
 'อาการ': 45,
 'ฉับ': 8,
 'พลัน': 27,
 'ทันใด': 17,
 'รักษา': 30,
 'เหมาะ': 54,
 'สมอ': 40,
 'ย่าง': 29,
 'ท่วงที': 19}

In [58]:
c_feat[:,:].todense()

matrix([[1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 1, 0, 2,
         0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 4,
         0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 0, 0, 1, 1, 0],
        [0, 1, 1, 1, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 0, 1,
         1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1],
        [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
         0, 2, 1, 1, 0, 0, 1, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 3,
         0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 6]], dtype=int64)

In [59]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 0 0 2 0 0 1 0 0 0 0 1 0 2 0 1 0 0 1 0 2 0 1 0 0 0 0 0 3 0 0 0 1 1 0 0
  2 0 0 0 0 4 0 0 0 0 0 1 1 0 0 0 0 3 0 0 1 1 0]
 [0 1 1 1 2 1 0 1 0 0 1 0 1 1 2 1 0 0 0 0 0 0 0 1 1 1 2 0 1 0 0 0 0 0 1 1
  0 0 2 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [0 1 0 2 0 2 0 0 1 1 0 0 0 1 0 1 1 1 0 1 0 0 2 1 1 0 0 1 2 1 1 1 0 0 0 1
  1 1 0 1 1 3 0 0 3 1 0 0 0 0 0 0 3 0 1 0 1 0 6]]


In [60]:
first_news.shape[1]

59

In [61]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 13
This is lowerpart : 52.230259428802384


0.24889786384693216

In [62]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 12
This is lowerpart : 24.899799195977465


0.4819315973414993

In [63]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 30
This is lowerpart : 79.13279977354523


0.379109548579744

# ข่าวที่ 6

In [64]:
clean_text = ["กรุงเทพธุรกิจ ลงข่าว จับตาพายุดีเปรสชั่น มินเดลเล หวั่นถล่มไทย ฝนตกหนักช่วง 24-27 กันยายน นี้(ภายหลังแก้ไขแล้ว)ส.ส.กิตติ์ธัญญา วาจาดี โพสต์เฟสบุคส์ว่า พายุดีเปรสชั่น มินเดึลเล ก่อตัวขึ้นในทะเลจีนใต้ เป็นพายุลูกที่ 16 ของปีนี้ ทำให้ฝนตกหนักถึงหนักมากที่อุบลราชธานี และพื้นที่ใกล้เคียง(ภายหลังลบโพสต์แล้ว แต่มีการแคปหน้าจอส่งกันทางไลน์)ไม่จริง เพราะพายุ มินเดึลเล อยู่แถวฟิลิปปินส์ มุ่งหน้าญี่ปุ่น ไม่สามารถข้ามมาเมืองไทยได้พายุที่เข้ามาชื่อ เตี้ยนหมู่ และได้อ่อนกำลังลงแล้ว","พายุไต้ฝุ่น 'มินเดึลเล' เคลื่อนที่มุ่งหน้าไปทางญี่ปุ่น และพายุไซโคลนเคลื่อนที่มุ่งหน้าไปทางอินเดีย","เปิดเส้นทางพายุ 3 ลูก 'เตี้ยนหมู่ มินเดึลเล ไซโคลน' ไทยโดนด้วย อีสาน-กลาง ฝนตกหนัก",'ประธานกรรมการบริหาร "ทีมกรุ๊ป" หรือ TEAMG เปิดเส้นทางพายุ 3 ลูก "พายุเตี้ยนหมู่" พายุไซโคลน และ พายุดีเปรสชั่น "มินเดึลเล" ไทยมีฝนตกหนักในภาคอีสาน ภาคกลาง']

**Step 2: Tokenize**

In [65]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['กรุงเทพ', 'ธุรกิจ', 'ลงข่าว', 'จับตา', 'พายุ', 'ดีเปรสชั่น', 'นเดลเล', 'หวั่น', 'ถล่ม', 'ไทย', 'ฝน', 'ตกหนัก', 'กันยายน', 'ส', 'ส', 'กิตติ์', 'ธัญ', 'ญา', 'วาจา', 'ดี', 'โพสต์', 'เฟส', 'บุคส์', 'พายุ', 'ดีเปรสชั่น', 'นเดึลเล', 'ก่อ', 'ตัว', 'ทะเลจีนใต้', 'พายุ', 'ลูก', 'ปี', 'ฝน', 'ตกหนัก', 'หนัก', 'อุบลราชธานี', 'พื้นที่', 'เคียง', 'ลบ', 'โพสต์', 'แค', 'ป', 'หน้า', 'จอ', 'ไลน์', 'พายุ', 'นเดึลเล', 'แถว', 'ฟิลิปปินส์', 'หน้า', 'ญี่ปุ่น', 'ข้าม', 'เมือง', 'ไทย', 'พายุ', 'ชื่อ', 'เตี้ย', 'น', 'หมู่', 'อ่อน']
tokenized text:
 ['พายุ', 'ไต้ฝุ่น', 'นเดึลเล', 'เคลื่อนที่', 'หน้า', 'ญี่ปุ่น', 'พายุ', 'ไซโคลน', 'เคลื่อนที่', 'หน้า', 'อินเดีย']
tokenized text:
 ['เส้น', 'พายุ', 'ลูก', 'เตี้ย', 'น', 'หมู่', 'นเดึลเล', 'ไซโคลน', 'ไทย', 'โดน', 'อีสาน', 'กลาง', 'ฝน', 'ตกหนัก']
tokenized text:
 ['ประธาน', 'กรรมการ', 'บริหาร', 'ทีม', 'กรุ๊ป', 'TEAMG', 'เส้น', 'พายุ', 'ลูก', 'ยุ', 'เตี้ย', 'น', 'หมู่', 'พายุ', 'ไซโคลน', 'พายุ', 'ดีเปรสชั่น', 'นเดึลเล', 'ไทย', 'ฝน', 'ตกหนัก', 'อีสาน'

In [66]:
tokens_list = [split_word(txt) for txt in clean_text]

In [67]:
tokens_list

[['กรุงเทพ',
  'ธุรกิจ',
  'ลงข่าว',
  'จับตา',
  'พายุ',
  'ดีเปรสชั่น',
  'นเดลเล',
  'หวั่น',
  'ถล่ม',
  'ไทย',
  'ฝน',
  'ตกหนัก',
  'กันยายน',
  'ส',
  'ส',
  'กิตติ์',
  'ธัญ',
  'ญา',
  'วาจา',
  'ดี',
  'โพสต์',
  'เฟส',
  'บุคส์',
  'พายุ',
  'ดีเปรสชั่น',
  'นเดึลเล',
  'ก่อ',
  'ตัว',
  'ทะเลจีนใต้',
  'พายุ',
  'ลูก',
  'ปี',
  'ฝน',
  'ตกหนัก',
  'หนัก',
  'อุบลราชธานี',
  'พื้นที่',
  'เคียง',
  'ลบ',
  'โพสต์',
  'แค',
  'ป',
  'หน้า',
  'จอ',
  'ไลน์',
  'พายุ',
  'นเดึลเล',
  'แถว',
  'ฟิลิปปินส์',
  'หน้า',
  'ญี่ปุ่น',
  'ข้าม',
  'เมือง',
  'ไทย',
  'พายุ',
  'ชื่อ',
  'เตี้ย',
  'น',
  'หมู่',
  'อ่อน'],
 ['พายุ',
  'ไต้ฝุ่น',
  'นเดึลเล',
  'เคลื่อนที่',
  'หน้า',
  'ญี่ปุ่น',
  'พายุ',
  'ไซโคลน',
  'เคลื่อนที่',
  'หน้า',
  'อินเดีย'],
 ['เส้น',
  'พายุ',
  'ลูก',
  'เตี้ย',
  'น',
  'หมู่',
  'นเดึลเล',
  'ไซโคลน',
  'ไทย',
  'โดน',
  'อีสาน',
  'กลาง',
  'ฝน',
  'ตกหนัก'],
 ['ประธาน',
  'กรรมการ',
  'บริหาร',
  'ทีม',
  'กรุ๊ป',
  'TEAMG',
  'เส้น',
  'พายุ',

**Step 3 + 4a: Bag of words + count word**

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x0000027AA3187AF0>)


In [69]:
cvec.vocabulary_

{'กรุงเทพ': 2,
 'ธุรกิจ': 22,
 'ลงข่าว': 36,
 'จับตา': 10,
 'พายุ': 32,
 'ดีเปรสชั่น': 15,
 'นเดลเล': 24,
 'หวั่น': 44,
 'ถล่ม': 18,
 'ไทย': 61,
 'ฝน': 31,
 'ตกหนัก': 16,
 'กันยายน': 5,
 'ส': 40,
 'กิตติ์': 6,
 'ธัญ': 21,
 'ญา': 12,
 'วาจา': 39,
 'ดี': 14,
 'โพสต์': 58,
 'เฟส': 52,
 'บุคส์': 27,
 'นเดึลเล': 25,
 'ก่อ': 7,
 'ตัว': 17,
 'ทะเลจีนใต้': 19,
 'ลูก': 38,
 'ปี': 30,
 'หนัก': 41,
 'อุบลราชธานี': 47,
 'พื้นที่': 33,
 'เคียง': 50,
 'ลบ': 37,
 'แค': 55,
 'ป': 28,
 'หน้า': 42,
 'จอ': 9,
 'ไลน์': 62,
 'แถว': 56,
 'ฟิลิปปินส์': 34,
 'ญี่ปุ่น': 13,
 'ข้าม': 8,
 'เมือง': 53,
 'ชื่อ': 11,
 'เตี้ย': 51,
 'น': 23,
 'หมู่': 43,
 'อ่อน': 48,
 'ไต้ฝุ่น': 60,
 'เคลื่อนที่': 49,
 'ไซโคลน': 59,
 'อินเดีย': 45,
 'เส้น': 54,
 'โดน': 57,
 'อีสาน': 46,
 'กลาง': 4,
 'ประธาน': 29,
 'กรรมการ': 1,
 'บริหาร': 26,
 'ทีม': 20,
 'กรุ๊ป': 3,
 'TEAMG': 0,
 'ยุ': 35}

In [70]:
c_feat[:,:].todense()

matrix([[0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0,
         1, 1, 1, 1, 2, 0, 1, 1, 0, 1, 2, 5, 1, 1, 0, 1, 1, 1, 1, 2, 1,
         2, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0, 0, 2, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
         0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 3, 0, 0, 1, 0, 0, 1, 0, 0, 0,
         0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]],
       dtype=int64)

In [71]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 0 1 1 1 1 2 0 1 1 0 1 2 5 1 1 0
  1 1 1 1 2 1 2 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 2 0 0 2 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 2 0 0 0
  0 0 0 0 0 0 2 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0
  0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 3 0 0 1
  0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0]]


In [72]:
first_news.shape[1]

63

In [73]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 17
This is lowerpart : 40.39801975344831


0.4208127057650866

In [74]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 17
This is lowerpart : 36.66060555964672


0.46371301675148385

In [75]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 29
This is lowerpart : 52.76362383309167


0.549621081594705

# ข่าวที่ 7

In [76]:
clean_text = ['"จิงจูฉ่าย" เป็นผัก มหัศจรรย์ สามารถใช้รักษามะเร็งได้ !————————————“ผม..นิกกี้ อิทธิเกษม KU37 อดีตนายกสมาคมม.เกษตรฯแห่งอเมริกา จะขอเล่าประสบการณ์ที่เคยสัมผัสมากับตัวเองเมื่อปี 2007 เพื่อเป็นcaseตัวอย่างและเป็นทางออกให้กับผู้ป่วยโรคมะเร็งทั้งหลาย       ผมมีร้านอาหารอยู่ในเขต N.Hooywood วันหนึ่งกลางปี 2007 ผมได้มีโอกาสเจอลูกศิษย์เก่าที่เคยมาเรียนแจกไพ่ที่ร้าน ถามทุกข์สุขกันไปมา ถึงได้รู้ว่าเขาเคยเป็นมะเร็งที่คอ 1 ครั้ง,มะเร็งที่ต่อมลูกหมาก 1 ครั้ง ปัจจุบันหายเป็นปลิดทิ้ง รวมทั้งพี่น้องญาติอีก8คนที่เป็นมะเร็งภายในเวลา5 ปี ทุกคนหายหมด ผมจึงเกิดความกระตือรือร้นที่จะถามว่า มีวิธีรักษากันอย่างไร? วันนั้นคุณแม่เขามาด้วยจึงอธิบายให้ฟังว่า "ฉันมีต้นจิงจูฉ่ายอยู่ในสวนหลังบ้านเยอะ ปกติก็จะใส่ในแกงจืดให้ลูกๆกินเป็นประจำ เพราะคนจีนบอกต่อๆกันมาถึงสรรพคุณช่วยฟอกเลือดและขับสารพิษออกจากร่างกายทำ ให้ลูกๆไม่ค่อยเป็นอะไรวันหนึ่งลูกฉันไปตรวจหมอกลับมา บอกว่าเป็นมะเร็ง อีก 2 เดือนต้องไปฉายแสง ฉันจึงลองเอาจิงจูฉ่าย1กำมือมาตำ จะได้น้ำออกมาแก้ว(เล็ก)หนึ่ง ให้เขากินตอนเช้าทุกวัน 2 เดือนถัดมาไปตรวจหมอ หมอถามไปทำอะไรมา ทุกอย่างปกติหมด มะเร็งได้หายไปแล้ว"        หลังจากคุยกันพักหนึ่ง ผมก็ได้ข้อสรุปออกมาว่า คน8คนเป็นมะเร็งในที่ต่างกัน กินน้ำจิงจูฮวยฉ่าย 2-3 เดือน ทุกคนหายหมด(ไม่มีใครต้องทำคีโมเลย) ผมจึงเกิดความเชื่อถือขึ้นมาระดับหนึ่ง        ก่อนหน้านั้น พี่สะใภ้ผม 2 คน (ปี2006) เป็นมะเร็งที่เต้านม,มดลูก คนหนึ่งต้องทำคีโม ใช้วิธี"นั่งสมาธิ"ทุกวัน 8 เดือน หายเป็นปกติ        ต่อมาผมได้ต้นจิงจูฉ่ายมา1 ต้น จากคุณแม่ของลูกศิษย์ จึงรีบนำมาเพาะขยาย ปลายปี 2008 ผมได้รู้ว่าแม่ค้าที่เช่าที่ในร้านผมคนหนึ่งเป็นมะเร็งเต้านมขั้น 3 หน้าคล้ำ เดินตัวแข็งแล้ว ผมรีบเชิญมานั่งคุยเล่าเรื่องจิงจูฉ่ายให้ฟังและเรื่องพี่สะใภ้นั่งสมาธิ ก็เลยแนะนำให้ทำ 2 อย่างควบคู่กันไป เชื่อไหมว่า 3 เดือนถัดไป หายเป็นปลิดทิ้ง (ผมให้เขาไปเพียง1ต้นไปปลูกและ เด็ดใบทานเลยวันละ1ใบ เพราะมีน้อย ก็ยังได้ผล)        ปี 2009 ผมมีต้นจิงจูฉ่ายมากขึ้นและแจกจ่ายให้กับคนรู้จักหลายคน ซึ่งกำลังอยู่ในระหว่าง 3 เดือน ทุกคนอาการดีขึ้นหมด ถ้าเราอยากช่วยผู้ที่เรารักและห่วงใยซึ่งกำลังเผชิญกับโรคร้ายนี้ ผมแนะนำวิธีนี้ ถ้าคุณต้องทำคีโมก็ทำไป กินใบจิงจูฉ่ายและนั่งสมาธิผมว่าคุณมีโอกาสหาย        ผมพยายามหาข้อมูลเพิ่มเติม พบว่าผักชนิดนี้ แพมย์จีนโบราณเรียกว่า"ยาเย็น"เป็น"หยิน"ช่วยฟอกเลือด,ขับสารพิษ..ฆ่าเชื้อ ไวรัสทุกชนิด..เพื่อนku37 คนหนึ่งลองให้ลูกกิน เนื้องอกที่คอก็ยุบลง      เนื่องจากที่ LA.เรามีน้อยมาก ถ้าเกิดใครต้องการมากๆ (จริงๆต้องทานวันละ1กำ)ลองคุยกับคุณน้อยที่สวนคุณน้อย จ.เชียงใหมเขาขายกก.ละ 35 บาท     ใครเป็นมะเร็งลองดูซิครับ ทานแล้วยังไงก็ไม่มีผลข้างเคียง แต่ คุณมีโอกาสหายครับ.ใครที่เคยสั่ง "เกาเหลาเลือดหมู" มาทาน เคยสงสัยกันไหมว่า ในชามเกาเหลาของเราจะมีผักสีเขียวชนิดหนึ่ง ที่ไม่ใช่ผักกาดหอม ขึ้นฉ่าย หรือใบตำลึงใส่ชามมาด้วย หลายคนไม่ทราบว่าเจ้าผักชนิดนี้มีชื่อเรียกว่าอะไร แล้วมีสรรพคุณอย่างไร เอ้า...ใครที่ไม่รู้ ตามกระปุกดอทคอมมาเลยค่ะ         เจ้าผักชนิดนี้เรียกว่า "จิงจูฉ่าย" ค่ะ หรือที่ชาวต่างชาติเรียกว่า "เซเลอรี่" (Celery) เป็นผักสมุนไพรชนิดหนึ่งของจีน มีชื่อวิทยาศาสตร์ว่า Apium graveolens L. ลักษณะต้น "จิงจูฉ่าย" จะเป็นกอคล้ายใบบัวบก สามารถเจริญงอกงามได้ดีในที่ที่มีแสงแดดรำไร ชื้น ดินโปร่งแต่ไม่แฉะ ชอบอากาศเย็นมากกว่าอากาศร้อน คุณค่าทางโภชนาการของ "จิงจูฉ่าย" มีไม่น้อยทีเดียวค่ะ เพราะ "จิงจูฉ่าย" 100 กรัม ให้พลังงาน 392 กิโลแคลอรี ประกอบด้วยสารอาหารนานาชนิด คือ โปรตีน, ไขมัน, คาร์โบไฮเดรต, เส้นใย, แคลเซียม, เหล็ก, ฟอสฟอรัส, วิตามินเอ, วิตามินบี6, วิตามินซี และวิตามินอี         มาที่สรรพคุณทางยากันบ้างดีกว่า จุดเด่นของ "จิงจูฉ่าย" คือมีกลิ่นหอม คล้าย ๆ กับตั้งโอ๋ ยิ่งโดนความร้อนจะยิ่งหอม และยิ่งเพิ่มสรรพคุณมากขึ้น โดยกลิ่นหอมของ "จิงจูฉ่าย" มาจากน้ำมันหอมระเหยที่มีอยู่ในลำต้นและใบนั่นเอง ประกอบด้วยสารไลโมนีน ซิลนีน และสารกลัยโคไซด์ที่มีชื่อว่า อะปิอิน ซึ่ง สารเหล่านี้มีสรรพคุณช่วยปรับสมดุลความดันโลหิต จึงเหมาะกับผู้ที่มีปัญหาเรื่องความดัน แถมยังช่วยขับลมในกระเพาะอาหารและลำไส้ได้ด้วยส่วนต้นสด และเมล็ดของ "จิงจูฉ่าย" มีโซเดียมต่ำ จึงดีต่อผู้ป่วยโรคไต          นอกจากนี้ ในทางการแพทย์เชื่อว่า "จิงจูฉ่าย" เป็นยาเย็น จึงช่วยบำรุงปอด ช่วยฟอกเลือด เลือดลมหมุนเวียนได้สะดวก คนจีนจึงนิยมนำผักชนิดนี้มาปรุงเป็นอาหารรับประทานในหน้าหนาว เพื่อช่วยในเรื่องการไหลเวียนของโลหิต ปรับสมดุลให้ร่างกายได้ดีนั่นเอง ส่วนที่หลายคนสงสัยว่า ทำไมเราจึงมักเห็น "จิงจูฉ่าย" อยู่ในเกาเหลาเลือดหมู นั่นก็เพราะ "จิงจูฉ่าย" มีสรรพคุณช่วยดับกลิ่นคาวเลือดได้ดีด้วยค่ะแต่จริง ๆ แล้ว "จิงจูฉ่าย" ไม่ได้ใช้ทำอาหารได้เพียงแค่ต้มเลือดหมูเท่านั้นนะ เพราะอาหารประเภทแกงจืดทั้งหลาย หรือผัดผัก ผัดฉ่าก็สามารถใช้ "จิงจูฉ่าย" เป็นส่วนผสมที่ลงตัวน่ารับประทานไม่แพ้กันจิงจูฉ่าย ในรูปแบบชา  คัดวัตถุดิบใบจิงจูฉ่ายคุณภาพ  ปลอดสาร ผ่านกระบวนการผลิตที่ได้มาตรฐาน สะอาดปลอดภัย ไม่มีสารปนเปื้อน รสชาติหอมหวานไม่ขม ดื่มง่าย โดยกระบวนการหมัก นวดอย่างพิถีพิถัน และอบด้วยไฟอ่อนเพื่อให้คงคุณค่าทางโภชนาการ และสรรพคุณทางยา  ชาจิงจูฉ่ายจึงมีประโยชน์มากมาย  และเนื่องจากเป็๋นสมุนไพรจึงไม่มีผลข้างเคียงในกรณีที่บริโภคเป็นปริมาณมากอยกได้ข้อมูลเพิ่มเติมจากเพื่อนในเวปค่ะLiked By: PinKenมะเร็งระยะสุดท้าย 2 ปี ไม่เข้ารับรักษา ค่ามะเร็งล…: https://youtu.be/uITAm2Gi5mQ','ผักจิงจูฉ่ายรักษามะเร็งระยะสุดท้ายทำให้ค่ามะเร็งลดลง กรมการแพทย์ โดย สถาบันมะเร็งแห่งชาติ ได้ดำเนินการตรวจสอบข้อเท็จจริงพบว่า น้ำปั่นผักจิงจูฉ่ายไม่สามารถนำมาใช้รักษามะเร็งระยะสุดท้ายในมนุษย์ได้','เตือน!!ข่าวปลอม “น้ำปั่นผักจิงจูฉ่าย"รักษามะเร็งระยะสุดท้าย','สถาบันมะเร็งแห่งชาติ แจ้งเตือนข่าวปลอม “น้ำปั่นผักจิงจูฉ่ายรักษามะเร็งระยะสุดท้าย" ยืนยันไม่สามารถนำมาใช้รักษามะเร็งระยะสุดท้ายในมนุษย์ได้']

**Step 2: Tokenize**

In [77]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['จิง', 'จู', 'ฉ่า', 'ย', 'ผัก', 'มหัศจรรย์', 'รักษา', 'มะเร็ง', '—', '—', '—', '—', '—', '—', '—', '—', '—', '—', '—', '—', 'ผม', 'นิ', 'กกี้', 'อิทธิ', 'เกษม', 'KU37', 'นายก', 'สมา', 'คมม', 'เกษตรฯ', 'อเมริกา', 'เล่า', 'ประสบการณ์', 'สัมผัส', 'ตัว', 'ปี', 'case', 'ตัวอย่าง', 'ทางออก', 'ป่วย', 'โรค', 'มะเร็ง', 'ผม', 'ร้าน', 'อาหาร', 'เขต', 'N.Hooywood', 'กลาง', 'ปี', 'ผม', 'โอกาส', 'เจอ', 'ลูก', 'ศิษย์', 'เก่า', 'เรียน', 'แจก', 'ไพ่', 'ร้าน', 'ถาม', 'ทุกข์', 'สุข', 'รู้', 'มะเร็ง', 'คอ', 'มะเร็ง', 'ต่อม', 'ลูก', 'หมาก', 'หาย', 'ปลิด', 'ทิ้ง', 'พี่', 'น้อง', 'ญาติ', 'คน', 'มะเร็ง', 'เวลา', 'ปี', 'คน', 'หาย', 'ผม', 'กระตือรือร้น', 'ถาม', 'วิธี', 'รักษา', 'แม่', 'อธิบาย', 'ฟัง', 'ต้น', 'จิง', 'จู', 'ฉ่า', 'ยอ', 'ยู่', 'สวน', 'บ้าน', 'ปกติ', 'ใส่', 'แกงจืด', 'ลูกๆ', 'กิน', 'ประจำ', 'คน', 'จีน', 'ต่อๆ', 'สรรพคุณ', 'ฟอก', 'เลือด', 'ขับ', 'สาร', 'พิษ', 'ร่างกาย', 'ทำ', 'ลูกๆ', 'ลูก', 'ตรวจ', 'หมอก', 'ลับ', 'มะเร็ง', 'เดือน', 'ฉาย', 'แสง', 'ลอง', 'จิง', 'จู', 'ฉ่า', 'ย', 'กำ'

In [78]:
tokens_list = [split_word(txt) for txt in clean_text]

In [79]:
tokens_list

[['จิง',
  'จู',
  'ฉ่า',
  'ย',
  'ผัก',
  'มหัศจรรย์',
  'รักษา',
  'มะเร็ง',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  '—',
  'ผม',
  'นิ',
  'กกี้',
  'อิทธิ',
  'เกษม',
  'KU37',
  'นายก',
  'สมา',
  'คมม',
  'เกษตรฯ',
  'อเมริกา',
  'เล่า',
  'ประสบการณ์',
  'สัมผัส',
  'ตัว',
  'ปี',
  'case',
  'ตัวอย่าง',
  'ทางออก',
  'ป่วย',
  'โรค',
  'มะเร็ง',
  'ผม',
  'ร้าน',
  'อาหาร',
  'เขต',
  'N.Hooywood',
  'กลาง',
  'ปี',
  'ผม',
  'โอกาส',
  'เจอ',
  'ลูก',
  'ศิษย์',
  'เก่า',
  'เรียน',
  'แจก',
  'ไพ่',
  'ร้าน',
  'ถาม',
  'ทุกข์',
  'สุข',
  'รู้',
  'มะเร็ง',
  'คอ',
  'มะเร็ง',
  'ต่อม',
  'ลูก',
  'หมาก',
  'หาย',
  'ปลิด',
  'ทิ้ง',
  'พี่',
  'น้อง',
  'ญาติ',
  'คน',
  'มะเร็ง',
  'เวลา',
  'ปี',
  'คน',
  'หาย',
  'ผม',
  'กระตือรือร้น',
  'ถาม',
  'วิธี',
  'รักษา',
  'แม่',
  'อธิบาย',
  'ฟัง',
  'ต้น',
  'จิง',
  'จู',
  'ฉ่า',
  'ยอ',
  'ยู่',
  'สวน',
  'บ้าน',
  'ปกติ',
  'ใส่',
  'แกงจืด',
  'ลูกๆ',
  'กิน',
  'ประจำ',
  'คน

**Step 3 + 4a: Bag of words + count word**

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [81]:
cvec.vocabulary_

{'จิง': 59,
 'จู': 62,
 'ฉ่า': 65,
 'ย': 164,
 'ผัก': 144,
 'มหัศจรรย์': 157,
 'รักษา': 176,
 'มะเร็ง': 159,
 '—': 344,
 'ผม': 141,
 'นิ': 109,
 'กกี้': 16,
 'อิทธิ': 248,
 'เกษม': 255,
 'KU37': 3,
 'นายก': 108,
 'สมา': 210,
 'คมม': 42,
 'เกษตรฯ': 254,
 'อเมริกา': 251,
 'เล่า': 292,
 'ประสบการณ์': 124,
 'สัมผัส': 220,
 'ตัว': 84,
 'ปี': 138,
 'case': 9,
 'ตัวอย่าง': 85,
 'ทางออก': 98,
 'ป่วย': 140,
 'โรค': 326,
 'ร้าน': 185,
 'อาหาร': 247,
 'เขต': 258,
 'N.Hooywood': 7,
 'กลาง': 25,
 'โอกาส': 328,
 'เจอ': 262,
 'ลูก': 198,
 'ศิษย์': 205,
 'เก่า': 257,
 'เรียน': 287,
 'แจก': 306,
 'ไพ่': 336,
 'ถาม': 95,
 'ทุกข์': 102,
 'สุข': 225,
 'รู้': 181,
 'คอ': 45,
 'ต่อม': 89,
 'หมาก': 232,
 'หาย': 239,
 'ปลิด': 133,
 'ทิ้ง': 101,
 'พี่': 150,
 'น้อง': 112,
 'ญาติ': 74,
 'คน': 41,
 'เวลา': 294,
 'กระตือรือร้น': 21,
 'วิธี': 204,
 'แม่': 314,
 'อธิบาย': 242,
 'ฟัง': 153,
 'ต้น': 92,
 'ยอ': 167,
 'ยู่': 170,
 'สวน': 215,
 'บ้าน': 119,
 'ปกติ': 120,
 'ใส่': 331,
 'แกงจืด': 301,
 'ลูกๆ': 199,
 'กิน'

In [82]:
c_feat[:,:].todense()

matrix([[ 1,  1,  1, ...,  1, 12,  1],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

In [83]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[ 1  1  1 ...  1 12  1]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [84]:
first_news.shape[1]

346

In [85]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 262
This is lowerpart : 502.46790942307945


0.521426334073397

In [86]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 117
This is lowerpart : 237.8844257197179


0.4918354770221598

In [87]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 146
This is lowerpart : 367.34588605291333


0.39744558342207714

# ข่าวที่ 8

In [88]:
clean_text = ['ในที่สุด ความจริงก็ถูกเปิดเผย โดย ผู้ตัดต่อพันธุกรรมเชื้อโควิด 19 เอง...**************โควิด19 มาจากฝีมือมนุษย์มีแหล่งที่มาจากห้องแลป ไวรัส P3รัฐคาโรไลน่าเหนือของอเมริกา!!!นาย Greg Roubini ผู้เชี่ยวชาญด้านข่าวกรองชื่อดังของอเมริกาให้สัมภาษณ์ของผู้สื่อข่าวช่องข่าวทีวีที่1 ของอเมริกาได้เป็นผู้เผยความลับนี้นาย Greg เผยว่า ไวรัสโควิด19 ได้รับการออกแบบทางพันธุกรรมเพื่อใช้เป็นอาวุธชีวภาพ- มีแหล่งที่มาจากห้องแลป BSL-3 รัฐ คาโรไลน่าเหนือ พัฒนาโดย ศาสตราจารย์ราล์ฟ บาร์ริก- พร้อมกันนั้น เขาระบุว่า ไวรัสถูก “รัฐบาลมืด” จากรัฐคาโรไลน่าเหนือส่งไปแพร่ระบาดในประเทศจีน อิตาลี และอเมริกาทั้งประเทศ##..ก่อนหน้านี้ในวันที่ 15 มีนาคม นายเกรก ก็ได้ทวิตข้อความถามนายทรัมป์ว่า- เหตุใดจึงไม่บอกประชาชนอเมริกาว่า ไวรัสผลิตจากอเมริกา? ทำไมไม่อธิบายให้ชัดเจนว่าตัวไวรัสเองแท้จริงแล้วคืออาวุธชีวภาพ?**บังเอิญ ศาสตราจารย์ Luc Montanier ผู้ได้รับรางวัลโนเบลเนื่องจากเป็นผู้ค้นพบไวรัสเอชไอวีได้เปิดเผยกับนักข่าวชาวฝรั่งเศสเมื่อไม่กี่วันที่ผ่านมาว่า- โควิด19 ไม่ใช่มาจากธรรมชาติ หากแต่ได้รับการพัฒนาอย่างประณีตโดยนักวิทยาศาสตร์ชีวโมเลกุล***ศาสตราจารย์ Luc Montanier ยืนยันว่า เป็นเรื่องเด่นชัดที่เจ้าหน้าที่ผู้เชี่ยวชาญได้นำเชื้อไวรัสที่มาจากค้างคาวเข้าไปเพิ่มความเข้มข้นของเชื้อเอชไอวีเข้าไปด้วย- นี่คือ การวางยาพิษที่ชั่วร้ายที่สุดที่ไม่เคยมีมาก่อนในประวัติศาสตร์ของโลก***นั่นคือการแพร่ระบาดของไวรัสโควิด19 สุดโหด ข่าวเกี่ยวกับ “เชื้อโควิด19 เป็นอาวุธชีวภาพที่มาจากการตัดต่อพันธุกรรมโดยฝีมือมนุษย์” มาโดยตลอด***นักวิทยาศาสตร์ทั่วโลกพยายามทำงานหาแหล่งที่มาของเชื้อไวรัสโดยนักวิทยาศาสตร์อินเดียค้นพบว่า เชื้อไวรัสโคโรน่าสายพันธ์ุใหม่มีเชื้อเอชไอวีแทรกอยู่ด้วย นี่แสดงให้เห็นว่าไวรัสตัวนี้มาจากการตัดต่อทางพันธุกรรม***กลางเดือนมีนาคม นักวิทยาศาสตร์ได้วิเคราะห์พบว่าเชื้อไวรัสโควิด19จากผู้ป่วยรายหนึ่งในรัฐวอชิงตันพบว่าวัฏจักรวิวัฒนาการของมันมียาวนานกว่าครึ่งปีมาแล้ว พร้อมๆกับการศึกษาลึกซึ้งลงไปว่า ประเทศต่างๆในโลกไม่น้อยได้เบนสายตาแห่งความสงสัยไปที่อเมริกา ประเทศต่างๆ ทั้งญี่ปุ่น อิตาลี ออสเตรเลีย ล้วนมีผู้ป่วยทียืนยันว่ามีแหล่งที่มาจากอเมริกาทั้งสิ้น*** ในเวลาต่อมา ROBERT REDFIELD ผู้อำนวยการศูนย์ควบคุมและป้องกันโรคแห่งสหรัฐอเมริกายอมรับว่า ผู้ป่วยตายจากไข้หวัดใหญ่ในเดือนกันยายน 2019 มีอยู่ไม่น้อยที่ตายจากเชื้อไวรัสโควิด19 นี้- ต่อปัญหานี้โฆษกกระทรวงต่างประเทศของจีน นายจ้าวลี่เจียงได้ทวิตข้อความในทวิตเตอร์ถามผู้อำนวยการศูนย์ควบคุมและป้องกันโรคแห่งสหรัฐอเมริกา ว่าผู้ป่วยรายแรกของอเมริกาเกิดขึ้นตอนไหน? ชื่ออะไร? อยู่โรงพยาบาลอะไร? และเป็นไปได้อย่างมากที่ทหารอเมริกานำเชื้อมาแพร่ที่อู่ฮั่น.>>>>อเมริกาต้องโปร่งใส ต้องเปิดเผยข้อมูลนี้ให้โลกได้รู้ความจริง**ด้วยความพยายามอย่างสุดความสามารถของคณะผู้สื่อข่าวคณะหนึ่งแห่งรัฐเวอร์จิเนีย ในที่สุดก็ได้ตามหาผู้ป่วยรายแรกจนพบ นั่นก็คือ ทหารอเมริกาที่เข้าร่วมการแข่งขันกีฬาทหารที่อู่ฮั่นของจีนในเดือนตุลาคม 2019 นางมีชื่อว่า "Maatje Benassi">>>นายทหารหญิงของอเมริกาคนนี้มีภูมิหลังพิเศษตรงที่นางมีความเกี่ยวข้องกับห้องปฏิบัติการชีวเคมี P4 ของนาย FORT DETRICK*** คนในครอบครัวก็มีหลายคนที่ยืนยันว่าผู้ติดเชื้อในจำนวนนี้มีอยู่คนหนึ่งเป็นผู้ป่วยที่ติดเชื้อรายแรกในฮอลแลนด์ ก่อนติดเชื้อเขาเคยไปในเขตพื้นที่ลอมบาร์เดียของอิตาลี ทำให้เขตพื้นที่นั้นเกิดการแพร่ระบาดของเชื้อโควิด19***มาถึงตรงนี้ หลักฐานเกี่ยวกับเชื้อไวรัสโควิด19 มีต้นกำเนิดจากอเมริกาอย่างแน่นอน มีห่วงโซ่เชื่อมร้อยอย่างครบถ้วน ทหารพิเศษ 5 คนที่อเมริกาส่งเครื่องบินมารับกลับไปภายหลังการแพร่ระบาดของไวรัสและห้องแลปที่ถูกปิดตาย ก็สามารถนำมาปะติดปะต่อกันได้แล้วหากว่ากันตามตรรกะของนายทรัมป์ เราก็สามารถเรียกเชื้อโควิด19 เป็น "ไวรัสนอร์ธคาโรไลนา" (Virus North Carolina) หรือ "ไวรัสอเมริกา"***ในขณะที่หลักฐานทั้งหมดต่างชี้ไปที่อเมริกาเจ้าหน้าที่ชั้นสูงของอเมริกายอมรับอย่างเปิดเผยว่า เชื้อโควิด19 ไม่จัดอยู่ในชั้นของโรคระบาด แต่จัดอยู่ในชั้นของอาวุธชีวภาพ>>>#”ความไร้ยางอายทำให้โลกตะลึงและได้เพิ่มข้อน่าสงสัยว่าอเมริกาเป็นผู้วางยาพิษคนทั้งโลก. เพื่อขายวัคซีนป้องกันมาแก้ปัญหาเศรษฐกิจที่ขาดดุลการค้า”>>>เรื่องทั้งหมดได้ปรากฏชัดเจนแล้ว แต่ทว่าทรัมป์ยังพยายามโยนบาปอย่างไม่คิดชีวิตให้จีนรับเคราะห์แทนอย่าง น่ารังเกลียดที่สุด***เชื้อโควิด19 ได้ก่อให้เกิดภัยพิบัติและความสูญเสียที่ยากจะประเมินได้ บาปนี้มันใหญ่หลวงเกินกว่าจะโยนออกไปแล้วโทษคนอื่น***ยังมีข้อน่าสงสัยที่นายเกรกได้ตีแผ่ออกมา นายราล์ฟ บาร์ริค ผู้รับผิดชอบพัฒนาไวรัส รัฐคาโรไลนาเหนือคนนี้เป็นใคร*** นาย บาร์ริคมาจากมหาวิทยาลัยคาโรไลนาเหนือ เขาเป็นหัวหน้านักไวรัสวิทยาที่เปลี่ยนโฉมใหม่ของโรคซาร์สโคโรนาไวรัสโดยการตัดต่อยีนในปี 2015- และเขายังเป็นผู้นำในการวิจัยและพัฒนาไวรัสดังกล่าวอีกด้วย ที่น่าตกใจก็คือ เขาเป็นบุคคลที่รับผิดชอบด้านการพัฒนาทางคลินิกของยาวิเศษ "RADEXIVIR" เป็นไปอย่างที่โบราณว่าไว้ คนที่วางยาพิษก่อนอื่นต้องเตรียม# ยาแก้พิษไว้ก่อนเสมอ!!!!- ยา RIDESIVIR ภายหลังจากปฏิบัติการทางคลินิกและถูกตั้งข้อสงสัยโดยผู้เชี่ยวชาญเกี่ยวกับประสิทธิภาพและความปลอดภัยของมันจึงทำให้ตกกระป๋องไปพร้อมๆกับการแพร่ระบาดที่ลุกลามออกไปทั่วโลก***อเมริกากลายเป็น “ศูนย์กลางการล้างโลก” ไปแล้ว- การแพร่ระบาดในช่วงแรกของอเมริกา ประธานาธิบดีทรัมป์ไม่ได้ให้ความสาคัญกับมันเลยโดยมองว่าเป็นไข้หวัดใหญ่ที่หนักกว่าปกติเท่านั้นเพราะรู้อยู่แก่ใจว่าคนของตนเองผลิตมันขึ้นมาจนกระทั่งเพื่อนรักของเขาคือ "นายสแตนลี่ย์ เชล่า" เจ้าพ่อวงการอสังหาริมทรัพย์ที่ยิ่งใหญ่แห่งนิวยอร์กเสียชีวิตจากเชื้อไวรัสโควิด19>>>>ถึงเวลานี้จีนได้ฟ้องร้องต่อศาลโลกว่า อเมริกาเป็นต้นเหตุในการแพร่เชื้อไวรัสโควิด 19 อย่างตั้งใจเพื่อทำลายล้างจีนและ ปชช ทั่วโลก***>>>ตอนนี้คงต้องรอดูการสืบสวนของศาลโลกว่าจะตัดสินออกมาเช่นไร? ซึ่งถึง ณ เวลานี้ ทรัมป์เริ่มรู้สึกตัวและให้ความสาคัญในระดับสูง #แต่ว่าสายไปเสียแล้ว!!!**Ny Ny*','อ้างว่า ดร. บาริค เป็นผู้พัฒนาเชื้อนี้เป็นอาวุธชีวภาพ แต่ความจริงแค่มีงานวิจัยศึกษาว่าเชื้อโคโรนาติดต่อจากค้างคาวมาสู่คน การความพยายามอ้างเรื่อง Fort Detrick ในทางที่ไม่มีหลักฐานสนับสนุนชัดเจน','โควิด-19 : จีนกับความพยายามแพร่ข่าวปลอมเรื่องโควิดมีต้นกำเนิดในสหรัฐฯ','การระดมเผยแพร่ข้อมูลเท็จที่กล่าวหาว่าเชื้อโรคโควิด-19 มีต้นกำเนิดมาจากฐานทัพทหารในรัฐแมริแลนด์ กำลังได้รับความสนใจอย่างมากในจีน ก่อนหน้าที่หน่วยงานด้านข่าวกรองของสหรัฐฯ จะเปิดเผยรายงานการสอบสวนเรื่องต้นกำเนิดของเชื้อไวรัสชนิดนี้']

**Step 2: Tokenize**

In [89]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เผย', 'ตัด', 'พันธุกรรม', 'เชื้อ', 'โค', 'วิด', 'โค', 'วิด', 'ฝีมือ', 'มนุษย์', 'แหล่ง', 'ที่มา', 'ห้อง', 'แลป', 'ไวรัส', 'P3', 'รัฐ', 'คา', 'โร', 'ไล', 'เหนือ', 'อเมริกา', 'Greg', 'Roubini', 'เชี่ยวชาญ', 'ข่าว', 'กรอง', 'ชื่อ', 'อเมริกา', 'สัมภาษณ์', 'สื่อ', 'ข่าว', 'ช่อง', 'ข่าว', 'ทีวี', 'อเมริกา', 'เผย', 'ลับ', 'Greg', 'เผย', 'ไวรัส', 'โค', 'วิด', 'ออกแบบ', 'พันธุกรรม', 'อาวุธ', 'ชีวภาพ', 'แหล่ง', 'ที่มา', 'ห้อง', 'แลป', 'BSL', 'รัฐ', 'คา', 'โร', 'ไล', 'เหนือ', 'พัฒนา', 'ศาสตรา', 'จาร', 'ย์', 'ราล์ฟ', 'บาร์', 'ริก', 'ระบุ', 'ไวรัส', 'รัฐบาล', 'มืด', 'รัฐ', 'คา', 'โร', 'ไล', 'เหนือ', 'แพร่', 'ระบาด', 'ประเทศ', 'จีน', 'อิตาลี', 'อเมริกา', 'ประเทศ', 'หน้า', 'มีนาคม', 'เก', 'รก', 'ทวิ', 'ต', 'ข้อความ', 'ถาม', 'ทรัมป์', 'ใด', 'ประชาชน', 'อเมริกา', 'ไวรัส', 'ผลิต', 'อเมริกา', 'อธิบาย', 'ชัดเจน', 'ตัว', 'ไวรัส', 'อาวุธ', 'ชีวภาพ', 'บังเอิญ', 'ศาสตราจารย์', 'Luc', 'Montanier', 'รางวัล', 'โน', 'เบล', 'ค้น', 'ไว', 'รัส', 'เอ', 'ช', 'ไอ', 'วี', 'เผย', 'ข่าว', 'ฝรั่งเศส', 'ก

In [90]:
tokens_list = [split_word(txt) for txt in clean_text]

In [91]:
tokens_list

[['เผย',
  'ตัด',
  'พันธุกรรม',
  'เชื้อ',
  'โค',
  'วิด',
  'โค',
  'วิด',
  'ฝีมือ',
  'มนุษย์',
  'แหล่ง',
  'ที่มา',
  'ห้อง',
  'แลป',
  'ไวรัส',
  'P3',
  'รัฐ',
  'คา',
  'โร',
  'ไล',
  'เหนือ',
  'อเมริกา',
  'Greg',
  'Roubini',
  'เชี่ยวชาญ',
  'ข่าว',
  'กรอง',
  'ชื่อ',
  'อเมริกา',
  'สัมภาษณ์',
  'สื่อ',
  'ข่าว',
  'ช่อง',
  'ข่าว',
  'ทีวี',
  'อเมริกา',
  'เผย',
  'ลับ',
  'Greg',
  'เผย',
  'ไวรัส',
  'โค',
  'วิด',
  'ออกแบบ',
  'พันธุกรรม',
  'อาวุธ',
  'ชีวภาพ',
  'แหล่ง',
  'ที่มา',
  'ห้อง',
  'แลป',
  'BSL',
  'รัฐ',
  'คา',
  'โร',
  'ไล',
  'เหนือ',
  'พัฒนา',
  'ศาสตรา',
  'จาร',
  'ย์',
  'ราล์ฟ',
  'บาร์',
  'ริก',
  'ระบุ',
  'ไวรัส',
  'รัฐบาล',
  'มืด',
  'รัฐ',
  'คา',
  'โร',
  'ไล',
  'เหนือ',
  'แพร่',
  'ระบาด',
  'ประเทศ',
  'จีน',
  'อิตาลี',
  'อเมริกา',
  'ประเทศ',
  'หน้า',
  'มีนาคม',
  'เก',
  'รก',
  'ทวิ',
  'ต',
  'ข้อความ',
  'ถาม',
  'ทรัมป์',
  'ใด',
  'ประชาชน',
  'อเมริกา',
  'ไวรัส',
  'ผลิต',
  'อเมริกา',
  'อธิบาย',
  'ชัดเจน',


**Step 3 + 4a: Bag of words + count word**

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [93]:
cvec.vocabulary_

{'เผย': 267,
 'ตัด': 82,
 'พันธุกรรม': 138,
 'เชื้อ': 259,
 'โค': 287,
 'วิด': 194,
 'ฝีมือ': 134,
 'มนุษย์': 148,
 'แหล่ง': 286,
 'ที่มา': 101,
 'ห้อง': 232,
 'แลป': 285,
 'ไวรัส': 310,
 'P3': 13,
 'รัฐ': 166,
 'คา': 46,
 'โร': 298,
 'ไล': 308,
 'เหนือ': 274,
 'อเมริกา': 242,
 'Greg': 7,
 'Roubini': 19,
 'เชี่ยวชาญ': 257,
 'ข่าว': 32,
 'กรอง': 21,
 'ชื่อ': 66,
 'สัมภาษณ์': 214,
 'สื่อ': 219,
 'ช่อง': 67,
 'ทีวี': 100,
 'ลับ': 182,
 'ออกแบบ': 235,
 'อาวุธ': 237,
 'ชีวภาพ': 62,
 'BSL': 0,
 'พัฒนา': 137,
 'ศาสตรา': 202,
 'จาร': 50,
 'ย์': 157,
 'ราล์ฟ': 171,
 'บาร์': 109,
 'ริก': 173,
 'ระบุ': 163,
 'รัฐบาล': 167,
 'มืด': 152,
 'แพร่': 282,
 'ระบาด': 162,
 'ประเทศ': 120,
 'จีน': 52,
 'อิตาลี': 239,
 'หน้า': 224,
 'มีนาคม': 151,
 'เก': 246,
 'รก': 158,
 'ทวิ': 96,
 'ต': 76,
 'ข้อความ': 35,
 'ถาม': 93,
 'ทรัมป์': 95,
 'ใด': 304,
 'ประชาชน': 115,
 'ผลิต': 130,
 'อธิบาย': 233,
 'ชัดเจน': 58,
 'ตัว': 84,
 'บังเอิญ': 106,
 'ศาสตราจารย์': 203,
 'Luc': 8,
 'Montanier': 10,
 'รางวัล': 169,
 'โน':

In [94]:
c_feat[:,:].todense()

matrix([[ 1,  1,  1, ...,  2, 21,  3],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  1,  0]], dtype=int64)

In [95]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[ 1  1  1 ...  2 21  3]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  1  0]]


In [96]:
first_news.shape[1]

312

In [97]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 75
This is lowerpart : 310.33691369219997


0.24167282940239235

In [98]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 86
This is lowerpart : 230.51247254758255


0.3730817645116701

In [99]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 151
This is lowerpart : 369.0


0.4092140921409214

# ข่าวที่ 9

In [100]:
clean_text = ['คลัง ขยายมาตรการช่วยค่าน้ำ ค่าไฟ ให้ผู้ถือบัตรคนจน','มาตรการบรรเทาภาระค่าไฟฟ้าและค่าน้ำประปา มีวัตถุประสงค์เพื่อบรรเทาภาระค่าใช้จ่ายด้านสาธารณูปโภคขั้นพื้นฐาน ได้แก่ ค่าไฟฟ้าและค่าน้ำประปาให้แก่ผู้มีบัตรสวัสดิการแห่งรัฐ (บัตรฯ) อย่างต่อเนื่องและครอบคลุมผู้มีบัตรฯ ให้สามารถได้รับประโยชน์จากมาตรการบรรเทาภาระค่าไฟฟ้าและค่าน้ำประปา (มาตรการฯ) ดังกล่าวให้มากขึ้น','เช็กเลย!เปิดเกณฑ์ลดค่าน้ำ-ค่าไฟผู้ถือบัตรคนจน เผยช่องทางลงทะเบียน','เช็กก่อนพลาดโอกาส! เปิดเกณฑ์ผู้ถือบัตรคนจน ได้ลดค่าน้ำ-ค่าไฟ 12 เดือน พร้อมช่องทางลงทะเบียน']

**Step 2: Tokenize**

In [101]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['คลัง', 'ขยาย', 'มาตรการ', 'ค่าน้ำ', 'ค่า', 'ไฟ', 'บัตร', 'คนจน']
tokenized text:
 ['มาตรการ', 'บรรเทา', 'ภาระ', 'ค่า', 'ไฟฟ้า', 'ค่าน้ำ', 'ประปา', 'วัตถุประสงค์', 'บรรเทา', 'ภาระ', 'ค่า', 'จ่าย', 'สาธารณูปโภค', 'พื้น', 'ฐาน', 'ค่า', 'ไฟฟ้า', 'ค่าน้ำ', 'ประปา', 'บัตร', 'สวัสดิการ', 'รัฐ', 'บัตรฯ', 'เนื่อง', 'ครอบคลุม', 'บัตรฯ', 'ประโยชน์', 'มาตรการ', 'บรรเทา', 'ภาระ', 'ค่า', 'ไฟฟ้า', 'ค่าน้ำ', 'ประปา', 'มาตร', 'การฯ']
tokenized text:
 ['เช็ก', 'เกณฑ์', 'ลด', 'ค่าน้ำ', 'ค่า', 'ไฟ', 'บัตร', 'คนจน', 'เผย', 'ช่อง', 'ทะเบียน']
tokenized text:
 ['เช็', 'กก่อ', 'นพ', 'ลาด', 'โอกาส', 'เกณฑ์', 'บัตร', 'คนจน', 'ลด', 'ค่าน้ำ', 'ค่า', 'ไฟ', 'เดือน', 'ช่อง', 'ทะเบียน']


In [102]:
tokens_list = [split_word(txt) for txt in clean_text]

In [103]:
tokens_list

[['คลัง', 'ขยาย', 'มาตรการ', 'ค่าน้ำ', 'ค่า', 'ไฟ', 'บัตร', 'คนจน'],
 ['มาตรการ',
  'บรรเทา',
  'ภาระ',
  'ค่า',
  'ไฟฟ้า',
  'ค่าน้ำ',
  'ประปา',
  'วัตถุประสงค์',
  'บรรเทา',
  'ภาระ',
  'ค่า',
  'จ่าย',
  'สาธารณูปโภค',
  'พื้น',
  'ฐาน',
  'ค่า',
  'ไฟฟ้า',
  'ค่าน้ำ',
  'ประปา',
  'บัตร',
  'สวัสดิการ',
  'รัฐ',
  'บัตรฯ',
  'เนื่อง',
  'ครอบคลุม',
  'บัตรฯ',
  'ประโยชน์',
  'มาตรการ',
  'บรรเทา',
  'ภาระ',
  'ค่า',
  'ไฟฟ้า',
  'ค่าน้ำ',
  'ประปา',
  'มาตร',
  'การฯ'],
 ['เช็ก',
  'เกณฑ์',
  'ลด',
  'ค่าน้ำ',
  'ค่า',
  'ไฟ',
  'บัตร',
  'คนจน',
  'เผย',
  'ช่อง',
  'ทะเบียน'],
 ['เช็',
  'กก่อ',
  'นพ',
  'ลาด',
  'โอกาส',
  'เกณฑ์',
  'บัตร',
  'คนจน',
  'ลด',
  'ค่าน้ำ',
  'ค่า',
  'ไฟ',
  'เดือน',
  'ช่อง',
  'ทะเบียน']]

**Step 3 + 4a: Bag of words + count word**

In [104]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [105]:
cvec.vocabulary_

{'คลัง': 5,
 'ขยาย': 2,
 'มาตรการ': 21,
 'ค่าน้ำ': 7,
 'ค่า': 6,
 'ไฟ': 35,
 'บัตร': 14,
 'คนจน': 3,
 'บรรเทา': 13,
 'ภาระ': 19,
 'ไฟฟ้า': 36,
 'ประปา': 16,
 'วัตถุประสงค์': 25,
 'จ่าย': 8,
 'สาธารณูปโภค': 27,
 'พื้น': 18,
 'ฐาน': 10,
 'สวัสดิการ': 26,
 'รัฐ': 22,
 'บัตรฯ': 15,
 'เนื่อง': 32,
 'ครอบคลุม': 4,
 'ประโยชน์': 17,
 'มาตร': 20,
 'การฯ': 1,
 'เช็ก': 30,
 'เกณฑ์': 28,
 'ลด': 23,
 'เผย': 33,
 'ช่อง': 9,
 'ทะเบียน': 11,
 'เช็': 29,
 'กก่อ': 0,
 'นพ': 12,
 'ลาด': 24,
 'โอกาส': 34,
 'เดือน': 31}

In [106]:
c_feat[:,:].todense()

matrix([[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 4, 3, 1, 0, 1, 0, 0, 3, 1, 2, 3, 1, 1, 3, 1,
         2, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 3],
        [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0]], dtype=int64)

In [107]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0]
 [0 1 0 0 1 0 4 3 1 0 1 0 0 3 1 2 3 1 1 3 1 2 1 0 0 1 1 1 0 0 0 0 1 0 0 0
  3]
 [0 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1
  0]
 [1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 1 1
  0]]


In [108]:
first_news.shape[1]

37

In [109]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 10
This is lowerpart : 25.612496949731398


0.39043440472151514

In [110]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 5
This is lowerpart : 9.38083151964686


0.5330017908890261

In [111]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 5
This is lowerpart : 10.954451150103324


0.45643546458763834

# ข่าวที่ 10

In [112]:
clean_text = ['''ช็อก! Bill Gates เรียกร้องให้ถอนวัคซีน Covid-19 ทั้งหมด "วัคซีนอันตรายกว่าที่คิด"โดย W. Gellesวอชิงตัน- (MaraviPost) - ในการประกาศที่น่าตกใจ Bill Gates มหาเศรษฐีผู้ร่วมก่อตั้ง Microsoft และกำลังสำคัญที่อยู่เบื้องหลังวัคซีน COVID-19 เรียกร้องให้มีการนำวัคซีนจากพันธุกรรม COVID-19 ทั้งหมดออกจากตลาดทันทีในการกล่าวสุนทรพจน์ทางโทรทัศน์ที่เจ็บปวดรวดร้าวบ่อยครั้ง 19 นาที เกตส์กล่าวว่า “เราทำผิดพลาดอย่างมหันต์ เราต้องการปกป้องผู้คนจากไวรัสอันตราย แต่กลับกลายเป็นว่าไวรัสมีอันตรายน้อยกว่าที่เราคิดไว้มาก และวัคซีนก็อันตรายเกินกว่าที่ใครจะจินตนาการได้”โดย W. Gelles“วัคซีนเหล่านี้—ไฟเซอร์, โมเดอร์นา, จอห์นสัน แอนด์ จอห์นสัน, แอสตร้าเซเนก้า—พวกมันกำลังฆ่าคนทั้งซ้ายและขวา—และพวกเขากำลังทำร้ายคนบางคนอย่างเลวร้ายมาก” เกตส์กล่าวต่อ โดยโบกมือขึ้นไปในอากาศเป็นครั้งคราวเพื่อให้ได้ผลลัพธ์ที่น่าทึ่ง“ข้อมูลของรัฐบาลเองแสดงให้เราเห็นว่านี่คือสิ่งที่กำลังเกิดขึ้น ระบบการรายงานของ CDC กำลังแสดงอะไร…ประมาณ 13,000 ผู้เสียชีวิตในสหรัฐอเมริกาและเหตุการณ์ไม่พึงประสงค์กว่าครึ่งล้าน เราทุกคนรู้ดีว่าระบบการรายงานเป็นเรื่องหลอกลวง“เราทราบดีว่า VAERS [ระบบการรายงานเหตุการณ์ไม่พึงประสงค์จากวัคซีนของศูนย์ควบคุมและป้องกันโรค] รวบรวมได้เพียงร้อยละหนึ่งของสิ่งที่เกิดขึ้น เรากำลังพูดถึงผู้เสียชีวิตจากวัคซีนโควิดมากกว่าล้านราย และผู้คนกว่า 60 ล้านคนที่มีผลข้างเคียงที่ไม่ดี”นี่ไม่ใช่สิ่งที่เราต้องการ สิ่งนี้ไม่เป็นที่ยอมรับ” นายเกทส์ยืนยันหุ้นในวอลล์สตรีทของบริษัทวัคซีนโควิดรายใหญ่ทั้งหมดร่วงลง 20% ถึง 30% เนื่องจากนายเกตส์ประกาศว่าเขาเข้าร่วมในคำร้องเร่งด่วนของพลเมืองที่ยื่นโดยองค์กรป้องกันสุขภาพเด็กของโรเบิร์ต เอฟ. เคนเนดี จูเนียร์ ที่เรียกร้องให้มีอาหารสหรัฐฯ และ สธ. ถอนวัคซีนโควิดทั้งหมดออกจากตลาดทันทีGates กล่าวต่อว่า “ผู้คนจำนวนมากที่รับวัคซีนเหล่านี้เสียชีวิต… หนึ่งวัน สองวัน ห้าวันหลังจากได้รับการฉีดวัคซีน คนอื่นๆ มีอาการอัมพาต ตาบอด ชัก หัวใจวาย ระบบภูมิคุ้มกันล้มเหลว ลิ่มเลือดอุดตัน สมองอักเสบ ปอดหรือไตถูกทำลาย การแท้งบุตร โรคภูมิต้านตนเอง ความล้มเหลวของระบบอวัยวะหลายส่วน ความเหนื่อยล้าถาวรอย่างถาวร และปัญหาร้ายแรงอื่นๆ อีกมากมาย“แน่นอน Media Mouthpieces ของเรา—ฉันหมายถึงสื่อกระแสหลัก ละเลยโศกนาฏกรรมเหล่านี้ทั้งหมดว่าเป็น 'แค่เรื่องบังเอิญ'”“เหตุผลที่พวกเขาพูดอย่างนั้น” เกทส์อธิบาย “เป็นเพราะสิ่งที่ฉันทำที่งาน 201 ซึ่งเป็นการจำลองสถานการณ์แพร่ระบาดของไวรัสโคโรน่าที่นิวยอร์กในเดือนตุลาคม 2019 เพียงไม่กี่สัปดาห์ก่อนที่เราจะประกาศการแพร่ระบาดที่เกิดขึ้นจริง ฉันได้รับหนังสือพิมพ์ ช่องทีวี และสถานีวิทยุรายใหญ่ทั้งหมดเพื่อตกลงที่จะปฏิบัติตามคำบรรยายอย่างเป็นทางการ—'วัคซีนมีความปลอดภัยและมีประสิทธิภาพ' และเพื่อเซ็นเซอร์ใครก็ตามที่ตั้งคำถามเกี่ยวกับ BS นี้“ดังนั้น สาธารณชนจึงไม่เคยได้ยินหลักฐานจากแพทย์และนักวิจัยทางการแพทย์ที่มีชื่อเสียงหลายร้อยคนที่เตือนว่าวัคซีนมีอันตรายและมักเป็นอันตรายถึงชีวิต”“นั่นเป็นความผิดพลาดครั้งใหญ่ของฉัน” เกทส์กล่าว ดูเหนื่อยและบางครั้งก็มีน้ำตา “เราไม่ควรทำอย่างนั้น ประชาชนมีสิทธิทุกอย่างที่จะได้รับข้อมูลข่าวสารที่ดี เพื่อรับข้อเท็จจริงทั้งหมดเพื่อที่พวกเขาจะได้ตัดสินใจอย่างมีเหตุมีผล”เปลี่ยนหัวข้อราวกับว่าต้องการแสดงความเห็นอกเห็นใจ มิสเตอร์เกทส์วางใจว่า: “ฉันผ่านช่วงเวลาที่ยากลำบากและค้นหาจิตวิญญาณมากมายตั้งแต่เมลินดาทิ้งฉัน การหย่าร้างครั้งนี้ทำให้ฉันต้องมองตัวเองให้ดี ฉันไม่ต้องการที่จะถูกจดจำในฐานะสัตว์ประหลาดที่ฆ่าคนนับล้านด้วยวัคซีนที่อันตรายถึงตาย ฉันไม่ใช่สัตว์ประหลาด ฉันไม่ใช่ฆาตกร ฉันไม่ต้องการให้ครอบครัว เพื่อน และบริษัทจำได้ว่าฉันเป็นนักฆ่าหมู่"บางคนเรียกฉันว่าเป็นนักสังคมสงเคราะห์หรือแม้แต่โรคจิตเภทเพราะแผนการณ์ของฉันในการช่วยเหลือมนุษยชาติ - เช่นการลดภาวะโลกร้อนด้วยการพ่นฝุ่นสู่บรรยากาศชั้นบนหรือปล่อยยุงดัดแปลงพันธุกรรมหลายล้านตัวเพื่อต่อสู้กับไข้เลือดออกและไวรัสซิกา''', '''The quint รายงานว่า ข่าวปลอมนี้ถูกแชร์ไปหลายสื่อโซเชียล ทั้งเฟซบุ๊ก ทวิตเตอร์ รวมทั้ง WhatsApp เพราะผู้ใช้เชื่อว่าเป็นความจริง และก้อปปี้เนื้อหาจากข่าวปลอมนี้ไปแชร์ ทั้งๆที่เกตส์เป็นหัวหอกในการวางแผนวัคซีนโควิด เมื่อพบว่ามีการแชร์ข่าวปลอมนี้ออกไป The quint ได้ เสริช์คีย์เวิร์ด จากกูเกิลเพื่อค้นหา เว็บไซต์ Daily Expose ซึ่งได้พบว่ามีบทความหลายชิ้นในเว็บไซต์ที่ตรวจสอบข้อเท็จจริงอย่าง Full Fact ได้หักล้างบทความหลายชิ้นที่ Daily Expose นำเสนอและเมื่อเช็คลิ้งค์ของข่าวนี้พบว่ามีการปรับพาดหัวข่าวใหม่ ว่าเป็นรายงานเชิงเสียดสี''', '''เช็คก่อนแชร์ ข่าวปลอม”บิล เกตส์เรียกร้องให้ถอนวัคซีนโควิด”''','''การระบาดของไวรัสโควิด-19 ส่งผลให้ประชาชนตื่นตัวและหาข้อมูลความรู้เกี่ยวกับไวรัส การฉีดวัคซีน เพื่อเป็นแนวทางการดูแลตัวและป้องกันโรคมากขึ้น ซึ่งในขณะเดียวกันก็มีผลให้เกิดการสร้างข้อมูลปลอม การสร้างข่าวเทียมขึ้นมามากมาย มีประชาชนบางกลุ่มหลงเชื่อจนส่งต่อและแพร่กระจายในโลกโซเชียลอย่างรวดเร็วไม่แพ้การกระจายของไวรัสเลยทีเดียว''']

**Step 2: Tokenize**

In [113]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ช็อก', 'Bill', 'Gates', 'ร้อง', 'ถอน', 'วัคซีน', 'Covid', 'วัคซีน', 'อันตราย', 'W', 'Gelles', 'วอชิงตัน', 'MaraviPost', 'ประกาศ', 'ตกใจ', 'Bill', 'Gates', 'มหา', 'เศรษฐี', 'ก่อ', 'Microsoft', 'เบื้อง', 'วัคซีน', 'COVID', 'ร้อง', 'การนำ', 'วัคซีน', 'พันธุกรรม', 'COVID', 'ตลาด', 'การก', 'ล่า', 'วสุ', 'นท', 'รพ', 'จน์', 'โทรทัศน์', 'เจ็บ', 'ปวด', 'ร้าว', 'นาที', 'เกตส์', 'ทำ', 'พลาด', 'มหันต์', 'ต้องการ', 'ปกป้อง', 'ผู้คน', 'ไวรัส', 'อันตราย', 'กลาย', 'ไวรัส', 'อันตราย', 'วัคซีน', 'อันตราย', 'จินตนาการ', 'W', 'Gelles', 'วัคซีน', '—', 'ไฟ', 'เซอร์', 'โม', 'เด', 'อร์', 'นา', 'จอห์น', 'สัน', 'แอนด์', 'จอห์น', 'สัน', 'แอ', 'สต', 'ร้า', 'เซ', 'เนก้า', '—', 'ฆ่า', 'คน', 'ซ้าย', 'ขวา', '—', 'ทำร้าย', 'คน', 'คน', 'เลว', 'ร้าย', 'เกตส์', 'โบกมือ', 'อากาศ', 'ผลลัพธ์', 'ทึ่ง', 'ข้อมูล', 'รัฐบาล', 'ระบบ', 'รายงาน', 'CDC', '…', '13,000', 'ชีวิต', 'สหรัฐอเมริกา', 'เหตุการณ์', 'ประสงค์', 'ครึ่ง', 'ล้าน', 'คน', 'รู้', 'ดี', 'ระบบ', 'รายงาน', 'เรื่อง', 'หลอก', 'ลวง', 'ทราบ', 'ดี', 'VAER

In [114]:
tokens_list = [split_word(txt) for txt in clean_text]

In [115]:
tokens_list

[['ช็อก',
  'Bill',
  'Gates',
  'ร้อง',
  'ถอน',
  'วัคซีน',
  'Covid',
  'วัคซีน',
  'อันตราย',
  'W',
  'Gelles',
  'วอชิงตัน',
  'MaraviPost',
  'ประกาศ',
  'ตกใจ',
  'Bill',
  'Gates',
  'มหา',
  'เศรษฐี',
  'ก่อ',
  'Microsoft',
  'เบื้อง',
  'วัคซีน',
  'COVID',
  'ร้อง',
  'การนำ',
  'วัคซีน',
  'พันธุกรรม',
  'COVID',
  'ตลาด',
  'การก',
  'ล่า',
  'วสุ',
  'นท',
  'รพ',
  'จน์',
  'โทรทัศน์',
  'เจ็บ',
  'ปวด',
  'ร้าว',
  'นาที',
  'เกตส์',
  'ทำ',
  'พลาด',
  'มหันต์',
  'ต้องการ',
  'ปกป้อง',
  'ผู้คน',
  'ไวรัส',
  'อันตราย',
  'กลาย',
  'ไวรัส',
  'อันตราย',
  'วัคซีน',
  'อันตราย',
  'จินตนาการ',
  'W',
  'Gelles',
  'วัคซีน',
  '—',
  'ไฟ',
  'เซอร์',
  'โม',
  'เด',
  'อร์',
  'นา',
  'จอห์น',
  'สัน',
  'แอนด์',
  'จอห์น',
  'สัน',
  'แอ',
  'สต',
  'ร้า',
  'เซ',
  'เนก้า',
  '—',
  'ฆ่า',
  'คน',
  'ซ้าย',
  'ขวา',
  '—',
  'ทำร้าย',
  'คน',
  'คน',
  'เลว',
  'ร้าย',
  'เกตส์',
  'โบกมือ',
  'อากาศ',
  'ผลลัพธ์',
  'ทึ่ง',
  'ข้อมูล',
  'รัฐบาล',
  'ระบบ',
  'รายง

**Step 3 + 4a: Bag of words + count word**

In [116]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [117]:
cvec.vocabulary_

{'ช็อก': 63,
 'Bill': 3,
 'Gates': 11,
 'ร้อง': 160,
 'ถอน': 92,
 'วัคซีน': 184,
 'Covid': 6,
 'อันตราย': 232,
 'W': 19,
 'Gelles': 12,
 'วอชิงตัน': 182,
 'MaraviPost': 13,
 'ประกาศ': 118,
 'ตกใจ': 78,
 'มหา': 144,
 'เศรษฐี': 275,
 'ก่อ': 30,
 'Microsoft': 15,
 'เบื้อง': 265,
 'COVID': 5,
 'การนำ': 28,
 'พันธุกรรม': 136,
 'ตลาด': 80,
 'การก': 26,
 'ล่า': 176,
 'วสุ': 181,
 'นท': 104,
 'รพ': 149,
 'จน์': 50,
 'โทรทัศน์': 298,
 'เจ็บ': 245,
 'ปวด': 126,
 'ร้าว': 166,
 'นาที': 106,
 'เกตส์': 240,
 'ทำ': 98,
 'พลาด': 134,
 'มหันต์': 143,
 'ต้องการ': 90,
 'ปกป้อง': 117,
 'ผู้คน': 132,
 'ไวรัส': 311,
 'กลาย': 24,
 'จินตนาการ': 56,
 '—': 312,
 'ไฟ': 310,
 'เซอร์': 250,
 'โม': 300,
 'เด': 252,
 'อร์': 229,
 'นา': 105,
 'จอห์น': 51,
 'สัน': 205,
 'แอนด์': 295,
 'แอ': 294,
 'สต': 193,
 'ร้า': 163,
 'เซ': 249,
 'เนก้า': 261,
 'ฆ่า': 46,
 'คน': 37,
 'ซ้าย': 67,
 'ขวา': 32,
 'ทำร้าย': 99,
 'เลว': 271,
 'ร้าย': 165,
 'โบกมือ': 299,
 'อากาศ': 235,
 'ผลลัพธ์': 131,
 'ทึ่ง': 103,
 'ข้อมูล': 36,
 'รัฐบา

In [118]:
c_feat[:,:].todense()

matrix([[1, 1, 1, ..., 4, 5, 2],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 3, 0, 0]], dtype=int64)

In [119]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 1 1 ... 4 5 2]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 3 0 0]]


In [120]:
first_news.shape[1]

314

In [121]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 35
This is lowerpart : 342.3068798607472


0.1022474336894375

In [122]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 30
This is lowerpart : 104.51315706646699


0.28704519930367206

In [123]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 57
This is lowerpart : 229.41011311622685


0.24846332720790687

# ข่าวที่ 11

In [124]:
clean_text = ['มีกันแชร์ในโลกโซเชี่ยลว่า มีผู้ติดโควิด เป็นคลัสเตอร์ใหม่ จำนวน 50 คน ที่ต.ตาลสุม อ.ตาลสุม จ.อุบลราชธานี จริงหรือไม่','เป็นกลุ่มการพนัน เพียงการตรวจเบื้องต้น อยู่ระหว่างรอสอบสวนโรค รอการตรวจ PCR ยืนยันอีกครั้ง 15.00 น .21-09-64 ข้อมูล : สำนักงานสาธารณสุขจังหวัดอุบลราชธานี']

**Step 2: Tokenize**

In [125]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
# print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
# print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['แชร์', 'โลก', 'โซ', 'เชี่', 'ยล', 'ติด', 'โค', 'วิด', 'คลัสเตอร์', 'จำนวน', 'คน', 'ที่ต', 'ตาล', 'สุม', 'อ', 'ตาล', 'สุม', 'จ', 'อุบลราชธานี']
tokenized text:
 ['เป็นก', 'ลุ่ม', 'พนัน', 'ตรวจ', 'เบื้อง', 'ต้น', 'รอ', 'สอบสวน', 'โรค', 'รอ', 'ตรวจ', 'PCR', '15.00', 'น', 'ข้อมูล', 'สำนักงาน', 'สาธารณสุข', 'จังหวัด', 'อุบลราชธานี']


In [126]:
tokens_list = [split_word(txt) for txt in clean_text]

In [127]:
tokens_list

[['แชร์',
  'โลก',
  'โซ',
  'เชี่',
  'ยล',
  'ติด',
  'โค',
  'วิด',
  'คลัสเตอร์',
  'จำนวน',
  'คน',
  'ที่ต',
  'ตาล',
  'สุม',
  'อ',
  'ตาล',
  'สุม',
  'จ',
  'อุบลราชธานี'],
 ['เป็นก',
  'ลุ่ม',
  'พนัน',
  'ตรวจ',
  'เบื้อง',
  'ต้น',
  'รอ',
  'สอบสวน',
  'โรค',
  'รอ',
  'ตรวจ',
  'PCR',
  '15.00',
  'น',
  'ข้อมูล',
  'สำนักงาน',
  'สาธารณสุข',
  'จังหวัด',
  'อุบลราชธานี']]

**Step 3 + 4a: Bag of words + count word**

In [128]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [129]:
cvec.vocabulary_

{'แชร์': 28,
 'โลก': 32,
 'โซ': 30,
 'เชี่': 25,
 'ยล': 15,
 'ติด': 10,
 'โค': 29,
 'วิด': 18,
 'คลัสเตอร์': 4,
 'จำนวน': 7,
 'คน': 3,
 'ที่ต': 12,
 'ตาล': 9,
 'สุม': 22,
 'อ': 23,
 'จ': 5,
 'อุบลราชธานี': 24,
 'เป็นก': 27,
 'ลุ่ม': 17,
 'พนัน': 14,
 'ตรวจ': 8,
 'เบื้อง': 26,
 'ต้น': 11,
 'รอ': 16,
 'สอบสวน': 19,
 'โรค': 31,
 'PCR': 1,
 '15.00': 0,
 'น': 13,
 'ข้อมูล': 2,
 'สำนักงาน': 21,
 'สาธารณสุข': 20,
 'จังหวัด': 6}

In [130]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
         0, 2, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1],
        [1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1,
         1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0]], dtype=int64)

In [131]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 1 1 1 0 1 0 2 1 0 1 0 0 1 0 0 1 0 0 0 2 1 1 1 0 0 1 1 1 0 1]
 [1 1 1 0 0 0 1 0 2 0 0 1 0 1 1 0 2 1 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0]]


In [132]:
first_news.shape[1]

33

In [133]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 1
This is lowerpart : 22.999999999999996


0.04347826086956522

In [134]:
# find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

In [135]:
# find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

# ข่าวที่ 12

In [136]:
clean_text = ['เปิดสถานศึกษา','ศธ.จ่อเปิดเรียนอำเภอปลอดโควิด 29 จังหวัดพื้นที่สีแดงเข้ม มั่นใจฉีดวัคซีนเด็กไร้ปัญหา ครูฉีดแล้วกว่า 70%','กระทรวงศึกษาธิการ เตรียมเปิดสถานศึกษาในพื้นที่ 29 จังหวัดสีแดงเข้ม','นางเกศทิพย์ ศุภวานิช ผู้ตรวจราชการกระทรวงศึกษาธิการ (ศธ.) ในฐานะโฆษกกระทรวงศึกษาธิการ  เปิดเผยว่า กระทรวงศึกษาธิการ ได้หารือกับกรมอนามัย กระทรวงสาธารณสุข (สธ.) เพื่อวางแนวทางกระจายวัคซีน โดยจะส่งไปที่จังหวัดเพื่อส่งต่อไปยังโรงเรียน   เมื่อวัคซีนพร้อมก็สามารถดำเนินการฉีดได้ทันที ส่วนการฉีดวัคซีนครูนั้น ดำเนินการไปแล้วกว่า 70% เหลืออีก 30%  คิดว่าไม่น่าจะมีปัญหา']

**Step 2: Tokenize**

In [137]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['สถาน', 'ศึกษา']
tokenized text:
 ['ศธ', 'จ่อ', 'เรียน', 'อำเภอ', 'ปลอด', 'โค', 'วิด', 'จังหวัด', 'พื้นที่', 'สี', 'แดง', 'เข้ม', 'มั่นใจ', 'ฉีด', 'วัคซีน', 'เด็ก', 'ไร้', 'ปัญหา', 'ครู', 'ฉีด', 'แล้วก']
tokenized text:
 ['กระทรวง', 'ศึกษาธิการ', 'เตรียม', 'สถาน', 'ศึกษา', 'พื้นที่', 'จังหวัด', 'สี', 'แดง', 'เข้ม']
tokenized text:
 ['เกศ', 'ทิพย์', 'ศุภ', 'วานิช', 'ตรวจ', 'ราชการ', 'กระทรวง', 'ศึกษาธิการ', 'ศธ', 'ฐานะ', 'โฆษก', 'กระทรวง', 'ศึกษาธิการ', 'เผย', 'กระทรวง', 'ศึกษาธิการ', 'กรม', 'อนามัย', 'กระทรวง', 'สาธารณสุข', 'สธ', 'วาง', 'แนวทาง', 'กระจาย', 'วัคซีน', 'จังหวัด', 'โรงเรียน', 'วัคซีน', 'ดำเนิน', 'ฉีด', 'ฉีด', 'วัคซีน', 'ครู', 'ดำเนิน', 'แล้วก', 'ปัญหา']


In [138]:
tokens_list = [split_word(txt) for txt in clean_text]

In [139]:
tokens_list

[['สถาน', 'ศึกษา'],
 ['ศธ',
  'จ่อ',
  'เรียน',
  'อำเภอ',
  'ปลอด',
  'โค',
  'วิด',
  'จังหวัด',
  'พื้นที่',
  'สี',
  'แดง',
  'เข้ม',
  'มั่นใจ',
  'ฉีด',
  'วัคซีน',
  'เด็ก',
  'ไร้',
  'ปัญหา',
  'ครู',
  'ฉีด',
  'แล้วก'],
 ['กระทรวง',
  'ศึกษาธิการ',
  'เตรียม',
  'สถาน',
  'ศึกษา',
  'พื้นที่',
  'จังหวัด',
  'สี',
  'แดง',
  'เข้ม'],
 ['เกศ',
  'ทิพย์',
  'ศุภ',
  'วานิช',
  'ตรวจ',
  'ราชการ',
  'กระทรวง',
  'ศึกษาธิการ',
  'ศธ',
  'ฐานะ',
  'โฆษก',
  'กระทรวง',
  'ศึกษาธิการ',
  'เผย',
  'กระทรวง',
  'ศึกษาธิการ',
  'กรม',
  'อนามัย',
  'กระทรวง',
  'สาธารณสุข',
  'สธ',
  'วาง',
  'แนวทาง',
  'กระจาย',
  'วัคซีน',
  'จังหวัด',
  'โรงเรียน',
  'วัคซีน',
  'ดำเนิน',
  'ฉีด',
  'ฉีด',
  'วัคซีน',
  'ครู',
  'ดำเนิน',
  'แล้วก',
  'ปัญหา']]

**Step 3 + 4a: Bag of words + count word**

In [140]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [141]:
cvec.vocabulary_

{'สถาน': 24,
 'ศึกษา': 21,
 'ศธ': 20,
 'จ่อ': 5,
 'เรียน': 35,
 'อำเภอ': 29,
 'ปลอด': 11,
 'โค': 39,
 'วิด': 19,
 'จังหวัด': 4,
 'พื้นที่': 13,
 'สี': 27,
 'แดง': 36,
 'เข้ม': 31,
 'มั่นใจ': 14,
 'ฉีด': 6,
 'วัคซีน': 16,
 'เด็ก': 32,
 'ไร้': 42,
 'ปัญหา': 12,
 'ครู': 3,
 'แล้วก': 38,
 'กระทรวง': 2,
 'ศึกษาธิการ': 22,
 'เตรียม': 33,
 'เกศ': 30,
 'ทิพย์': 10,
 'ศุภ': 23,
 'วานิช': 18,
 'ตรวจ': 9,
 'ราชการ': 15,
 'ฐานะ': 7,
 'โฆษก': 40,
 'เผย': 34,
 'กรม': 0,
 'อนามัย': 28,
 'สาธารณสุข': 26,
 'สธ': 25,
 'วาง': 17,
 'แนวทาง': 37,
 'กระจาย': 1,
 'โรงเรียน': 41,
 'ดำเนิน': 8}

In [142]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
         0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
         1],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0],
        [1, 1, 4, 1, 1, 0, 2, 1, 2, 1, 1, 0, 1, 0, 0, 1, 3, 1, 1, 0, 1,
         0, 3, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
         0]], dtype=int64)

In [143]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0]
 [0 0 0 1 1 1 2 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1
  1 0 1 1 0 0 1]
 [0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0 0
  1 0 0 0 0 0 0]
 [1 1 4 1 1 0 2 1 2 1 1 0 1 0 0 1 3 1 1 0 1 0 3 1 0 1 1 0 1 0 1 0 0 0 1 0
  0 1 1 0 1 1 0]]


In [144]:
first_news.shape[1]

43

In [145]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 0
This is lowerpart : 6.782329983125268


0.0

In [146]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 2
This is lowerpart : 4.47213595499958


0.4472135954999579

In [147]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 0
This is lowerpart : 11.313708498984761


0.0

# ข่าวที่ 13

In [148]:
clean_text = ['เพิ่งรู้นะเนี่ย วิธีเลือกใช้ยาสีฟันให้ดูที่แถบท้ายหลอดยาสีฟันครับ สีแดงผลิตมาจากสารเคมีผสมธรรมชาติ,สีดำผลิตจากสารเคมีล้วนๆ,สีน้ำเงินผลิตมาจากธรรมชาติผสมตัวยาของแพทย์,สีเขียวผลิตจากธรรมชาติครับ','ศูนย์ชัวร์ก่อนแชร์รายงานว่า ปลายหลอดที่ระบุตามสีนั้น เพื่อให้เซนเซอร์เครื่องจักรในการตรวจจับ ก้นหลอดยาสีฟัน ไม่เกี่ยวข้องกับวัตถุดิบ หรือสารประกอบในยาสีฟันแต่อย่างใด หากต้องการทราบส่วนผสมของยาสีฟัน เราสามารถศึกษารายละเอียดได้จากข้างหลอดยาสีฟันที่มีการพิมพ์บอกไว้','แถบสีปลายหลอดยาสีฟัน ไม่ใช่ตัวบ่งชี้ชนิดสารเคมี','ความจริงของแถบสีบนหลอดยาสีฟัน! เป็นคนละเรื่องกับข้อมูลผิดๆ ที่มีการส่งต่อกันในโลกโซเชียลที่บอกว่า แถบสีต่างๆ ที่ก้นหลอดยาสีฟันคือตัวบ่งชี้ว่ายาสีฟันนั้นทำมาจากสารเคมีหรือวัสดุธรรมชาติ ทำให้ผู้รับข่าวสารเกิดความเข้าผิดในวงกว้าง']

**Step 2: Tokenize**

In [149]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['รู้', 'วิธี', 'เลือก', 'ยาสีฟัน', 'ดู', 'แถบ', 'ท้าย', 'หลอด', 'ยาสีฟัน', 'สี', 'แดง', 'ผลิต', 'สาร', 'เคมี', 'ผสม', 'ธรรมชาติ', 'สี', 'ดำ', 'ผลิต', 'สาร', 'เคมี', 'ล้วนๆ', 'สีน้ำเงิน', 'ผลิต', 'ธรรมชาติ', 'ผสม', 'ตัว', 'ยา', 'แพทย์', 'สี', 'เขียว', 'ผลิต', 'ธรรมชาติ']
tokenized text:
 ['ศูนย์', 'ชัวร์', 'แชร์', 'รายงาน', 'ปลาย', 'หลอด', 'ระบุ', 'สี', 'เซนเซอร์', 'เครื่องจักร', 'ตรวจ', 'ก้น', 'หลอด', 'ยาสีฟัน', 'วัตถุดิบ', 'สารประกอบ', 'ยาสีฟัน', 'ใด', 'ต้องการ', 'ทราบ', 'ผสม', 'ยาสีฟัน', 'ศึกษา', 'ละเอียด', 'หลอด', 'ยาสีฟัน', 'พิมพ์']
tokenized text:
 ['แถบ', 'สี', 'ปลาย', 'หลอด', 'ยาสีฟัน', 'ตัว', 'บ่ง', 'ชี้', 'ชนิด', 'สาร', 'เคมี']
tokenized text:
 ['แถบ', 'สี', 'หลอด', 'ยาสีฟัน', 'คนละ', 'เรื่อง', 'ข้อ', 'มูล', 'โลก', 'โซ', 'เชีย', 'ล', 'แถบ', 'สี', 'ก้น', 'หลอด', 'ยาสีฟัน', 'ตัว', 'บ่ง', 'ชี้', 'ยาสีฟัน', 'ทำ', 'สาร', 'เคมี', 'วัสดุ', 'ธรรมชาติ', 'ผู้รับ', 'ข่าวสาร', 'วง']


In [150]:
tokens_list = [split_word(txt) for txt in clean_text]

In [151]:
tokens_list

[['รู้',
  'วิธี',
  'เลือก',
  'ยาสีฟัน',
  'ดู',
  'แถบ',
  'ท้าย',
  'หลอด',
  'ยาสีฟัน',
  'สี',
  'แดง',
  'ผลิต',
  'สาร',
  'เคมี',
  'ผสม',
  'ธรรมชาติ',
  'สี',
  'ดำ',
  'ผลิต',
  'สาร',
  'เคมี',
  'ล้วนๆ',
  'สีน้ำเงิน',
  'ผลิต',
  'ธรรมชาติ',
  'ผสม',
  'ตัว',
  'ยา',
  'แพทย์',
  'สี',
  'เขียว',
  'ผลิต',
  'ธรรมชาติ'],
 ['ศูนย์',
  'ชัวร์',
  'แชร์',
  'รายงาน',
  'ปลาย',
  'หลอด',
  'ระบุ',
  'สี',
  'เซนเซอร์',
  'เครื่องจักร',
  'ตรวจ',
  'ก้น',
  'หลอด',
  'ยาสีฟัน',
  'วัตถุดิบ',
  'สารประกอบ',
  'ยาสีฟัน',
  'ใด',
  'ต้องการ',
  'ทราบ',
  'ผสม',
  'ยาสีฟัน',
  'ศึกษา',
  'ละเอียด',
  'หลอด',
  'ยาสีฟัน',
  'พิมพ์'],
 ['แถบ',
  'สี',
  'ปลาย',
  'หลอด',
  'ยาสีฟัน',
  'ตัว',
  'บ่ง',
  'ชี้',
  'ชนิด',
  'สาร',
  'เคมี'],
 ['แถบ',
  'สี',
  'หลอด',
  'ยาสีฟัน',
  'คนละ',
  'เรื่อง',
  'ข้อ',
  'มูล',
  'โลก',
  'โซ',
  'เชีย',
  'ล',
  'แถบ',
  'สี',
  'ก้น',
  'หลอด',
  'ยาสีฟัน',
  'ตัว',
  'บ่ง',
  'ชี้',
  'ยาสีฟัน',
  'ทำ',
  'สาร',
  'เคมี',
  'วัสดุ',
  'ธร

**Step 3 + 4a: Bag of words + count word**

In [152]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [153]:
cvec.vocabulary_

{'รู้': 27,
 'วิธี': 34,
 'เลือก': 48,
 'ยาสีฟัน': 24,
 'ดู': 8,
 'แถบ': 51,
 'ท้าย': 14,
 'หลอด': 41,
 'สี': 39,
 'แดง': 50,
 'ผลิต': 18,
 'สาร': 37,
 'เคมี': 43,
 'ผสม': 19,
 'ธรรมชาติ': 15,
 'ดำ': 7,
 'ล้วนๆ': 30,
 'สีน้ำเงิน': 40,
 'ตัว': 10,
 'ยา': 23,
 'แพทย์': 52,
 'เขียว': 42,
 'ศูนย์': 36,
 'ชัวร์': 5,
 'แชร์': 49,
 'รายงาน': 26,
 'ปลาย': 17,
 'ระบุ': 25,
 'เซนเซอร์': 46,
 'เครื่องจักร': 44,
 'ตรวจ': 9,
 'ก้น': 0,
 'วัตถุดิบ': 32,
 'สารประกอบ': 38,
 'ใด': 55,
 'ต้องการ': 11,
 'ทราบ': 12,
 'ศึกษา': 35,
 'ละเอียด': 29,
 'พิมพ์': 21,
 'บ่ง': 16,
 'ชี้': 6,
 'ชนิด': 4,
 'คนละ': 3,
 'เรื่อง': 47,
 'ข้อ': 2,
 'มูล': 22,
 'โลก': 54,
 'โซ': 53,
 'เชีย': 45,
 'ล': 28,
 'ทำ': 13,
 'วัสดุ': 33,
 'ผู้รับ': 20,
 'ข่าวสาร': 1,
 'วง': 31}

In [154]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 3, 0, 0, 4, 2, 0,
         0, 0, 1, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 3, 1, 1,
         1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
         1, 0, 0, 4, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 3,
         0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
         0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 0, 2,
         0, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 1, 1, 0]], dtype=int64)

In [155]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 3 0 0 4 2 0 0 0 1 2 0 0 1 0 0 1 0 0 0 1 0
  0 2 0 3 1 1 1 2 0 0 0 0 1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 4 1 1 0 0 1 0 0 1 0 0 1
  1 0 1 1 0 3 0 0 1 0 1 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0]
 [1 1 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 3 0 0 0 1 0 0 1 0 1 0 0
  0 1 0 2 0 2 0 1 0 1 0 1 0 0 0 2 0 1 1 0]]


In [156]:
first_news.shape[1]

56

In [157]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 16
This is lowerpart : 54.08326913195984


0.29584010465345556

In [158]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 12
This is lowerpart : 26.739483914241873


0.44877455520405946

In [159]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 24
This is lowerpart : 51.62363799656122


0.46490330653563583

# ข่าวที่ 14

In [160]:
a = remove_emojis('⚠️ประกาศ เตือนภัย มิจฉาชีพแฝงตัวในสื่อออนไลน์ช่องทางต่างๆ โปรดระวัง เฟซบุ๊ก บัญชีไลน์ หรือ อีเมล ปลอม ส่งคิวอาร์โค้ด พร้อมเพย์ PromptPay เพื่อให้ท่านโอนเงินค่าวัคซีน🌐เพจเฟซบุ๊กปลอม นำตราสัญลักษณ์และภาพไปใช้ โดยชื่อเพจไม่ใช่ของโรงพยาบาลจุฬาภรณ์ ราชวิทยาลัยจุฬาภรณ์ราชวิทยาลัยจุฬาภรณ์มีเฟซบุ๊ค 2 เพจเท่านั้นที่ใช้สื่อสารเรื่องการจัดสรรวัคซีน ได้แก่1️⃣ โรงพยาบาลจุฬาภรณ์ ราชวิทยาลัยจุฬาภรณ์ (โรงพยาบาลจุฬาภรณ์ ราชวิทยาลัยจุฬาภรณ์)ผู้ติดตาม 662kและ2️⃣ ศูนย์ข้อมูลราชวิทยาลัยจุฬาภรณ์ (ศูนย์ข้อมูลราชวิทยาลัยจุฬาภรณ์)ผู้ติดตาม 155k❎บัญชีไลน์ปลอม ส่งคิวอาร์โค้ด พร้อมเพย์ PromptPay ให้โอนเงินราชวิทยาลัยจุฬาภรณ์ มีช่องทาง LINE Official account: @ChulabhornhospitalAdd Friend > http://nav.cx/8DqLuQm มีผู้ติดตามเป็น Friends 2.6M สำหรับใช้สื่อสารบริการและประกาศประชาสัมพันธ์ต่างๆเท่านั้น ไม่มีบริการตอบคำถามประชาชนในไลน์บัญชีนี้ทั้งนี้ ไม่มีบริการส่ง Link โอนเงินค่าวัคซีนซิโนฟาร์ม หรือวัคซีนโควิด19 ทุกชนิดผ่านไลน์ !!! และไม่มีการจัดตั้งบัญชี LINE ศูนย์ข้อมูลราชวิทยาลัยจุฬาภรณ์ เพื่อบริการตอบคำถามประชาชน📧อีเมลปลอม ส่งคิวอาร์โค้ด พร้อมเพย์ PromptPay ให้โอนเงินองค์กร/นิติบุคคล ที่ได้รับการจัดสรรวัคซีนสามารถตรวจสอบวิธีการโอนเงิน และพิมพ์ใบนำฝากเงิน ผ่านการล็อคอินเข้าสู่ระบบ “ลงทะเบียนองค์กรผู้ได้รับการจัดสรรวัคซีน” ของราชวิทยาลัยจุฬาภรณ์ เท่านั้น เพื่อโอนเงินผ่านช่องทางต่างๆที่ราชวิทยาลัยจุฬาภรณ์กำหนด❌ไม่มีการโอนเงินผ่านพร้อมเพย์ประกาศ ณ วันที่ 18 กันยายน 2564')

In [161]:
clean_text = [a,'การจัดสรรวัคซีนสามารถตรวจสอบวิธีการโอนเงิน และพิมพ์ใบนำฝากเงิน ผ่านการล็อกอินเข้าสู่ระบบ “ลงทะเบียนองค์กรผู้ได้รับการจัดสรรวัคซีน” ของราชวิทยาลัยจุฬาภรณ์เท่านั้น เพื่อโอนเงินผ่านช่องทางต่างๆ ที่ราชวิทยาลัยจุฬาภรณ์กำหนด โดยไม่มีการโอนเงินผ่านพร้อมเพย์...','ราชวิทยาลัยจุฬาภรณ์เตือนภัยมิจฉาชีพ หลอกโอนเงินค่าวัคซีน','ราชวิทยาลัยจุฬาภรณ์ ได้โพสต์เตือนภัยมิจฉาชีพ หลอกโอนเงินค่าวัคซีน แฉวิธีลวงเหยื่อ สร้างเฟซบุ๊ก บัญชีไลน์ หรือ อีเมล ปลอม ส่งคิวอาร์โค้ด พร้อมเพย์']

**Step 2: Tokenize**

In [162]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ประกาศ', 'เตือน', 'ภัย', 'มิจฉาชีพ', 'แฝง', 'ตัว', 'สื่อ', 'ออนไลน์', 'ช่อง', 'โปรด', 'ระวัง', 'เฟ', 'ซบุ๊ก', 'บัญชี', 'ไลน์', 'อีเมล', 'ปลอม', 'คิว', 'อาร์', 'โค้ด', 'เพย์', 'PromptPay', 'ท่าน', 'โอน', 'เงิน', 'ค่า', 'วัค', 'ซีน', 'เพจ', 'เฟ', 'ซบุ๊ก', 'ปลอม', 'ตรา', 'สัญลักษณ์', 'ภาพ', 'ชื่อ', 'เพจ', 'โรง', 'พยาบาล', 'จุฬา', 'ภ', 'รณ์', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'เฟ', 'ซบุ๊ค', 'เพจ', 'สื่อสาร', 'เรื่อง', 'จัดสรร', 'วัคซีน', '1⃣', 'โรง', 'พยาบาล', 'จุฬา', 'ภ', 'รณ์', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'โรง', 'พยาบาล', 'จุฬา', 'ภ', 'รณ์', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'ติดตาม', '662k', '2⃣', 'ศูนย์', 'ข้อมูล', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'ศูนย์', 'ข้อมูล', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'ติดตาม', '155k', 'บัญชี', 'ไลน์', 'ปลอม', 'คิว', 'อาร์', 'โค้ด', 'เพย์', 'PromptPay', 'โอน', 'เงิน', 'ราช', 'วิทยา', 'ลัย', 'จุฬา', 'ภ', 'รณ์', 'ช่อง', 'LINE', 'Official', 'acc

In [163]:
tokens_list = [split_word(txt) for txt in clean_text]

In [164]:
tokens_list

[['ประกาศ',
  'เตือน',
  'ภัย',
  'มิจฉาชีพ',
  'แฝง',
  'ตัว',
  'สื่อ',
  'ออนไลน์',
  'ช่อง',
  'โปรด',
  'ระวัง',
  'เฟ',
  'ซบุ๊ก',
  'บัญชี',
  'ไลน์',
  'อีเมล',
  'ปลอม',
  'คิว',
  'อาร์',
  'โค้ด',
  'เพย์',
  'PromptPay',
  'ท่าน',
  'โอน',
  'เงิน',
  'ค่า',
  'วัค',
  'ซีน',
  'เพจ',
  'เฟ',
  'ซบุ๊ก',
  'ปลอม',
  'ตรา',
  'สัญลักษณ์',
  'ภาพ',
  'ชื่อ',
  'เพจ',
  'โรง',
  'พยาบาล',
  'จุฬา',
  'ภ',
  'รณ์',
  'ราช',
  'วิทยา',
  'ลัย',
  'จุฬา',
  'ภ',
  'รณ์',
  'ราช',
  'วิทยา',
  'ลัย',
  'จุฬา',
  'ภ',
  'รณ์',
  'เฟ',
  'ซบุ๊ค',
  'เพจ',
  'สื่อสาร',
  'เรื่อง',
  'จัดสรร',
  'วัคซีน',
  '1⃣',
  'โรง',
  'พยาบาล',
  'จุฬา',
  'ภ',
  'รณ์',
  'ราช',
  'วิทยา',
  'ลัย',
  'จุฬา',
  'ภ',
  'รณ์',
  'โรง',
  'พยาบาล',
  'จุฬา',
  'ภ',
  'รณ์',
  'ราช',
  'วิทยา',
  'ลัย',
  'จุฬา',
  'ภ',
  'รณ์',
  'ติดตาม',
  '662k',
  '2⃣',
  'ศูนย์',
  'ข้อมูล',
  'ราช',
  'วิทยา',
  'ลัย',
  'จุฬา',
  'ภ',
  'รณ์',
  'ศูนย์',
  'ข้อมูล',
  'ราช',
  'วิทยา',
  'ลัย',
  'จุฬา',
  'ภ'

**Step 3 + 4a: Bag of words + count word**

In [165]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [166]:
cvec.vocabulary_

{'ประกาศ': 40,
 'เตือน': 79,
 'ภัย': 49,
 'มิจฉาชีพ': 51,
 'แฝง': 86,
 'ตัว': 33,
 'สื่อ': 70,
 'ออนไลน์': 74,
 'ช่อง': 25,
 'โปรด': 90,
 'ระวัง': 54,
 'เฟ': 82,
 'ซบุ๊ก': 26,
 'บัญชี': 39,
 'ไลน์': 96,
 'อีเมล': 77,
 'ปลอม': 43,
 'คิว': 19,
 'อาร์': 75,
 'โค้ด': 88,
 'เพย์': 81,
 'PromptPay': 12,
 'ท่าน': 36,
 'โอน': 93,
 'เงิน': 78,
 'ค่า': 20,
 'วัค': 60,
 'ซีน': 29,
 'เพจ': 80,
 'ตรา': 31,
 'สัญลักษณ์': 68,
 'ภาพ': 50,
 'ชื่อ': 24,
 'โรง': 92,
 'พยาบาล': 45,
 'จุฬา': 22,
 'ภ': 48,
 'รณ์': 52,
 'ราช': 55,
 'วิทยา': 63,
 'ลัย': 57,
 'ซบุ๊ค': 27,
 'สื่อสาร': 71,
 'เรื่อง': 83,
 'จัดสรร': 21,
 'วัคซีน': 61,
 '1⃣': 1,
 'ติดตาม': 34,
 '662k': 4,
 '2⃣': 3,
 'ศูนย์': 65,
 'ข้อมูล': 17,
 '155k': 0,
 'LINE': 9,
 'Official': 11,
 'account': 13,
 'ChulabhornhospitalAdd': 6,
 'Friend': 7,
 'http': 14,
 'nav.cx': 15,
 '8DqLuQm': 5,
 'Friends': 8,
 '2.6M': 2,
 'สำหรับ': 69,
 'บริการ': 38,
 'ประชาสัมพันธ์': 42,
 'ไม่มี': 95,
 'ตอบ': 32,
 'คำถาม': 18,
 'ประชาชน': 41,
 'Link': 10,
 'ซิ': 28,
 'โน': 

In [167]:
c_feat[:,:].todense()

matrix([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  3,  1,  1,  1,
          1,  3,  2,  3,  2,  3, 13,  1,  1,  3,  2,  1,  1,  1,  1,  1,
          2,  1,  3,  1,  1,  1,  4,  4,  3,  2,  1,  4,  1,  3,  1,  1,
         13,  1,  1,  1, 13,  1,  1, 10,  0, 10,  0,  1,  1,  5,  1, 10,
          1,  3,  0,  1,  1,  1,  1,  2,  0,  2,  1,  3,  1,  2,  8,  1,
          3,  4,  3,  1,  0,  0,  1,  1,  3,  1,  1,  0,  3,  7,  1,  4,
          4],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  2,  2,  0,  0,  1,  0,  0,  0,  0,  1,  0,
          0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,
          2,  0,  0,  0,  2,  1,  0,  2,  0,  2,  1,  0,  0,  2,  0,  2,
          1,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  4,  0,
          0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  1,  1,
          0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0

In [168]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[ 1  1  1  1  1  1  1  1  1  2  1  1  3  1  1  1  1  3  2  3  2  3 13  1
   1  3  2  1  1  1  1  1  2  1  3  1  1  1  4  4  3  2  1  4  1  3  1  1
  13  1  1  1 13  1  1 10  0 10  0  1  1  5  1 10  1  3  0  1  1  1  1  2
   0  2  1  3  1  2  8  1  3  4  3  1  0  0  1  1  3  1  1  0  3  7  1  4
   4]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  2  0
   0  1  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  1  0
   2  0  0  0  2  1  0  2  0  2  1  0  0  2  0  2  1  0  0  1  0  0  0  0
   0  1  0  0  0  0  4  0  0  1  0  0  0  0  0  0  0  0  0  0  0  3  1  1
   0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   1  1  0  1  1  0  0  1  0  1  0  0  0  1  0  1  0  0  0  0  0  0  0  0
   1  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
   0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  1  0
   0  0  1  0  0  0 

In [169]:
first_news.shape[1]

97

In [170]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 228
This is lowerpart : 296.3950067055786


0.7692437282740117

In [171]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 94
This is lowerpart : 132.55187663703595


0.7091563121161857

In [172]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 127
This is lowerpart : 194.03607911932255


0.654517451478193

# ข่าวที่ 15

In [173]:
clean_text = ['ประชาชนที่ได้รับความเสียหายหรือเกิดอาการไม่พึงประสงค์หลังรับวัคซีนโควิด-19 ที่ภาครัฐจัดให้ สามารถยื่นคำร้องขอรับเงินช่วยเหลือได้ที่ รพ. ที่ฉีดสำนักงานสาธารณสุขจังหวัด หรือ สปสช. พื้นที่ ทั้ง 13 เขตทั่วประเทศ ภายใน 2 ปีนับตั้งแต่วันที่ทราบความเสียหาย 1.กรณีเสียชีวิตหรือทุพพลภาพถาวร ช่วยเหลือไม่เกิน 400,000 บาท 2.กรณีสูญเสียอวัยวะหรือพิการ ช่วยเหลือไม่เกิน 240,000 บาท 3.กรณีเกิดภาวะเจ็บป่วยที่ต้องรับการรักษาไม่เกิน 1 แสนบาท โดยเตรียมเอกสาร 1.สำเนาบัตรประชาชน 2.สำเนาใบมรณบัตร (กรณีเสียชีวิต) 3.ความเห็นแพทย์ผู้ให้การรักษา พร้อมแนบแบบคำร้อง ดาวน์โหลดที่ https://www.nhso.go.th/storage/files/841⋯dsLD-d-aETThbvoZMhUwbsijTsXLgSjv_icv-Fos สอบถามเพิ่มเติม สายด่วน สปสช. 1330 #TPchannel #วิทยุและโทรทัศน์รัฐสภา','ได้รับความเสียหาย (แพ้วัคซีน) หรือเกิดอาการไม่พึงประสงค์หลังได้รับการ "ฉีดวัคซีนโควิด-19" ที่ภาครัฐจัดให้ สามารถยื่นคำร้องขอรับเงินช่วยเหลือได้ที่โรงพยาบาล ที่ฉีดสำนักงานสาธารณสุขจังหวัด หรือ สปสช. พื้นที่ทั้ง 13 เขตทั่วประเทศ ภายใน 2 ปีนับตั้งแต่วันที่ทราบความเสียหาย','"ฉีดวัคซีน" แล้วแพ้ ดาวน์โหลดเอกสารยื่นคำร้องรับเงินที่นี่ เช็คเงื่อนไข','ฉีดวัคซีน" แล้วแพ้ ดาวน์โหลดเอกสารยื่นคำร้องรับเงินที่นี่ เช็คเงื่อนไข โดยสามารถยื่นคำร้องขอรับเงินช่วยเหลือได้ที่โรงพยาบาล ที่ฉีดสำนักงานสาธารณสุขจังหวัด หรือ สปสช. พื้นที่ทั้ง 13 เขตทั่วประเทศ']

**Step 2: Tokenize**

In [174]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ประชาชน', 'หาย', 'อาการ', 'ประสงค์', 'วัคซีน', 'โค', 'วิด', 'รัฐ', 'ยื่น', 'คำร้อง', 'ขอรับ', 'เงิน', 'รพ', 'ฉีด', 'สำนักงาน', 'สาธารณสุข', 'จังหวัด', 'สปสช', 'พื้นที่', 'เขต', 'ประเทศ', 'ปี', 'ทราบ', 'หาย', 'กรณี', 'ชีวิต', 'ทุพพลภาพ', 'ถาวร', '400,000', 'บาท', 'กรณี', 'สูญ', 'อวัยวะ', 'พิการ', '240,000', 'บาท', 'กรณี', 'ภาวะ', 'เจ็บ', 'ป่วย', 'รักษา', 'แสน', 'บาท', 'เตรียม', 'เอกสาร', 'สำเนา', 'บัตร', 'ประชาชน', 'สำเนา', 'ใบ', 'มรณบัตร', 'กรณี', 'ชีวิต', 'แพทย์', 'ให้การ', 'รักษา', 'แนบ', 'คำร้อง', 'ดาวน์โหลด', 'https', 'www.nhso.go.th', 'storage', 'files', '⋯', 'dsLD', 'd', 'aETThbvoZMhUwbsijTsXLgSjv_icv', 'Fos', 'สอบถาม', 'เติม', 'สาย', 'ด่วน', 'สปสช', 'TPchannel', 'วิทยุ', 'โทรทัศน์', 'รัฐสภา']
tokenized text:
 ['หาย', 'แพ้', 'วัคซีน', 'อาการ', 'ประสงค์', 'ฉีด', 'วัคซีน', 'โค', 'วิด', 'รัฐ', 'ยื่น', 'คำร้อง', 'ขอรับ', 'เงิน', 'โรง', 'พยาบาล', 'ฉีด', 'สำนักงาน', 'สาธารณสุข', 'จังหวัด', 'สปสช', 'พื้นที่', 'เขต', 'ประเทศ', 'ปี', 'ทราบ', 'หาย']
tokenized text:
 ['ฉี

In [175]:
tokens_list = [split_word(txt) for txt in clean_text]

In [176]:
tokens_list

[['ประชาชน',
  'หาย',
  'อาการ',
  'ประสงค์',
  'วัคซีน',
  'โค',
  'วิด',
  'รัฐ',
  'ยื่น',
  'คำร้อง',
  'ขอรับ',
  'เงิน',
  'รพ',
  'ฉีด',
  'สำนักงาน',
  'สาธารณสุข',
  'จังหวัด',
  'สปสช',
  'พื้นที่',
  'เขต',
  'ประเทศ',
  'ปี',
  'ทราบ',
  'หาย',
  'กรณี',
  'ชีวิต',
  'ทุพพลภาพ',
  'ถาวร',
  '400,000',
  'บาท',
  'กรณี',
  'สูญ',
  'อวัยวะ',
  'พิการ',
  '240,000',
  'บาท',
  'กรณี',
  'ภาวะ',
  'เจ็บ',
  'ป่วย',
  'รักษา',
  'แสน',
  'บาท',
  'เตรียม',
  'เอกสาร',
  'สำเนา',
  'บัตร',
  'ประชาชน',
  'สำเนา',
  'ใบ',
  'มรณบัตร',
  'กรณี',
  'ชีวิต',
  'แพทย์',
  'ให้การ',
  'รักษา',
  'แนบ',
  'คำร้อง',
  'ดาวน์โหลด',
  'https',
  'www.nhso.go.th',
  'storage',
  'files',
  '⋯',
  'dsLD',
  'd',
  'aETThbvoZMhUwbsijTsXLgSjv_icv',
  'Fos',
  'สอบถาม',
  'เติม',
  'สาย',
  'ด่วน',
  'สปสช',
  'TPchannel',
  'วิทยุ',
  'โทรทัศน์',
  'รัฐสภา'],
 ['หาย',
  'แพ้',
  'วัคซีน',
  'อาการ',
  'ประสงค์',
  'ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'รัฐ',
  'ยื่น',
  'คำร้อง',
  'ขอรับ',
 

**Step 3 + 4a: Bag of words + count word**

In [177]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [178]:
cvec.vocabulary_

{'ประชาชน': 25,
 'หาย': 50,
 'อาการ': 52,
 'ประสงค์': 26,
 'วัคซีน': 40,
 'โค': 65,
 'วิด': 41,
 'รัฐ': 38,
 'ยื่น': 35,
 'คำร้อง': 14,
 'ขอรับ': 13,
 'เงิน': 54,
 'รพ': 36,
 'ฉีด': 16,
 'สำนักงาน': 47,
 'สาธารณสุข': 45,
 'จังหวัด': 15,
 'สปสช': 43,
 'พื้นที่': 32,
 'เขต': 53,
 'ประเทศ': 27,
 'ปี': 28,
 'ทราบ': 21,
 'กรณี': 12,
 'ชีวิต': 17,
 'ทุพพลภาพ': 22,
 'ถาวร': 20,
 '400': 2,
 '000': 0,
 'บาท': 24,
 'สูญ': 49,
 'อวัยวะ': 51,
 'พิการ': 31,
 '240': 1,
 'ภาวะ': 33,
 'เจ็บ': 56,
 'ป่วย': 29,
 'รักษา': 37,
 'แสน': 64,
 'เตรียม': 58,
 'เอกสาร': 60,
 'สำเนา': 48,
 'บัตร': 23,
 'ใบ': 68,
 'มรณบัตร': 34,
 'แพทย์': 62,
 'ให้การ': 69,
 'แนบ': 61,
 'ดาวน์โหลด': 18,
 'https': 9,
 'www.nhso.go.th': 11,
 'storage': 10,
 'files': 8,
 '⋯': 70,
 'dsLD': 7,
 'd': 6,
 'aETThbvoZMhUwbsijTsXLgSjv_icv': 5,
 'Fos': 3,
 'สอบถาม': 44,
 'เติม': 59,
 'สาย': 46,
 'ด่วน': 19,
 'TPchannel': 4,
 'วิทยุ': 42,
 'โทรทัศน์': 66,
 'รัฐสภา': 39,
 'แพ้': 63,
 'โรง': 67,
 'พยาบาล': 30,
 'เช็ค': 57,
 'เงื่อนไข': 55}

In [179]:
c_feat[:,:].todense()

matrix([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 2, 1, 1, 1,
         1, 1, 1, 3, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
         1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1,
         0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0,
         0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 0, 0,
         1, 0, 0, 0, 1, 0, 0, 0]],

In [180]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[2 1 1 1 1 1 1 1 1 1 1 1 4 1 2 1 1 2 1 1 1 1 1 1 3 2 1 1 1 1 0 1 1 1 1 1
  1 2 1 1 1 1 1 2 1 1 1 1 2 1 2 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1
  0 0 1 0 2 1 0 1 0 1 0 1 0 0 2 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1 2 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 2
  0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 2 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0]]


In [181]:
first_news.shape[1]

71

In [182]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 28
This is lowerpart : 61.065538563088104


0.45852375429511044

In [183]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 33.61547262794323


0.2379856469234918

In [184]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 22
This is lowerpart : 60.13318551349164


0.3658545578807566

# ข่าวที่ 16

In [185]:
clean_text = ['ไม่ควรเอา น้ำผึ้ง แช่ในตู้เย็น เพราะจะทำให้น้ำผึ้งตกผลึกและข้นเร็วขึ้น จริงหรือ','ปัจจัยที่มีผลต่อการเกิดผลึกในน้ำผึ้ง ดังนี้ อุณหภูมิในการเก็บรักษา หากเก็บรักษาที่อุณหภูมิต่ำ ให้เกิดการตกผลึกได้ ถ้าเก็บในที่อุณหภูมิต่ำ (ตู้เย็น) น้ำผึ้งจะมีผลึกน้ำตาลกลูโคสตกเป็นเกล็ดเล็กๆ','วิธีสังเกตน้ำผึ้งแท้','น้ำผึ้งแท้มีคุณสมบัติทางยาและมีคุณค่าทางโภชนาการมากมาย และด้วยเหตุที่น้ำผึ้งมีราคาแพงเมื่อเทียบกับน้ำตาล จึงมีคนหัวขี้โกงทำน้ำผึ้งปลอมออกมาหลอกขายเพื่อที่จะได้กำไรมากๆ ซึ่งสำหรับผู้ที่คุ้นเคยกับการกินน้ำผึ้งเป็นประจำอยู่แล้ว จะสามารถพิสูจน์ได้ง่ายๆ เพียงแค่ดูลักษณะหรือ การชิม แต่สำหรับผู้ที่ไม่คุ้นเคย หากไปเจอน้ำผึ้งปลอมประโยชน์ที่จะได้จากน้ำผึ้งก็ลดลง ถ้าไม่มั่นใจก็ลองพิสูจน์ได้ตามวิธีการดังต่อไปนี้']

**Step 2: Tokenize**

In [186]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['น้ำ', 'ผึ้ง', 'แช่', 'ตู้', 'เย็น', 'น้ำ', 'ผึ้ง', 'ตกผลึก', 'ข้น']
tokenized text:
 ['ปัจจัย', 'ผลึก', 'น้ำ', 'ผึ้ง', 'ดังนี้', 'อุณหภูมิ', 'รักษา', 'รักษา', 'อุณหภูมิ', 'ต่ำ', 'ตกผลึก', 'อุณหภูมิ', 'ต่ำ', 'ตู้', 'เย็น', 'น้ำ', 'ผึ้ง', 'ผลึก', 'น้ำตาล', 'กลูโคส', 'ตก', 'เกล็ด']
tokenized text:
 ['วิธี', 'สังเกต', 'น้ำ', 'ผึ้ง']
tokenized text:
 ['น้ำ', 'ผึ้ง', 'คุณสมบัติ', 'ยา', 'คุณค่า', 'โภชนาการ', 'น้ำ', 'ผึ้ง', 'ราคา', 'แพง', 'เทียบ', 'น้ำตาล', 'คน', 'หัว', 'ขี้', 'โกง', 'ทำ', 'น้ำ', 'ผึ้ง', 'ปลอม', 'หลอก', 'ขาย', 'กำ', 'มากๆ', 'สำหรับ', 'คุ้น', 'กิน', 'น้ำ', 'ผึ้ง', 'ประจำ', 'พิสูจน์', 'ดู', 'ลักษณะ', 'ชิม', 'สำหรับ', 'คุ้น', 'เจอ', 'น้ำ', 'ผึ้ง', 'ปลอม', 'ประโยชน์', 'น้ำ', 'ผึ้ง', 'ลด', 'มั่นใจ', 'ลอง', 'พิสูจน์', 'วิธี']


In [187]:
tokens_list = [split_word(txt) for txt in clean_text]

In [188]:
tokens_list

[['น้ำ', 'ผึ้ง', 'แช่', 'ตู้', 'เย็น', 'น้ำ', 'ผึ้ง', 'ตกผลึก', 'ข้น'],
 ['ปัจจัย',
  'ผลึก',
  'น้ำ',
  'ผึ้ง',
  'ดังนี้',
  'อุณหภูมิ',
  'รักษา',
  'รักษา',
  'อุณหภูมิ',
  'ต่ำ',
  'ตกผลึก',
  'อุณหภูมิ',
  'ต่ำ',
  'ตู้',
  'เย็น',
  'น้ำ',
  'ผึ้ง',
  'ผลึก',
  'น้ำตาล',
  'กลูโคส',
  'ตก',
  'เกล็ด'],
 ['วิธี', 'สังเกต', 'น้ำ', 'ผึ้ง'],
 ['น้ำ',
  'ผึ้ง',
  'คุณสมบัติ',
  'ยา',
  'คุณค่า',
  'โภชนาการ',
  'น้ำ',
  'ผึ้ง',
  'ราคา',
  'แพง',
  'เทียบ',
  'น้ำตาล',
  'คน',
  'หัว',
  'ขี้',
  'โกง',
  'ทำ',
  'น้ำ',
  'ผึ้ง',
  'ปลอม',
  'หลอก',
  'ขาย',
  'กำ',
  'มากๆ',
  'สำหรับ',
  'คุ้น',
  'กิน',
  'น้ำ',
  'ผึ้ง',
  'ประจำ',
  'พิสูจน์',
  'ดู',
  'ลักษณะ',
  'ชิม',
  'สำหรับ',
  'คุ้น',
  'เจอ',
  'น้ำ',
  'ผึ้ง',
  'ปลอม',
  'ประโยชน์',
  'น้ำ',
  'ผึ้ง',
  'ลด',
  'มั่นใจ',
  'ลอง',
  'พิสูจน์',
  'วิธี']]

**Step 3 + 4a: Bag of words + count word**

In [189]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [190]:
cvec.vocabulary_

{'น้ำ': 18,
 'ผึ้ง': 25,
 'แช่': 45,
 'ตู้': 15,
 'เย็น': 44,
 'ตกผลึก': 14,
 'ข้น': 5,
 'ปัจจัย': 23,
 'ผลึก': 24,
 'ดังนี้': 11,
 'อุณหภูมิ': 40,
 'รักษา': 30,
 'ต่ำ': 16,
 'น้ำตาล': 19,
 'กลูโคส': 0,
 'ตก': 13,
 'เกล็ด': 41,
 'วิธี': 35,
 'สังเกต': 36,
 'คุณสมบัติ': 8,
 'ยา': 29,
 'คุณค่า': 7,
 'โภชนาการ': 48,
 'ราคา': 31,
 'แพง': 46,
 'เทียบ': 43,
 'คน': 6,
 'หัว': 39,
 'ขี้': 4,
 'โกง': 47,
 'ทำ': 17,
 'ปลอม': 22,
 'หลอก': 38,
 'ขาย': 3,
 'กำ': 1,
 'มากๆ': 28,
 'สำหรับ': 37,
 'คุ้น': 9,
 'กิน': 2,
 'ประจำ': 20,
 'พิสูจน์': 26,
 'ดู': 12,
 'ลักษณะ': 34,
 'ชิม': 10,
 'เจอ': 42,
 'ประโยชน์': 21,
 'ลด': 32,
 'มั่นใจ': 27,
 'ลอง': 33}

In [191]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0,
         0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 2, 1, 0,
         0, 0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1,
         0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 0, 0, 0, 0, 1, 6, 1, 1,
         1, 2, 0, 0, 6, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 0,
         1, 1, 0, 0, 1, 1, 1]], dtype=int64)

In [192]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 2 0 2 1 0 0 0 1 2 2 0 0 0 0 2 0 0 0 0 0
  0 0 0 0 3 1 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
  1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 1 1 1 2 1 0 1 0 0 0 0 1 6 1 1 1 2 0 0 6 2 1 1 1 0 1 1 1 1 1
  0 2 1 1 0 0 1 1 0 0 1 1 1]]


In [193]:
first_news.shape[1]

49

In [194]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 11
This is lowerpart : 22.226110770892866


0.49491339773243237

In [195]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 4
This is lowerpart : 7.211102550927978


0.5547001962252291

In [196]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 24
This is lowerpart : 38.83297567789519


0.6180314431495257

# ข่าวที่ 17

In [197]:
clean_text = ['คณะรัฐมนตรี ไฟเขียวให้ต่างชาติมาซื้อที่ดินหรืออสังหาในไทย ได้แล้ว จริงหรือ','มติคณะรัฐมนตรี เห็นชอบ ออกวีซ่าประเภทผู้พำนักระยะยาว ให้ต่างชาติที่มีศักยภาพสูง อาศัยอยู่ในไทย พร้อมแก้กฎหมายถือครองที่ดินรองรับ ที่เห็นชอบหลักการมาตรการกระตุ้นเศรษฐกิจและการลงทุนโดยการดึงดูดชาวต่างชาติที่มีศักยภาพสูงสู่ประเทศไทย ในลักษณะผู้พำนักระยะยาว (long-term stay) โดย มี 4 กลุ่มเป้าหมาย ได้แก่1.กลุ่มประชากรโลกผู้มีความมั่งคั่งสูง2.กลุ่มผู้เกษียณอายุจากต่างประเทศ3.กลุ่มที่ต้องการทำงานจากประเทศไทย4.กลุ่มผู้มีทักษะเชี่ยวชาญพิเศษ','ครม.ตั้งเป้าให้ทันปี 65 แก้กฎหมายถือครองที่ดิน ดูดชาวต่างชาติศักยภาพสูงซื้อได้','รัชดา เผย มติคณะรัฐมนตรี เห็นชอบ ออกวีซ่าประเภทผู้พำนักระยะยาว ให้ต่างชาติที่มีศักยภาพสูง อาศัยอยู่ในไทย พร้อมแก้กฎหมายถือครองที่ดินรองรับ อยากทำให้ทัน ปี 2565']

**Step 2: Tokenize**

In [198]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['คณะ', 'รัฐมนตรี', 'ไฟ', 'เขียว', 'ชาติ', 'ซื้อ', 'ดิน', 'อสัง', 'หา', 'ไทย']
tokenized text:
 ['มติ', 'คณะ', 'รัฐมนตรี', 'ชอบ', 'วีซ่า', 'ประเภท', 'พำนัก', 'ชาติ', 'ศักยภาพ', 'อาศัย', 'ไทย', 'แก้', 'กฎหมาย', 'ครอง', 'ที่ดิน', 'รองรับ', 'ชอบ', 'หลัก', 'มาตรการ', 'กระตุ้น', 'เศรษฐกิจ', 'ลงทุน', 'ดึงดูด', 'ชาติ', 'ศักยภาพ', 'ประเทศไทย', 'ลักษณะ', 'พำนัก', 'long', 'term', 'stay', 'เป้า', 'หมาย', 'ประชากร', 'โลก', 'มั่งคั่ง', 'เกษียณ', 'อายุ', 'ประเทศ', 'ต้องการ', 'ทำงาน', 'ประเทศไทย', 'ทักษะ', 'เชี่ยวชาญ', 'พิเศษ']
tokenized text:
 ['ครม', 'เป้า', 'ปี', 'แก้', 'กฎหมาย', 'ครอง', 'ที่ดิน', 'ดูด', 'ชาติ', 'ศักยภาพ', 'ซื้อ']
tokenized text:
 ['รัช', 'ดา', 'เผย', 'มติ', 'คณะ', 'รัฐมนตรี', 'ชอบ', 'วีซ่า', 'ประเภท', 'พำนัก', 'ชาติ', 'ศักยภาพ', 'อาศัย', 'ไทย', 'แก้', 'กฎหมาย', 'ครอง', 'ที่ดิน', 'รองรับ', 'ปี']


In [199]:
tokens_list = [split_word(txt) for txt in clean_text]

In [200]:
tokens_list

[['คณะ',
  'รัฐมนตรี',
  'ไฟ',
  'เขียว',
  'ชาติ',
  'ซื้อ',
  'ดิน',
  'อสัง',
  'หา',
  'ไทย'],
 ['มติ',
  'คณะ',
  'รัฐมนตรี',
  'ชอบ',
  'วีซ่า',
  'ประเภท',
  'พำนัก',
  'ชาติ',
  'ศักยภาพ',
  'อาศัย',
  'ไทย',
  'แก้',
  'กฎหมาย',
  'ครอง',
  'ที่ดิน',
  'รองรับ',
  'ชอบ',
  'หลัก',
  'มาตรการ',
  'กระตุ้น',
  'เศรษฐกิจ',
  'ลงทุน',
  'ดึงดูด',
  'ชาติ',
  'ศักยภาพ',
  'ประเทศไทย',
  'ลักษณะ',
  'พำนัก',
  'long',
  'term',
  'stay',
  'เป้า',
  'หมาย',
  'ประชากร',
  'โลก',
  'มั่งคั่ง',
  'เกษียณ',
  'อายุ',
  'ประเทศ',
  'ต้องการ',
  'ทำงาน',
  'ประเทศไทย',
  'ทักษะ',
  'เชี่ยวชาญ',
  'พิเศษ'],
 ['ครม',
  'เป้า',
  'ปี',
  'แก้',
  'กฎหมาย',
  'ครอง',
  'ที่ดิน',
  'ดูด',
  'ชาติ',
  'ศักยภาพ',
  'ซื้อ'],
 ['รัช',
  'ดา',
  'เผย',
  'มติ',
  'คณะ',
  'รัฐมนตรี',
  'ชอบ',
  'วีซ่า',
  'ประเภท',
  'พำนัก',
  'ชาติ',
  'ศักยภาพ',
  'อาศัย',
  'ไทย',
  'แก้',
  'กฎหมาย',
  'ครอง',
  'ที่ดิน',
  'รองรับ',
  'ปี']]

**Step 3 + 4a: Bag of words + count word**

In [201]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [202]:
cvec.vocabulary_

{'คณะ': 5,
 'รัฐมนตรี': 31,
 'ไฟ': 51,
 'เขียว': 43,
 'ชาติ': 9,
 'ซื้อ': 10,
 'ดิน': 12,
 'อสัง': 39,
 'หา': 38,
 'ไทย': 50,
 'มติ': 26,
 'ชอบ': 8,
 'วีซ่า': 34,
 'ประเภท': 22,
 'พำนัก': 24,
 'ศักยภาพ': 35,
 'อาศัย': 41,
 'แก้': 48,
 'กฎหมาย': 3,
 'ครอง': 7,
 'ที่ดิน': 18,
 'รองรับ': 29,
 'หลัก': 37,
 'มาตรการ': 28,
 'กระตุ้น': 4,
 'เศรษฐกิจ': 47,
 'ลงทุน': 32,
 'ดึงดูด': 13,
 'ประเทศไทย': 21,
 'ลักษณะ': 33,
 'long': 0,
 'term': 2,
 'stay': 1,
 'เป้า': 45,
 'หมาย': 36,
 'ประชากร': 19,
 'โลก': 49,
 'มั่งคั่ง': 27,
 'เกษียณ': 42,
 'อายุ': 40,
 'ประเทศ': 20,
 'ต้องการ': 15,
 'ทำงาน': 17,
 'ทักษะ': 16,
 'เชี่ยวชาญ': 44,
 'พิเศษ': 25,
 'ครม': 6,
 'ปี': 23,
 'ดูด': 14,
 'รัช': 30,
 'ดา': 11,
 'เผย': 46}

In [203]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
         2, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 1,
         1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
        [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 1, 0, 1, 0, 1, 0]], dtype=int64)

In [204]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1]
 [1 1 1 1 1 1 0 1 2 2 0 0 0 1 0 1 1 1 1 1 1 2 1 0 2 1 1 1 1 1 0 1 1 1 1 2
  1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0]
 [0 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1
  0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0]]


In [205]:
first_news.shape[1]

52

In [206]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 5
This is lowerpart : 23.45207879911715


0.21320071635561041

In [207]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 2
This is lowerpart : 10.488088481701515


0.19069251784911848

In [208]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 4
This is lowerpart : 14.142135623730953


0.28284271247461895

# ข่าวที่ 18

In [209]:
clean_text = ['เจ้าแมลงก้นกระดก ห้ามตบ ห้ามบี้ ห้ามปัด เด็ดขาดเลยนะคะ เพราะอาจทำให้เกิดผื่นผิวหนังอักเสบขึ้นมาได้ค่ะ','ผื่นที่เกิดจากแมลงชนิดนี้ไม่ได้เกิดจากการที่มันกัดหรือต่อย แต่เกิดจากการที่ไปโดนแล้วบี้ทำให้สารเคมีในตัวที่ชื่อว่า paederin จะทำให้ผิวหนังบริเวณที่สัมผัสไหม้มักเเป็นรอยแปลกๆตามที่มือไปสัมผัส จะเห็นว่าแมลงชนิดนี้ไม่ได้อันตราย กัดต่อยแล้วถึงกับชีวิต แต่ สามารทำให้ผิวหนังไหม้พองได้ ดังนั้น ควรป้องกันไว้ก่อนจะดีที่สุด.','แมลงก้นกระดก ห้ามตบเด็ดขาด','ถ้ามาเกาะบนตัวคุณ ห้ามตบมันเด็ดขาด เพราะในตัวมันมีสารพิษชนิดเหลว เมื่อสัมผัสผิวเราจะทำให้ผิวหนังบริเวณที่สัมผัสไหม้มักเเป็นรอยแปลกๆ']

**Step 2: Tokenize**

In [210]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เจ้า', 'แมลง', 'ก้น', 'กระดก', 'ห้าม', 'ตบ', 'ห้าม', 'บี้', 'ห้าม', 'ปัด', 'เด็ด', 'ผื่น', 'ผิวหนัง', 'อักเสบ']
tokenized text:
 ['ผื่น', 'แมลง', 'ชนิด', 'กัด', 'ต่อย', 'โดน', 'บี้', 'สาร', 'เคมี', 'ตัว', 'ชื่อ', 'paederin', 'ผิวหนัง', 'บริเวณ', 'สัมผัส', 'ไหม้', 'เเป็', 'นร', 'อย', 'แปลกๆ', 'มือ', 'สัมผัส', 'แมลง', 'ชนิด', 'อันตราย', 'กัด', 'ต่อย', 'ชีวิต', 'สา', 'มาร', 'ผิวหนัง', 'ไหม้', 'พอง', 'ป้องกัน', 'ดี']
tokenized text:
 ['แมลง', 'ก้น', 'กระดก', 'ห้าม', 'ตบ', 'เด็ด']
tokenized text:
 ['เกาะ', 'ตัว', 'ห้าม', 'ตบ', 'เด็ด', 'ตัว', 'สาร', 'พิษ', 'ชนิด', 'เหลว', 'สัมผัส', 'ผิว', 'ผิวหนัง', 'บริเวณ', 'สัมผัส', 'ไหม้', 'เเป็', 'นร', 'อย', 'แปลกๆ']


In [211]:
tokens_list = [split_word(txt) for txt in clean_text]

In [212]:
tokens_list

[['เจ้า',
  'แมลง',
  'ก้น',
  'กระดก',
  'ห้าม',
  'ตบ',
  'ห้าม',
  'บี้',
  'ห้าม',
  'ปัด',
  'เด็ด',
  'ผื่น',
  'ผิวหนัง',
  'อักเสบ'],
 ['ผื่น',
  'แมลง',
  'ชนิด',
  'กัด',
  'ต่อย',
  'โดน',
  'บี้',
  'สาร',
  'เคมี',
  'ตัว',
  'ชื่อ',
  'paederin',
  'ผิวหนัง',
  'บริเวณ',
  'สัมผัส',
  'ไหม้',
  'เเป็',
  'นร',
  'อย',
  'แปลกๆ',
  'มือ',
  'สัมผัส',
  'แมลง',
  'ชนิด',
  'อันตราย',
  'กัด',
  'ต่อย',
  'ชีวิต',
  'สา',
  'มาร',
  'ผิวหนัง',
  'ไหม้',
  'พอง',
  'ป้องกัน',
  'ดี'],
 ['แมลง', 'ก้น', 'กระดก', 'ห้าม', 'ตบ', 'เด็ด'],
 ['เกาะ',
  'ตัว',
  'ห้าม',
  'ตบ',
  'เด็ด',
  'ตัว',
  'สาร',
  'พิษ',
  'ชนิด',
  'เหลว',
  'สัมผัส',
  'ผิว',
  'ผิวหนัง',
  'บริเวณ',
  'สัมผัส',
  'ไหม้',
  'เเป็',
  'นร',
  'อย',
  'แปลกๆ']]

**Step 3 + 4a: Bag of words + count word**

In [213]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [214]:
cvec.vocabulary_

{'เจ้า': 32,
 'แมลง': 37,
 'ก้น': 3,
 'กระดก': 1,
 'ห้าม': 26,
 'ตบ': 8,
 'บี้': 13,
 'ปัด': 14,
 'เด็ด': 33,
 'ผื่น': 18,
 'ผิวหนัง': 17,
 'อักเสบ': 28,
 'ชนิด': 4,
 'กัด': 2,
 'ต่อย': 10,
 'โดน': 38,
 'สาร': 25,
 'เคมี': 31,
 'ตัว': 9,
 'ชื่อ': 6,
 'paederin': 0,
 'บริเวณ': 12,
 'สัมผัส': 23,
 'ไหม้': 39,
 'เเป็': 35,
 'นร': 11,
 'อย': 27,
 'แปลกๆ': 36,
 'มือ': 22,
 'อันตราย': 29,
 'ชีวิต': 5,
 'สา': 24,
 'มาร': 21,
 'พอง': 19,
 'ป้องกัน': 15,
 'ดี': 7,
 'เกาะ': 30,
 'พิษ': 20,
 'เหลว': 34,
 'ผิว': 16}

In [215]:
c_feat[:,:].todense()

matrix([[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 2, 0, 2, 1, 1, 1, 0, 1, 2, 1, 1, 1, 0, 1, 0, 2, 1, 1, 0,
         1, 1, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 2, 1, 2],
        [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
         0, 0, 2, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]],
       dtype=int64)

In [216]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 3 0 1 0 0 0 1 1 0 0
  0 1 0 0]
 [1 0 2 0 2 1 1 1 0 1 2 1 1 1 0 1 0 2 1 1 0 1 1 2 1 1 0 1 0 1 0 1 0 0 0 1
  1 2 1 2]
 [0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0
  0 1 0 0]
 [0 0 0 0 1 0 0 0 1 2 0 1 1 0 0 0 1 1 0 0 1 0 0 2 0 1 1 1 0 0 1 0 0 1 1 1
  1 0 0 1]]


In [217]:
first_news.shape[1]

40

In [218]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 6
This is lowerpart : 31.304951684997057


0.19166296949998196

In [219]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 10.954451150103322


0.7302967433402214

In [220]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 6
This is lowerpart : 21.908902300206645


0.27386127875258304

# ข่าวที่ 19

In [221]:
clean_text = ['การฉีดวัคซีน Sinopharm ให้กับเด็ก ยังไม่ผ่านการอนุมัติจาก อย. ไทย จริงหรือไม่','“อย.ยังไม่อนุญาต ต้องย้ำว่ายังไม่ได้อนุญาต ไม่ใช่ไม่อนุญาต เพราะยังต้องให้เขาส่งข้อมูลสำคัญ การวิจัยเฟส 3 มาเพิ่มเติมทั้งความปลอดภัยและประสิทธิภาพ รวมถึงปริมาณการใช้วัคซีนในเด็กเล็ก”','อย. ยังไม่อนุมัติใช้ ‘ซิโนฟาร์ม’ ในเด็ก รอบริษัทส่งผลศึกษาเฟส 3 เพิ่มเติม','อย. ยังไม่อนุมัติใช้ ‘ซิโนฟาร์ม’ ในเด็ก รอบริษัทส่งผลศึกษาเฟส 3 เพิ่มเติม รวมถึงขนาดการใช้วัคซีนและระยะห่างในการฉีดวัคซีน']

**Step 2: Tokenize**

In [222]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ฉีด', 'วัคซีน', 'Sinopharm', 'เด็ก', 'อนุมัติ', 'อย', 'ไทย']
tokenized text:
 ['อย', 'อนุญาต', 'ย้ำ', 'อนุญาต', 'อนุญาต', 'ข้อมูล', 'วิจัย', 'เฟส', 'เติม', 'ปลอดภัย', 'ประสิทธิภาพ', 'ปริมาณ', 'วัคซีน', 'เด็ก']
tokenized text:
 ['อย', 'อนุมัติ', 'ซิ', 'โน', 'ฟาร์ม', 'เด็ก', 'รอ', 'บริษัท', 'ศึกษา', 'เฟส', 'เติม']
tokenized text:
 ['อย', 'อนุมัติ', 'ซิ', 'โน', 'ฟาร์ม', 'เด็ก', 'รอ', 'บริษัท', 'ศึกษา', 'เฟส', 'เติม', 'ขนาด', 'วัคซีน', 'ห่าง', 'ฉีด', 'วัคซีน']


In [223]:
tokens_list = [split_word(txt) for txt in clean_text]

In [224]:
tokens_list

[['ฉีด', 'วัคซีน', 'Sinopharm', 'เด็ก', 'อนุมัติ', 'อย', 'ไทย'],
 ['อย',
  'อนุญาต',
  'ย้ำ',
  'อนุญาต',
  'อนุญาต',
  'ข้อมูล',
  'วิจัย',
  'เฟส',
  'เติม',
  'ปลอดภัย',
  'ประสิทธิภาพ',
  'ปริมาณ',
  'วัคซีน',
  'เด็ก'],
 ['อย',
  'อนุมัติ',
  'ซิ',
  'โน',
  'ฟาร์ม',
  'เด็ก',
  'รอ',
  'บริษัท',
  'ศึกษา',
  'เฟส',
  'เติม'],
 ['อย',
  'อนุมัติ',
  'ซิ',
  'โน',
  'ฟาร์ม',
  'เด็ก',
  'รอ',
  'บริษัท',
  'ศึกษา',
  'เฟส',
  'เติม',
  'ขนาด',
  'วัคซีน',
  'ห่าง',
  'ฉีด',
  'วัคซีน']]

**Step 3 + 4a: Bag of words + count word**

In [225]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [226]:
cvec.vocabulary_

{'ฉีด': 3,
 'วัคซีน': 12,
 'Sinopharm': 0,
 'เด็ก': 19,
 'อนุมัติ': 17,
 'อย': 18,
 'ไทย': 23,
 'อนุญาต': 16,
 'ย้ำ': 10,
 'ข้อมูล': 2,
 'วิจัย': 13,
 'เฟส': 21,
 'เติม': 20,
 'ปลอดภัย': 8,
 'ประสิทธิภาพ': 6,
 'ปริมาณ': 7,
 'ซิ': 4,
 'โน': 22,
 'ฟาร์ม': 9,
 'รอ': 11,
 'บริษัท': 5,
 'ศึกษา': 14,
 'ขนาด': 1,
 'ห่าง': 15}

In [227]:
c_feat[:,:].todense()

matrix([[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
         0, 0, 1],
        [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 3, 0, 1, 1, 1,
         1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
         1, 1, 0],
        [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 1,
         1, 1, 0]], dtype=int64)

In [228]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1]
 [0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 3 0 1 1 1 1 0 0]
 [0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0]
 [0 1 0 1 1 1 0 0 0 1 0 1 2 0 1 1 0 1 1 1 1 1 1 0]]


In [229]:
first_news.shape[1]

24

In [230]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 3
This is lowerpart : 11.832159566199234


0.25354627641855493

In [231]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 3
This is lowerpart : 8.774964387392123


0.3418817293789138

In [232]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 6
This is lowerpart : 11.224972160321824


0.5345224838248488

# ข่าวที่ 20

In [233]:
clean_text = ['หลังฉีดวัคซันโควิด จำเป็นต้องตรวจดูปริมาณภูมิคุ้มกันหลังฉีดวัคซีน หรือไม่','ปัจจุบันนี้พบว่า ส่วนใหญ่หรือมากกว่า 90% ของผู้ฉีดวัคซีนโควิดจะมีภูมิคุ้มกัน อีกทั้งยังไม่มีข้อมูลบ่งชี้ถึงระดับภูมิคุ้มกันที่สัมพันธ์กับการป้องกันการติดโรคโควิด-19 (Correlates of protection) ดังนั้นการตรวจภูมิคุ้มกันหลังรับวัคซีน ณ ปัจจุบันจึงเป็นเรื่องของความสบายใจของผู้เข้ารับการตรวจภูมิที่มีทุนทรัพย์เพียงพอ','ฉีดวัคซีนโควิด-19 แล้วจำเป็นต้องตรวจภูมิคุ้มกันไหม? ฟังคำอธิบายชัดๆ จาก ‘สหเวชศาสตร์ ธรรมศาสตร์’','ท่ามกลางความสับสนและความไม่มั่นใจ เกิดเป็นคำถามดังขึ้นให้ได้ยินอย่างหนาหูว่า ที่สุดแล้ว จำเป็นต้อง “ตรวจภูมิ” หลังฉีดวัคซีนโควิด-19 หรือไม่ ?']

**Step 2: Tokenize**

In [234]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ฉีด', 'วัค', 'ซัน', 'โค', 'วิด', 'ตรวจ', 'ดู', 'ปริมาณ', 'ภูมิคุ้มกัน', 'ฉีด', 'วัคซีน']
tokenized text:
 ['ฉีด', 'วัคซีน', 'โค', 'วิด', 'ภูมิคุ้มกัน', 'ไม่มี', 'ข้อมูล', 'บ่ง', 'ชี้', 'ระดับ', 'ภูมิคุ้มกัน', 'สัมพันธ์', 'ป้องกัน', 'ติด', 'โรค', 'โค', 'วิด', 'Correlates', 'of', 'protection', 'ตรวจ', 'ภูมิคุ้มกัน', 'วัคซีน', 'เรื่อง', 'สบายใจ', 'ตรวจ', 'ภูมิ', 'ทุน', 'ทรัพย์']
tokenized text:
 ['ฉีด', 'วัคซีน', 'โค', 'วิด', 'ตรวจ', 'ภูมิคุ้มกัน', 'ไหม', 'ฟัง', 'อธิ', 'บาย', 'ชัดๆ', 'สห', 'เวชศาสตร์', 'ธรรมศาสตร์']
tokenized text:
 ['ท่ามกลาง', 'สับสน', 'มั่นใจ', 'คำถาม', 'ได้ยิน', 'หนาหู', 'ตรวจ', 'ภูมิ', 'ฉีด', 'วัคซีน', 'โค', 'วิด']


In [235]:
tokens_list = [split_word(txt) for txt in clean_text]

In [236]:
tokens_list

[['ฉีด',
  'วัค',
  'ซัน',
  'โค',
  'วิด',
  'ตรวจ',
  'ดู',
  'ปริมาณ',
  'ภูมิคุ้มกัน',
  'ฉีด',
  'วัคซีน'],
 ['ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'ภูมิคุ้มกัน',
  'ไม่มี',
  'ข้อมูล',
  'บ่ง',
  'ชี้',
  'ระดับ',
  'ภูมิคุ้มกัน',
  'สัมพันธ์',
  'ป้องกัน',
  'ติด',
  'โรค',
  'โค',
  'วิด',
  'Correlates',
  'of',
  'protection',
  'ตรวจ',
  'ภูมิคุ้มกัน',
  'วัคซีน',
  'เรื่อง',
  'สบายใจ',
  'ตรวจ',
  'ภูมิ',
  'ทุน',
  'ทรัพย์'],
 ['ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'ตรวจ',
  'ภูมิคุ้มกัน',
  'ไหม',
  'ฟัง',
  'อธิ',
  'บาย',
  'ชัดๆ',
  'สห',
  'เวชศาสตร์',
  'ธรรมศาสตร์'],
 ['ท่ามกลาง',
  'สับสน',
  'มั่นใจ',
  'คำถาม',
  'ได้ยิน',
  'หนาหู',
  'ตรวจ',
  'ภูมิ',
  'ฉีด',
  'วัคซีน',
  'โค',
  'วิด']]

**Step 3 + 4a: Bag of words + count word**

In [237]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [238]:
cvec.vocabulary_

{'ฉีด': 5,
 'วัค': 25,
 'ซัน': 8,
 'โค': 36,
 'วิด': 27,
 'ตรวจ': 10,
 'ดู': 9,
 'ปริมาณ': 18,
 'ภูมิคุ้มกัน': 22,
 'วัคซีน': 26,
 'ไม่มี': 39,
 'ข้อมูล': 3,
 'บ่ง': 17,
 'ชี้': 7,
 'ระดับ': 24,
 'สัมพันธ์': 31,
 'ป้องกัน': 19,
 'ติด': 11,
 'โรค': 37,
 'Correlates': 0,
 'of': 1,
 'protection': 2,
 'เรื่อง': 34,
 'สบายใจ': 28,
 'ภูมิ': 21,
 'ทุน': 13,
 'ทรัพย์': 12,
 'ไหม': 40,
 'ฟัง': 20,
 'อธิ': 33,
 'บาย': 16,
 'ชัดๆ': 6,
 'สห': 29,
 'เวชศาสตร์': 35,
 'ธรรมศาสตร์': 15,
 'ท่ามกลาง': 14,
 'สับสน': 30,
 'มั่นใจ': 23,
 'คำถาม': 4,
 'ได้ยิน': 38,
 'หนาหู': 32}

In [239]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
         1, 3, 0, 1, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
         0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]],
       dtype=int64)

In [240]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 2 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0
  1 0 0 0 0]
 [1 1 1 1 0 1 0 1 0 0 2 1 1 1 0 0 0 1 0 1 0 1 3 0 1 0 2 2 1 0 0 1 0 0 1 0
  2 1 0 1 0]
 [0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1
  1 0 0 0 1]
 [0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0
  1 0 1 0 0]]


In [241]:
first_news.shape[1]

41

In [242]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 13
This is lowerpart : 23.643180835073775


0.5498414147691577

In [243]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 13.49073756323204


0.5188745216627709

In [244]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 6
This is lowerpart : 12.489995996796795


0.4803844614152615

# ข่าวที่ 21

In [245]:
clean_text = ['เห็นกันว่าพืชกระท่อมนั้นถูกเปลี่ยนให้ถูกกฎหมายแล้วนำมาต้มได้ส่วนผมสงสัยว่าแล้วถ้าเรานำมาต้มแล้วผสมกับยาแก้ไอแล้วจะผิดกฎหมายเปล่าครับ','ปัจจุบันพืชกระท่อมพ้นจากบัญชียาเสพติดประเภท 5 แล้ว และสามารถนำมาใช้ในวิถีชีวิตท้องถิ่น ทั้งการเคี้ยวใบ การปลูก การครอบครองและการขายใบสดที่ไม่ได้ปรุงหรือทำเป็นอาหารได้อย่างเสรีไม่ผิดกฎหมาย กำหนดให้อาหารที่ปรุงจากพืชกระท่อมเป็นอาหารที่ห้ามผลิต นำเข้าหรือจำหน่ายหรือแม้กระทั่งน้ำต้มกระท่อมที่ไม่ได้ผสมกับสิ่งใดเลยก็เป็นสิ่งที่ห้ามผลิตเพื่อจำหน่ายตามประกาศฉบับนี้ หากการฝ่าฝืน ผลิต และขายอาหาร ที่ พ.ร.บ.อาหาร ห้าม มีโทษตามมาตรา','ยังไม่ปลดล็อก! เตือน ‘ต้มน้ำท่อม-ทำอาหาร-ผสมสมุนไพร’ ยังผิดกฎหมาย','"สมศักดิ์" เตือนต้มน้ำท่อม-ทำอาหาร-ผสมสมุนไพร ยังผิดก.ม. แนะ สธ.ปลดล็อกกฎหมายแปรรูปกระท่อมผสมอาหาร-ทำผลิตภัณฑ์สมุนไพรจำหน่าย ชี้เป็นอุปสรรคการค้าวิถีชาวบ้าน']

**Step 2: Tokenize**

In [246]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['พืช', 'กระท่อม', 'กฎหมาย', 'ต้ม', 'ผม', 'สงสัย', 'ต้ม', 'ผสม', 'ยา', 'แก้', 'ไอ', 'กฎหมาย', 'เปล่า']
tokenized text:
 ['พืช', 'กระท่อม', 'พ้น', 'บัญชี', 'ยา', 'เสพ', 'ติด', 'ประเภท', 'วิถี', 'ชีวิต', 'ท้อง', 'ถิ่น', 'เคี้ยว', 'ใบ', 'ปลูก', 'ครอบ', 'ครอง', 'ขาย', 'ใบ', 'สด', 'ปรุง', 'ทำ', 'อาหาร', 'เสรี', 'กฎหมาย', 'อาหาร', 'ปรุง', 'พืช', 'กระท่อม', 'อาหาร', 'ห้าม', 'ผลิต', 'จำหน่าย', 'น้ำ', 'ต้ม', 'กระท่อม', 'ผสม', 'ใด', 'ห้าม', 'ผลิต', 'จำหน่าย', 'ประกาศ', 'ฉบับ', 'ฝ่าฝืน', 'ผลิต', 'ขาย', 'อาหาร', 'พ', 'ร', 'บ', 'อาหาร', 'ห้าม', 'โทษ', 'มาตรา']
tokenized text:
 ['ปลด', 'ล็อก', 'เตือน', 'ต้ม', 'น้ำ', 'ท่อม', 'ทำ', 'อาหาร', 'ผสม', 'สมุนไพร', 'กฎหมาย']
tokenized text:
 ['สม', 'ศักดิ์', 'เตือน', 'ต้ม', 'น้ำ', 'ท่อม', 'ทำ', 'อาหาร', 'ผสม', 'สมุนไพร', 'ผิดก', 'ม', 'แนะ', 'สธ', 'ปลด', 'ล็อก', 'กฎหมาย', 'แปรรูป', 'กระท่อม', 'ผสม', 'อาหาร', 'ทำ', 'ผลิตภัณฑ์', 'สมุนไพร', 'จำหน่าย', 'ชี้', 'อุปสรรค', 'ค้า', 'วิถี', 'บ้าน']


In [247]:
tokens_list = [split_word(txt) for txt in clean_text]

In [248]:
tokens_list

[['พืช',
  'กระท่อม',
  'กฎหมาย',
  'ต้ม',
  'ผม',
  'สงสัย',
  'ต้ม',
  'ผสม',
  'ยา',
  'แก้',
  'ไอ',
  'กฎหมาย',
  'เปล่า'],
 ['พืช',
  'กระท่อม',
  'พ้น',
  'บัญชี',
  'ยา',
  'เสพ',
  'ติด',
  'ประเภท',
  'วิถี',
  'ชีวิต',
  'ท้อง',
  'ถิ่น',
  'เคี้ยว',
  'ใบ',
  'ปลูก',
  'ครอบ',
  'ครอง',
  'ขาย',
  'ใบ',
  'สด',
  'ปรุง',
  'ทำ',
  'อาหาร',
  'เสรี',
  'กฎหมาย',
  'อาหาร',
  'ปรุง',
  'พืช',
  'กระท่อม',
  'อาหาร',
  'ห้าม',
  'ผลิต',
  'จำหน่าย',
  'น้ำ',
  'ต้ม',
  'กระท่อม',
  'ผสม',
  'ใด',
  'ห้าม',
  'ผลิต',
  'จำหน่าย',
  'ประกาศ',
  'ฉบับ',
  'ฝ่าฝืน',
  'ผลิต',
  'ขาย',
  'อาหาร',
  'พ',
  'ร',
  'บ',
  'อาหาร',
  'ห้าม',
  'โทษ',
  'มาตรา'],
 ['ปลด',
  'ล็อก',
  'เตือน',
  'ต้ม',
  'น้ำ',
  'ท่อม',
  'ทำ',
  'อาหาร',
  'ผสม',
  'สมุนไพร',
  'กฎหมาย'],
 ['สม',
  'ศักดิ์',
  'เตือน',
  'ต้ม',
  'น้ำ',
  'ท่อม',
  'ทำ',
  'อาหาร',
  'ผสม',
  'สมุนไพร',
  'ผิดก',
  'ม',
  'แนะ',
  'สธ',
  'ปลด',
  'ล็อก',
  'กฎหมาย',
  'แปรรูป',
  'กระท่อม',
  'ผสม',
  'อาหาร',
  'ทำ',

**Step 3 + 4a: Bag of words + count word**

In [249]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [250]:
cvec.vocabulary_

{'พืช': 32,
 'กระท่อม': 1,
 'กฎหมาย': 0,
 'ต้ม': 11,
 'ผม': 25,
 'สงสัย': 41,
 'ผสม': 28,
 'ยา': 36,
 'แก้': 54,
 'ไอ': 60,
 'เปล่า': 51,
 'พ้น': 33,
 'บัญชี': 18,
 'เสพ': 52,
 'ติด': 10,
 'ประเภท': 21,
 'วิถี': 39,
 'ชีวิต': 8,
 'ท้อง': 15,
 'ถิ่น': 12,
 'เคี้ยว': 49,
 'ใบ': 59,
 'ปลูก': 24,
 'ครอบ': 4,
 'ครอง': 3,
 'ขาย': 2,
 'สด': 42,
 'ปรุง': 22,
 'ทำ': 13,
 'อาหาร': 47,
 'เสรี': 53,
 'ห้าม': 46,
 'ผลิต': 26,
 'จำหน่าย': 6,
 'น้ำ': 16,
 'ใด': 58,
 'ประกาศ': 20,
 'ฉบับ': 7,
 'ฝ่าฝืน': 30,
 'พ': 31,
 'ร': 37,
 'บ': 17,
 'โทษ': 57,
 'มาตรา': 35,
 'ปลด': 23,
 'ล็อก': 38,
 'เตือน': 50,
 'ท่อม': 14,
 'สมุนไพร': 45,
 'สม': 44,
 'ศักดิ์': 40,
 'ผิดก': 29,
 'ม': 34,
 'แนะ': 55,
 'สธ': 43,
 'แปรรูป': 56,
 'ผลิตภัณฑ์': 27,
 'ชี้': 9,
 'อุปสรรค': 48,
 'ค้า': 5,
 'บ้าน': 19}

In [251]:
c_feat[:,:].todense()

matrix([[2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [1, 3, 2, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
         1, 2, 0, 1, 0, 3, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 0, 1, 0, 0,
         1, 0, 0, 0, 3, 5, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 2, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 2, 1, 0, 1, 0, 0, 1, 0,
         0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
         0, 1, 1, 2, 0, 2, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]],
       dtype=int64)

In [252]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[2 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0
  1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1]
 [1 3 2 1 1 0 2 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 2 0 1 0 3 0 1 0 1 1 2 1 0 1
  1 1 0 1 0 0 1 0 0 0 3 5 0 1 0 0 1 1 0 0 0 1 1 2 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 1 1 0 0 1 0 1 0 2 1 0 1 0 0 1 0 0 0 1 0 0 0 1 2 1 0 0 0 0 1 0
  0 0 1 1 1 0 0 1 1 2 0 2 1 0 1 0 0 0 0 1 1 0 0 0 0]]


In [253]:
first_news.shape[1]

61

In [254]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 11
This is lowerpart : 41.641325627314025


0.2641606585354408

In [255]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 5
This is lowerpart : 13.674794331177344


0.36563621206356534

In [256]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 25.41653005427767


0.2754113163776218

# ข่าวที่ 22

In [257]:
clean_text = ['กินไข่ต้ม สามารถช่วยเรื่องการลลดน้ำหนักได้ โดยผลการวิจัยจากศูนย์ Rochester Centre for Obesity in America และมหาวิทยาลัยลุยเซียนา ในประเทศสหรัฐอเมริกา ที่ได้ออกมาทำการวิจัยพร้อมเผยว่า กินไข่เป็นอาหารเช้าสามารถช่วย จำกัด ปริมาณแคลอรี่ของคุณตลอดทั้งวัน ได้มากกว่า 400 กิโลแคลอรี','รับประทานอาหารเช้าที่มีไข่เป็นส่วนประกอบ หรืออาหารเช้าแบบเบเกิล ซึ่งมี กิโลแคลอรี ในปริมาณเท่ากันและระดับโปรตีนเกือบเท่ากัน นักวิจัยได้บันทึกพฤติกรรมการกินของผู้หญิงและพบว่าก่อนอาหารกลางวันผู้หญิงที่กินไข่เป็นอาหารเช้ารู้สึกหิวน้อยลง','"ไข่ต้ม"ทานแล้ว ช่วยลดน้ำหนักได้ มีผลวิจัย','เรื่องสุขภาพถือเป็นเรื่องสำคัญที่มองข้ามไม่ได้ หลายคนมองหาวิธีลดความอ้วน ผลการวิจัยในประเทศสหรัฐอเมริกาออกมาเผยว่า กินไข่เป็นอาหารเช้าสามารถช่วย จำกัด ปริมาณแคลอรี่ของคุณตลอดทั้งวันได้']

**Step 2: Tokenize**

In [258]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['กิน', 'ไข่', 'ต้ม', 'เรื่อ', 'งการล', 'ลด', 'น้ำ', 'หนัก', 'วิจัย', 'ศูนย์', 'Rochester', 'Centre', 'for', 'Obesity', 'in', 'America', 'มหาวิทยาลัย', 'ลุยเซียนา', 'ประเทศ', 'สหรัฐอเมริกา', 'ทำการ', 'วิจัย', 'เผย', 'กิน', 'ไข่', 'อาหาร', 'เช้า', 'จำกัด', 'ปริ', 'มาณ', 'แคลอรี่', 'กิโล', 'แคลอรี']
tokenized text:
 ['ประทาน', 'อาหาร', 'เช้า', 'ไข่', 'อาหาร', 'เช้า', 'เบ', 'เกิล', 'กิโล', 'แคลอรี', 'ปริมาณ', 'ระดับ', 'โปรตีน', 'วิจัย', 'บันทึก', 'พฤติกรรม', 'กิน', 'หญิง', 'อาหาร', 'กลาง', 'หญิง', 'กิน', 'ไข่', 'อาหาร', 'เช้า', 'รู้สึก', 'หิว']
tokenized text:
 ['ไข่', 'ต้ม', 'ทาน', 'ลด', 'น้ำ', 'หนัก', 'วิจัย']
tokenized text:
 ['เรื่อง', 'สุขภาพ', 'เรื่อง', 'ข้าม', 'คน', 'หา', 'วิธี', 'ลด', 'อ้วน', 'วิจัย', 'ประเทศ', 'สหรัฐอเมริกา', 'เผย', 'กิน', 'ไข่', 'อาหาร', 'เช้า', 'จำกัด', 'ปริ', 'มาณ', 'แคลอรี่']


In [259]:
tokens_list = [split_word(txt) for txt in clean_text]

In [260]:
tokens_list

[['กิน',
  'ไข่',
  'ต้ม',
  'เรื่อ',
  'งการล',
  'ลด',
  'น้ำ',
  'หนัก',
  'วิจัย',
  'ศูนย์',
  'Rochester',
  'Centre',
  'for',
  'Obesity',
  'in',
  'America',
  'มหาวิทยาลัย',
  'ลุยเซียนา',
  'ประเทศ',
  'สหรัฐอเมริกา',
  'ทำการ',
  'วิจัย',
  'เผย',
  'กิน',
  'ไข่',
  'อาหาร',
  'เช้า',
  'จำกัด',
  'ปริ',
  'มาณ',
  'แคลอรี่',
  'กิโล',
  'แคลอรี'],
 ['ประทาน',
  'อาหาร',
  'เช้า',
  'ไข่',
  'อาหาร',
  'เช้า',
  'เบ',
  'เกิล',
  'กิโล',
  'แคลอรี',
  'ปริมาณ',
  'ระดับ',
  'โปรตีน',
  'วิจัย',
  'บันทึก',
  'พฤติกรรม',
  'กิน',
  'หญิง',
  'อาหาร',
  'กลาง',
  'หญิง',
  'กิน',
  'ไข่',
  'อาหาร',
  'เช้า',
  'รู้สึก',
  'หิว'],
 ['ไข่', 'ต้ม', 'ทาน', 'ลด', 'น้ำ', 'หนัก', 'วิจัย'],
 ['เรื่อง',
  'สุขภาพ',
  'เรื่อง',
  'ข้าม',
  'คน',
  'หา',
  'วิธี',
  'ลด',
  'อ้วน',
  'วิจัย',
  'ประเทศ',
  'สหรัฐอเมริกา',
  'เผย',
  'กิน',
  'ไข่',
  'อาหาร',
  'เช้า',
  'จำกัด',
  'ปริ',
  'มาณ',
  'แคลอรี่']]

**Step 3 + 4a: Bag of words + count word**

In [261]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [262]:
cvec.vocabulary_

{'กิน': 7,
 'ไข่': 49,
 'ต้ม': 13,
 'เรื่อ': 44,
 'งการล': 11,
 'ลด': 27,
 'น้ำ': 16,
 'หนัก': 35,
 'วิจัย': 29,
 'ศูนย์': 31,
 'Rochester': 3,
 'Centre': 1,
 'for': 4,
 'Obesity': 2,
 'in': 5,
 'America': 0,
 'มหาวิทยาลัย': 23,
 'ลุยเซียนา': 28,
 'ประเทศ': 19,
 'สหรัฐอเมริกา': 32,
 'ทำการ': 15,
 'เผย': 43,
 'อาหาร': 38,
 'เช้า': 41,
 'จำกัด': 12,
 'ปริ': 20,
 'มาณ': 24,
 'แคลอรี่': 47,
 'กิโล': 8,
 'แคลอรี': 46,
 'ประทาน': 18,
 'เบ': 42,
 'เกิล': 40,
 'ปริมาณ': 21,
 'ระดับ': 25,
 'โปรตีน': 48,
 'บันทึก': 17,
 'พฤติกรรม': 22,
 'หญิง': 34,
 'กลาง': 6,
 'รู้สึก': 26,
 'หิว': 37,
 'ทาน': 14,
 'เรื่อง': 45,
 'สุขภาพ': 33,
 'ข้าม': 9,
 'คน': 10,
 'หา': 36,
 'วิธี': 30,
 'อ้วน': 39}

In [263]:
c_feat[:,:].todense()

matrix([[1, 1, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
         0, 0, 1, 1, 0, 0, 1, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
         0, 1, 1, 0, 1, 1, 0, 2],
        [0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 4, 0, 1, 3,
         1, 0, 0, 0, 1, 0, 1, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
         0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
         0, 1, 0, 2, 0, 1, 0, 1]], dtype=int64)

In [264]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 1 1 1 1 1 0 2 1 0 0 1 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 2 0 1 1 0 0 1
  0 0 1 0 0 1 0 1 1 0 1 1 0 2]
 [0 0 0 0 0 0 1 2 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 2 0
  0 1 4 0 1 3 1 0 0 0 1 0 1 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 1 1 0 0
  1 0 1 1 0 1 0 1 0 2 0 1 0 1]]


In [265]:
first_news.shape[1]

50

In [266]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 19
This is lowerpart : 44.59820624195552


0.4260261028643312

In [267]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 16.522711641858308


0.48418202613504185

In [268]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 16
This is lowerpart : 29.949958263743873


0.5342244506353423

# ข่าวที่ 23

In [269]:
clean_text = ['คณะรัฐมนตรี อนุมัติหลักการให้บริษัทหรือห้างหุ้นส่วนนิติบุคคลนำรายจ่ายค่าซื้อชุดตรวจโควิด-19 แบบเร่งด่วน (Antigen Test Kit) สำหรับพนักงานหรือลูกจ้าง มาหักเป็นรายจ่ายได้ 1.5 เท่า มีผลจนถึง 31 มีนาคม 2565 เพื่อให้ผู้ประกอบการจัดหาชุดตรวจ ATK ให้พนักงานหรือลูกจ้างใช้ตรวจหาเชื้อโควิด-19 ตามมาตรการที่กระทรวงสาธารณสุขกำหนด','ครม.ไฟเขียว สรรพากรออกมาตรการภาษีเพื่อสนับสนุนรายจ่ายค่าชื้อชุดตรวจโควิด ให้บริษัทหรือห้างหุ้นส่วนนิติบุคคล หักรายจ่ายได้ 1.5 เท่า กรณีซื้อชุดตรวจ ATK ตรวจโควิดพนักงาน ถึง 31 มี.ค. 65','สรรพากรให้บริษัทซื้อชุดตรวจ ATK หักรายจ่ายได้ 1.5 เท่า','ครม.ไฟเขียว สรรพากรออกมาตรการภาษีเพื่อสนับสนุนรายจ่ายค่าชื้อชุดตรวจโควิด ให้บริษัทหรือห้างหุ้นส่วนนิติบุคคล หักรายจ่ายได้ 1.5 เท่า กรณีซื้อชุดตรวจ ATK ตรวจโควิดพนักงาน ถึง 31 มี.ค. 65']

**Step 2: Tokenize**

In [270]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['คณะ', 'รัฐมนตรี', 'อนุมัติ', 'หลัก', 'บริษัท', 'ห้าง', 'หุ้น', 'นิติบุคคล', 'จ่าย', 'ค่า', 'ซื้อ', 'ชุด', 'ตรวจ', 'โค', 'วิด', 'เร่ง', 'ด่วน', 'Antigen', 'Test', 'Kit', 'สำหรับ', 'พนักงาน', 'ลูกจ้าง', 'หัก', 'จ่าย', '1.5', 'มีนาคม', 'ชุด', 'ตรวจ', 'ATK', 'พนักงาน', 'ลูกจ้าง', 'ตรวจ', 'หา', 'เชื้อ', 'โค', 'วิด', 'มาตรการ', 'กระทรวง', 'สาธารณสุข']
tokenized text:
 ['ครม', 'ไฟ', 'เขียว', 'สรรพากร', 'มาตรการ', 'ภาษี', 'สนับสนุน', 'จ่าย', 'ค่า', 'ชื้อ', 'ชุด', 'ตรวจ', 'โค', 'วิด', 'บริษัท', 'ห้าง', 'หุ้น', 'นิติบุคคล', 'หัก', 'จ่าย', '1.5', 'กรณี', 'ซื้อ', 'ชุด', 'ตรวจ', 'ATK', 'ตรวจ', 'โค', 'วิด', 'พนักงาน', 'ค']
tokenized text:
 ['สรรพากร', 'บริษัท', 'ซื้อ', 'ชุด', 'ตรวจ', 'ATK', 'หัก', 'จ่าย', '1.5']
tokenized text:
 ['ครม', 'ไฟ', 'เขียว', 'สรรพากร', 'มาตรการ', 'ภาษี', 'สนับสนุน', 'จ่าย', 'ค่า', 'ชื้อ', 'ชุด', 'ตรวจ', 'โค', 'วิด', 'บริษัท', 'ห้าง', 'หุ้น', 'นิติบุคคล', 'หัก', 'จ่าย', '1.5', 'กรณี', 'ซื้อ', 'ชุด', 'ตรวจ', 'ATK', 'ตรวจ', 'โค', 'วิด', 'พนักงาน', 'ค']


In [271]:
tokens_list = [split_word(txt) for txt in clean_text]

In [272]:
tokens_list

[['คณะ',
  'รัฐมนตรี',
  'อนุมัติ',
  'หลัก',
  'บริษัท',
  'ห้าง',
  'หุ้น',
  'นิติบุคคล',
  'จ่าย',
  'ค่า',
  'ซื้อ',
  'ชุด',
  'ตรวจ',
  'โค',
  'วิด',
  'เร่ง',
  'ด่วน',
  'Antigen',
  'Test',
  'Kit',
  'สำหรับ',
  'พนักงาน',
  'ลูกจ้าง',
  'หัก',
  'จ่าย',
  '1.5',
  'มีนาคม',
  'ชุด',
  'ตรวจ',
  'ATK',
  'พนักงาน',
  'ลูกจ้าง',
  'ตรวจ',
  'หา',
  'เชื้อ',
  'โค',
  'วิด',
  'มาตรการ',
  'กระทรวง',
  'สาธารณสุข'],
 ['ครม',
  'ไฟ',
  'เขียว',
  'สรรพากร',
  'มาตรการ',
  'ภาษี',
  'สนับสนุน',
  'จ่าย',
  'ค่า',
  'ชื้อ',
  'ชุด',
  'ตรวจ',
  'โค',
  'วิด',
  'บริษัท',
  'ห้าง',
  'หุ้น',
  'นิติบุคคล',
  'หัก',
  'จ่าย',
  '1.5',
  'กรณี',
  'ซื้อ',
  'ชุด',
  'ตรวจ',
  'ATK',
  'ตรวจ',
  'โค',
  'วิด',
  'พนักงาน',
  'ค'],
 ['สรรพากร', 'บริษัท', 'ซื้อ', 'ชุด', 'ตรวจ', 'ATK', 'หัก', 'จ่าย', '1.5'],
 ['ครม',
  'ไฟ',
  'เขียว',
  'สรรพากร',
  'มาตรการ',
  'ภาษี',
  'สนับสนุน',
  'จ่าย',
  'ค่า',
  'ชื้อ',
  'ชุด',
  'ตรวจ',
  'โค',
  'วิด',
  'บริษัท',
  'ห้าง',
  'หุ้น',
  'นิ

**Step 3 + 4a: Bag of words + count word**

In [273]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [274]:
cvec.vocabulary_

{'คณะ': 8,
 'รัฐมนตรี': 23,
 'อนุมัติ': 35,
 'หลัก': 30,
 'บริษัท': 18,
 'ห้าง': 34,
 'หุ้น': 33,
 'นิติบุคคล': 17,
 'จ่าย': 11,
 'ค่า': 10,
 'ซื้อ': 14,
 'ชุด': 13,
 'ตรวจ': 16,
 'โค': 39,
 'วิด': 25,
 'เร่ง': 38,
 'ด่วน': 15,
 'Antigen': 2,
 'Test': 4,
 'Kit': 3,
 'สำหรับ': 29,
 'พนักงาน': 19,
 'ลูกจ้าง': 24,
 'หัก': 31,
 '1.5': 0,
 'มีนาคม': 22,
 'ATK': 1,
 'หา': 32,
 'เชื้อ': 37,
 'มาตรการ': 21,
 'กระทรวง': 6,
 'สาธารณสุข': 28,
 'ครม': 9,
 'ไฟ': 40,
 'เขียว': 36,
 'สรรพากร': 27,
 'ภาษี': 20,
 'สนับสนุน': 26,
 'ชื้อ': 12,
 'กรณี': 5,
 'ค': 7}

In [275]:
c_feat[:,:].todense()

matrix([[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 1, 3, 1, 1, 2, 0,
         1, 1, 1, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 0],
        [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 1, 2, 1, 0, 3, 1, 1, 1, 1,
         1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 1, 2, 1, 0, 3, 1, 1, 1, 1,
         1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 1]],
       dtype=int64)

In [276]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 1 1 1 1 0 1 0 1 0 1 2 0 2 1 1 3 1 1 2 0 1 1 1 2 2 0 0 1 1 1 1 1 1 1 1
  0 1 1 2 0]
 [1 1 0 0 0 1 0 1 0 1 1 2 1 2 1 0 3 1 1 1 1 1 0 0 0 2 1 1 0 0 0 1 0 1 1 0
  1 0 0 2 1]
 [1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
  0 0 0 0 0]
 [1 1 0 0 0 1 0 1 0 1 1 2 1 2 1 0 3 1 1 1 1 1 0 0 0 2 1 1 0 0 0 1 0 1 1 0
  1 0 0 2 1]]


In [277]:
first_news.shape[1]

41

In [278]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 37
This is lowerpart : 51.08815909777921


0.7242382707348008

In [279]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 12
This is lowerpart : 22.847319317591726


0.5252257314388902

In [280]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 37
This is lowerpart : 51.08815909777921


0.7242382707348008

# ข่าวที่ 24

In [281]:
clean_text = ['แอปเป๋าตังมีดีกว่าที่คิด ไม่น่าเชื่อว่าในเป๋าตัง ก็มีเรื่องสุขภาพใกล้ตัวทุกเพศวัยตั้งแต่แรกเกิด จนถึงผู้สูงอายุ เช่นบริการปรึกษาโรคซึมเศร้าและตรวจสอบความสามารถในการช่วยเหลือตนเองมีบริการสำหรับผู้หญิง เช่น สิทธิตรวจคัดกรองมะเร็งปากมดลูก และสิทธิฝากครรภ์ใครมีแอปเป๋าตังแล้ว ลองกดเข้าไปปรึกษาเรื่องสุขภาพได้','“กระเป๋าสุขภาพ หรือ Health Wallet” ให้บริการสร้างเสริมสุขภาพและป้องกันโรคก่อนเกิดการรักษาแก่คนไทยทุกคนฟรี! สามารถเช็คสิทธิประโยชน์ของตัวเองได้ มีการเก็บข้อมูลประวัติการรักษาจะเชื่อมโยงกับสำนักงานหลักประกันสุขภาพถ้วนหน้า สามารถเช็คสถานพยาบาลที่ใกล้บ้านหรือสะดวกเพื่อทำนัดพบหมอ ฉีดวัคซีน ตรวจคัดกรองโรคต่างๆ','แอปฯ เป๋าตัง เปิดฟีเจอร์ ตรวจสอบสิทธิ-นัดหมอ “โรคซึมเศร้าในผู้สูงวัย” ผ่านวอลเล็ต “กระเป๋าสุขภาพ”','เพราะการป้องกันย่อมดีกว่าการรักษา...ระบบสาธารณสุขทั่วโลกจึงใส่ใจและให้ความสำคัญกับการป้องกันและสร้างเสริมความรู้ด้านสุขภาพให้แก่ประชาชนไม่น้อยไปกว่าการค้นหาวิธีการรักษาโรค']

**Step 2: Tokenize**

In [282]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['แอ', 'ป', 'เป๋า', 'ตัง', 'ดี', 'เป๋า', 'ตัง', 'เรื่อง', 'สุขภาพ', 'ตัว', 'เพศ', 'วัย', 'อายุ', 'บริการ', 'ปรึกษา', 'โรค', 'ซึม', 'เศร้า', 'ตรวจ', 'สอบ', 'บริการ', 'สำหรับ', 'หญิง', 'สิทธิ', 'ตรวจ', 'คัด', 'กรอง', 'มะเร็ง', 'ปาก', 'มดลูก', 'สิทธิ', 'ฝาก', 'ครรภ์', 'แอ', 'ป', 'เป๋า', 'ตัง', 'ลอง', 'กด', 'เข้าไป', 'ปรึกษา', 'เรื่อง', 'สุขภาพ']
tokenized text:
 ['กระเป๋า', 'สุขภาพ', 'Health', 'Wallet', 'บริการ', 'สร้าง', 'เสริมสุข', 'ภาพ', 'ป้องกัน', 'โรค', 'รักษา', 'คน', 'ไทย', 'คน', 'ฟรี', 'เช็ค', 'สิทธิ', 'ประโยชน์', 'ตัว', 'ข้อมูล', 'ประวัติการ', 'รักษา', 'เชื่อม', 'โยง', 'สำนักงาน', 'หลัก', 'ประกัน', 'สุขภาพ', 'ถ้วน', 'หน้า', 'เช็ค', 'สถาน', 'พยาบาล', 'บ้าน', 'สะดวก', 'ทำ', 'นัด', 'หมอ', 'ฉีด', 'วัคซีน', 'ตรวจ', 'คัด', 'กรอง', 'โรค']
tokenized text:
 ['แอปฯ', 'เป๋า', 'ตัง', 'ฟีเจอร์', 'ตรวจ', 'สอบ', 'สิทธิ', 'นัด', 'หมอ', 'โรค', 'ซึม', 'เศร้า', 'วัย', 'ผ่า', 'นว', 'อลเล็ต', 'กระเป๋า', 'สุขภาพ']
tokenized text:
 ['ป้องกัน', 'ดี', 'รักษา', 'ระบบ', 'สาธารณสุข', 'โลก', 

In [283]:
tokens_list = [split_word(txt) for txt in clean_text]

In [284]:
tokens_list

[['แอ',
  'ป',
  'เป๋า',
  'ตัง',
  'ดี',
  'เป๋า',
  'ตัง',
  'เรื่อง',
  'สุขภาพ',
  'ตัว',
  'เพศ',
  'วัย',
  'อายุ',
  'บริการ',
  'ปรึกษา',
  'โรค',
  'ซึม',
  'เศร้า',
  'ตรวจ',
  'สอบ',
  'บริการ',
  'สำหรับ',
  'หญิง',
  'สิทธิ',
  'ตรวจ',
  'คัด',
  'กรอง',
  'มะเร็ง',
  'ปาก',
  'มดลูก',
  'สิทธิ',
  'ฝาก',
  'ครรภ์',
  'แอ',
  'ป',
  'เป๋า',
  'ตัง',
  'ลอง',
  'กด',
  'เข้าไป',
  'ปรึกษา',
  'เรื่อง',
  'สุขภาพ'],
 ['กระเป๋า',
  'สุขภาพ',
  'Health',
  'Wallet',
  'บริการ',
  'สร้าง',
  'เสริมสุข',
  'ภาพ',
  'ป้องกัน',
  'โรค',
  'รักษา',
  'คน',
  'ไทย',
  'คน',
  'ฟรี',
  'เช็ค',
  'สิทธิ',
  'ประโยชน์',
  'ตัว',
  'ข้อมูล',
  'ประวัติการ',
  'รักษา',
  'เชื่อม',
  'โยง',
  'สำนักงาน',
  'หลัก',
  'ประกัน',
  'สุขภาพ',
  'ถ้วน',
  'หน้า',
  'เช็ค',
  'สถาน',
  'พยาบาล',
  'บ้าน',
  'สะดวก',
  'ทำ',
  'นัด',
  'หมอ',
  'ฉีด',
  'วัคซีน',
  'ตรวจ',
  'คัด',
  'กรอง',
  'โรค'],
 ['แอปฯ',
  'เป๋า',
  'ตัง',
  'ฟีเจอร์',
  'ตรวจ',
  'สอบ',
  'สิทธิ',
  'นัด',
  'หมอ',
  'โรค

**Step 3 + 4a: Bag of words + count word**

In [285]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [286]:
cvec.vocabulary_

{'แอ': 69,
 'ป': 22,
 'เป๋า': 63,
 'ตัง': 14,
 'ดี': 12,
 'เรื่อง': 65,
 'สุขภาพ': 53,
 'ตัว': 15,
 'เพศ': 64,
 'วัย': 43,
 'อายุ': 59,
 'บริการ': 20,
 'ปรึกษา': 27,
 'โรค': 72,
 'ซึม': 11,
 'เศร้า': 66,
 'ตรวจ': 13,
 'สอบ': 47,
 'สำหรับ': 51,
 'หญิง': 54,
 'สิทธิ': 52,
 'คัด': 8,
 'กรอง': 3,
 'มะเร็ง': 37,
 'ปาก': 28,
 'มดลูก': 36,
 'ฝาก': 31,
 'ครรภ์': 7,
 'ลอง': 41,
 'กด': 2,
 'เข้าไป': 60,
 'กระเป๋า': 4,
 'Health': 0,
 'Wallet': 1,
 'สร้าง': 46,
 'เสริมสุข': 68,
 'ภาพ': 35,
 'ป้องกัน': 29,
 'รักษา': 39,
 'คน': 6,
 'ไทย': 75,
 'ฟรี': 33,
 'เช็ค': 62,
 'ประโยชน์': 26,
 'ข้อมูล': 5,
 'ประวัติการ': 25,
 'เชื่อม': 61,
 'โยง': 71,
 'สำนักงาน': 50,
 'หลัก': 57,
 'ประกัน': 23,
 'ถ้วน': 16,
 'หน้า': 55,
 'สถาน': 45,
 'พยาบาล': 32,
 'บ้าน': 21,
 'สะดวก': 48,
 'ทำ': 17,
 'นัด': 19,
 'หมอ': 56,
 'ฉีด': 10,
 'วัคซีน': 42,
 'แอปฯ': 70,
 'ฟีเจอร์': 34,
 'ผ่า': 30,
 'นว': 18,
 'อลเล็ต': 58,
 'ระบบ': 38,
 'สาธารณสุข': 49,
 'โลก': 73,
 'ใส่ใจ': 74,
 'เสริม': 67,
 'รู้': 40,
 'ประชาชน': 24,
 'ค้นหา':

In [287]:
c_feat[:,:].todense()

matrix([[0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 3, 1, 0, 0, 0, 0, 2,
         0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0,
         3, 1, 2, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0],
        [1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
         1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0,
         1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 1, 1, 0, 0, 0, 1, 2,
         0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0,
         0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0

In [288]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 1 1 0 0 0 1 1 0 0 1 1 2 3 1 0 0 0 0 2 0 2 0 0 0 0 2 1 0 0 1 0 0 0 0
  1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 2 2 1 0 0 0 0 1 1 0 0 3 1 2 1 0 0 2 0 0
  1 0 0 0]
 [1 1 0 1 1 1 2 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 1
  0 0 0 2 0 0 1 0 0 1 1 0 1 0 1 0 1 2 0 1 1 1 0 0 0 1 2 0 0 0 0 0 1 0 0 1
  2 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0
  0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0
  1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0
  0 0 1 2 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  1 1 1 0]]


In [289]:
first_news.shape[1]

76

In [290]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 15
This is lowerpart : 61.91930232165088


0.2422507915557546

In [291]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 17
This is lowerpart : 35.749125863438955


0.4755361030347905

In [292]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 4
This is lowerpart : 38.61346915261564


0.10359079584873414

# ข่าวที่ 25

In [293]:
clean_text = ['ประเทศออสเตรเลีย จะเปิดประเทศรับนักท่องเที่ยว ในเดือนพฤศจิกายน 2554 นี้','คาดว่าผู้ใช้เขียนปี พ.ศ. ผิด เป็นปี 2554 โดยจริงแล้วเป็น พฤศจิกายน 2564รัฐบาลออสเตรเลียพร้อมเเปิดประเทศอย่างปลอดภัย เพื่อให้พลเมืองออสเตรเลียในต่างประเทศกลับบ้านเกิดได้ รวมทั้งอนุญาตให้นักท่องเที่ยวเดินทางเข้าออสเตรเลียได้อีกครั้ง โดยแผนการเปิดประเทศออสเตรเลีย เบื้องต้นจะมีการประเมินความพร้อมขึ้นอยู่กับอัตราการฉีดวัคซีนในแต่ละพื้นที่ แต่คาดว่าน่าจะเริ่มได้ในเดือน พ.ย. นี้ มุ่งไปที่ชาวออสเตรเลียที่ได้รับการฉีดวัคซีนแล้วและผู้มีที่พำนักถาวรในออสเตรเลียที่จะได้รับอนุญาตให้เดินทางเข้ามาในประเทศออสเตรเลียได้ รัฐบาลออสเตรเลียจะยังคงมีการกำหนดจำนวนผู้ที่จะได้เดินทางเข้าประเทศ รวมทั้งเมื่อเข้ามาแล้วจะต้องกักตัวเป็นเวลา 14 วัน','ออสเตรเลียเตรียมเปิดพรมแดน พ.ย. 64 หลังล็อกสกัดโควิดนาน 18 เดือน','รัฐบาลออสเตรเลียเตรียมเปิดพรมแดนระหว่างประเทศ ในเดือน พ.ย. อนุญาตให้พลเมืองออสเตรเลียเดินทางไปต่างประเทศได้แล้ว หลังล็อกสกัดโควิดมานานถึง 18 เดือน']

**Step 2: Tokenize**

In [294]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ประเทศ', 'ออสเตรเลีย', 'ประเทศ', 'ท่อง', 'เที่ยว', 'เดือน', 'พฤศจิกายน']
tokenized text:
 ['คาด', 'ปี', 'พ', 'ศ', 'ปี', 'พฤศจิกายน', 'รัฐบาล', 'ออสเตรเลีย', 'เเปิด', 'ประเทศ', 'ปลอดภัย', 'พลเมือง', 'ออสเตรเลีย', 'ประเทศ', 'บ้าน', 'อนุญาต', 'ท่อง', 'เที่ยว', 'เดิน', 'ออสเตรเลีย', 'แผนการ', 'ประเทศ', 'ออสเตรเลีย', 'เบื้อง', 'ต้น', 'ประเมิน', 'อัตรา', 'ฉีด', 'วัคซีน', 'พื้นที่', 'คาด', 'เดือน', 'พ', 'ย', 'ออสเตรเลีย', 'ฉีด', 'วัคซีน', 'พำนัก', 'ถาวร', 'ออสเตรเลีย', 'อนุญาต', 'เดิน', 'ประเทศ', 'ออสเตรเลีย', 'รัฐบาล', 'ออสเตรเลีย', 'จำนวน', 'เดิน', 'ประเทศ', 'กักตัว', 'เวลา']
tokenized text:
 ['ออสเตรเลีย', 'เตรียม', 'พรมแดน', 'พ', 'ย', 'ล็อก', 'สกัด', 'โค', 'วิด', 'เดือน']
tokenized text:
 ['รัฐบาล', 'ออสเตรเลีย', 'เตรียม', 'พรมแดน', 'ประเทศ', 'เดือน', 'พ', 'ย', 'อนุญาต', 'พลเมือง', 'ออสเตรเลีย', 'เดิน', 'ประเทศ', 'ล็อก', 'สกัด', 'โค', 'วิด', 'เดือน']


In [295]:
tokens_list = [split_word(txt) for txt in clean_text]

In [296]:
tokens_list

[['ประเทศ', 'ออสเตรเลีย', 'ประเทศ', 'ท่อง', 'เที่ยว', 'เดือน', 'พฤศจิกายน'],
 ['คาด',
  'ปี',
  'พ',
  'ศ',
  'ปี',
  'พฤศจิกายน',
  'รัฐบาล',
  'ออสเตรเลีย',
  'เเปิด',
  'ประเทศ',
  'ปลอดภัย',
  'พลเมือง',
  'ออสเตรเลีย',
  'ประเทศ',
  'บ้าน',
  'อนุญาต',
  'ท่อง',
  'เที่ยว',
  'เดิน',
  'ออสเตรเลีย',
  'แผนการ',
  'ประเทศ',
  'ออสเตรเลีย',
  'เบื้อง',
  'ต้น',
  'ประเมิน',
  'อัตรา',
  'ฉีด',
  'วัคซีน',
  'พื้นที่',
  'คาด',
  'เดือน',
  'พ',
  'ย',
  'ออสเตรเลีย',
  'ฉีด',
  'วัคซีน',
  'พำนัก',
  'ถาวร',
  'ออสเตรเลีย',
  'อนุญาต',
  'เดิน',
  'ประเทศ',
  'ออสเตรเลีย',
  'รัฐบาล',
  'ออสเตรเลีย',
  'จำนวน',
  'เดิน',
  'ประเทศ',
  'กักตัว',
  'เวลา'],
 ['ออสเตรเลีย',
  'เตรียม',
  'พรมแดน',
  'พ',
  'ย',
  'ล็อก',
  'สกัด',
  'โค',
  'วิด',
  'เดือน'],
 ['รัฐบาล',
  'ออสเตรเลีย',
  'เตรียม',
  'พรมแดน',
  'ประเทศ',
  'เดือน',
  'พ',
  'ย',
  'อนุญาต',
  'พลเมือง',
  'ออสเตรเลีย',
  'เดิน',
  'ประเทศ',
  'ล็อก',
  'สกัด',
  'โค',
  'วิด',
  'เดือน']]

**Step 3 + 4a: Bag of words + count word**

In [297]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [298]:
cvec.vocabulary_

{'ประเทศ': 8,
 'ออสเตรเลีย': 26,
 'ท่อง': 6,
 'เที่ยว': 31,
 'เดือน': 29,
 'พฤศจิกายน': 14,
 'คาด': 1,
 'ปี': 11,
 'พ': 12,
 'ศ': 23,
 'รัฐบาล': 19,
 'เเปิด': 34,
 'ปลอดภัย': 10,
 'พลเมือง': 15,
 'บ้าน': 7,
 'อนุญาต': 25,
 'เดิน': 28,
 'แผนการ': 35,
 'เบื้อง': 32,
 'ต้น': 4,
 'ประเมิน': 9,
 'อัตรา': 27,
 'ฉีด': 3,
 'วัคซีน': 21,
 'พื้นที่': 17,
 'ย': 18,
 'พำนัก': 16,
 'ถาวร': 5,
 'จำนวน': 2,
 'กักตัว': 0,
 'เวลา': 33,
 'เตรียม': 30,
 'พรมแดน': 13,
 'ล็อก': 20,
 'สกัด': 24,
 'โค': 36,
 'วิด': 22}

In [299]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
        [1, 2, 1, 2, 1, 1, 1, 1, 5, 1, 1, 2, 2, 0, 1, 1, 1, 1, 1, 2, 0,
         2, 0, 1, 0, 2, 8, 1, 3, 1, 0, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
         0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
         0, 1, 0, 1, 1, 2, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1]], dtype=int64)

In [300]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 1 0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0
  0]
 [1 2 1 2 1 1 1 1 5 1 1 2 2 0 1 1 1 1 1 2 0 2 0 1 0 2 8 1 3 1 0 1 1 1 1 1
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0
  1]
 [0 0 0 0 0 0 0 0 2 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 1 1 2 0 1 2 1 0 0 0 0 0
  1]]


In [301]:
first_news.shape[1]

37

In [302]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 22
This is lowerpart : 36.373066958946424


0.6048431391510365

In [303]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 2
This is lowerpart : 9.486832980505138


0.21081851067789195

In [304]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 14.696938456699067


0.5443310539518174

# ข่าวที่ 26

In [305]:
clean_text = ['"เด็กที่ไม่ได้ฉีดวัคซีนก็สามารถเข้าเรียนได้ ควบคู่กับการเข้มงวดมาตรการป้องกันโรค ภายใต้การพิจารณาของคณะกรรมการโรคติดต่อจังหวัด"สาธิต ปิตุเตชะ รมช.สธ.','เด็กที่ไม่ได้ฉีดวัคซีนก็สามารถเข้าเรียนได้ ควบคู่กับการเข้มงวดมาตรการป้องกันโรค ภายใต้การพิจารณาของคณะกรรมการโรคติดต่อจังหวัด สำหรับเด็กที่มีโรคประจำตัวไม่ได้เรียนหนังสือ หรือกลุ่มนอกระบบการศึกษา สามารถเข้ารับวัคซีนที่โรงพยาบาลที่รักษาประจำได้ ส่วนเด็กเรียนที่บ้านหรือโฮมสคูล ลงทะเบียนรับวัคซีนกับโรงพยาบาลใกล้บ้านได้เช่นกัน','นายกฯ Kick off ฉีดวัคซีนไฟเซอร์นักเรียนอายุ 12–18 ปี','วันที่ 4 ก.ย. 2564 พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรี เป็นประธานเปิดงาน Kick off สร้างเกราะป้องกันด้วยวัคซีนเด็กปลอดภัย เรียนอุ่นใจ ต้อนรับเปิดเทอม เพื่อสร้างความเชื่อมั่นในการฉีดวัคซีนในเด็กกับพ่อแม่ ผู้ปกครอง พร้อมชูมาตรการ Sandbox ลดความเสี่ยงของโควิด-19 ณ โรงเรียนพิบูลย์อุปถัมภ์ กรุงเทพฯ']

**Step 2: Tokenize**

In [306]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เด็ก', 'ฉีด', 'วัคซีน', 'เรียน', 'ควบคู่', 'เข้ม', 'งวด', 'มาตรการ', 'ป้องกัน', 'โรค', 'พิจารณา', 'คณะ', 'กรรมการ', 'โรค', 'ติดต่อ', 'จังหวัด', 'สาธิต', 'ปิตุ', 'เต', 'ชะ', 'รมช', 'สธ']
tokenized text:
 ['เด็ก', 'ฉีด', 'วัคซีน', 'เรียน', 'ควบคู่', 'เข้ม', 'งวด', 'มาตรการ', 'ป้องกัน', 'โรค', 'พิจารณา', 'คณะ', 'กรรมการ', 'โรค', 'ติดต่อ', 'จังหวัด', 'สำหรับ', 'เด็ก', 'โรค', 'ประจำ', 'ตัว', 'เรียน', 'หนังสือ', 'ระบบ', 'ศึกษา', 'วัคซีน', 'โรง', 'พยาบาล', 'รักษา', 'ประจำ', 'เด็ก', 'เรียน', 'บ้าน', 'โฮม', 'สคูล', 'ทะเบียน', 'วัคซีน', 'โรง', 'พยาบาล', 'บ้าน']
tokenized text:
 ['นา', 'ยกฯ', 'Kick', 'off', 'ฉีด', 'วัคซีน', 'ไฟ', 'เซอร์', 'นักเรียน', 'อายุ', '–', 'ปี']
tokenized text:
 ['ก', 'ย', 'พล', 'อ', 'ประยุทธ์', 'จันทร์', 'โอชา', 'นายก', 'รัฐมนตรี', 'ประธาน', 'งาน', 'Kick', 'off', 'สร้าง', 'เกราะ', 'ป้องกัน', 'วัคซีน', 'เด็ก', 'ปลอดภัย', 'เรียน', 'อุ่น', 'ใจ', 'ต้อนรับ', 'เทอม', 'สร้าง', 'มั่น', 'ฉีด', 'วัคซีน', 'เด็ก', 'พ่อ', 'แม่', 'ปกครอง', 'ชู', 'มาตรการ', 'Sandbox',

In [307]:
tokens_list = [split_word(txt) for txt in clean_text]

In [308]:
tokens_list

[['เด็ก',
  'ฉีด',
  'วัคซีน',
  'เรียน',
  'ควบคู่',
  'เข้ม',
  'งวด',
  'มาตรการ',
  'ป้องกัน',
  'โรค',
  'พิจารณา',
  'คณะ',
  'กรรมการ',
  'โรค',
  'ติดต่อ',
  'จังหวัด',
  'สาธิต',
  'ปิตุ',
  'เต',
  'ชะ',
  'รมช',
  'สธ'],
 ['เด็ก',
  'ฉีด',
  'วัคซีน',
  'เรียน',
  'ควบคู่',
  'เข้ม',
  'งวด',
  'มาตรการ',
  'ป้องกัน',
  'โรค',
  'พิจารณา',
  'คณะ',
  'กรรมการ',
  'โรค',
  'ติดต่อ',
  'จังหวัด',
  'สำหรับ',
  'เด็ก',
  'โรค',
  'ประจำ',
  'ตัว',
  'เรียน',
  'หนังสือ',
  'ระบบ',
  'ศึกษา',
  'วัคซีน',
  'โรง',
  'พยาบาล',
  'รักษา',
  'ประจำ',
  'เด็ก',
  'เรียน',
  'บ้าน',
  'โฮม',
  'สคูล',
  'ทะเบียน',
  'วัคซีน',
  'โรง',
  'พยาบาล',
  'บ้าน'],
 ['นา',
  'ยกฯ',
  'Kick',
  'off',
  'ฉีด',
  'วัคซีน',
  'ไฟ',
  'เซอร์',
  'นักเรียน',
  'อายุ',
  '–',
  'ปี'],
 ['ก',
  'ย',
  'พล',
  'อ',
  'ประยุทธ์',
  'จันทร์',
  'โอชา',
  'นายก',
  'รัฐมนตรี',
  'ประธาน',
  'งาน',
  'Kick',
  'off',
  'สร้าง',
  'เกราะ',
  'ป้องกัน',
  'วัคซีน',
  'เด็ก',
  'ปลอดภัย',
  'เรียน',
  'อุ่น

**Step 3 + 4a: Bag of words + count word**

In [309]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [310]:
cvec.vocabulary_

{'เด็ก': 61,
 'ฉีด': 12,
 'วัคซีน': 45,
 'เรียน': 64,
 'ควบคู่': 7,
 'เข้ม': 59,
 'งวด': 8,
 'มาตรการ': 37,
 'ป้องกัน': 30,
 'โรค': 68,
 'พิจารณา': 33,
 'คณะ': 6,
 'กรรมการ': 4,
 'ติดต่อ': 16,
 'จังหวัด': 10,
 'สาธิต': 51,
 'ปิตุ': 28,
 'เต': 62,
 'ชะ': 13,
 'รมช': 40,
 'สธ': 49,
 'สำหรับ': 52,
 'ประจำ': 24,
 'ตัว': 15,
 'หนังสือ': 53,
 'ระบบ': 41,
 'ศึกษา': 47,
 'โรง': 69,
 'พยาบาล': 31,
 'รักษา': 42,
 'บ้าน': 22,
 'โฮม': 72,
 'สคูล': 48,
 'ทะเบียน': 18,
 'นา': 20,
 'ยกฯ': 39,
 'Kick': 0,
 'off': 2,
 'ไฟ': 74,
 'เซอร์': 60,
 'นักเรียน': 19,
 'อายุ': 55,
 '–': 75,
 'ปี': 29,
 'ก': 3,
 'ย': 38,
 'พล': 32,
 'อ': 54,
 'ประยุทธ์': 26,
 'จันทร์': 11,
 'โอชา': 71,
 'นายก': 21,
 'รัฐมนตรี': 43,
 'ประธาน': 25,
 'งาน': 9,
 'สร้าง': 50,
 'เกราะ': 58,
 'ปลอดภัย': 27,
 'อุ่น': 57,
 'ใจ': 73,
 'ต้อนรับ': 17,
 'เทอม': 63,
 'มั่น': 36,
 'พ่อ': 35,
 'แม่': 66,
 'ปกครอง': 23,
 'ชู': 14,
 'Sandbox': 1,
 'ลด': 44,
 'เสี่ยง': 65,
 'โค': 67,
 'วิด': 46,
 'โรงเรียน': 70,
 'พิบูลย์': 34,
 'อุปถัมภ์': 56,
 'ก

In [311]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
         0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
         0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
         1, 0, 0, 3, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 3, 0,
         0, 3, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0],
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
         1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 1, 1, 2, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0

In [312]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0
  0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 2 0 0 0
  0 0 0 0]
 [0 0 0 0 1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 0 0 0 2 0 2 0 0 0 0 0 1 2 0 1 0 0
  0 1 0 0 0 1 1 0 0 3 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 3 0 0 3 0 0 0 3 2 0 0
  1 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 1]
 [1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1
  1 1 1 0 0 0 0 1 1 2 1 0 0 0 2 0 0 0 1 0 1 1 1 0 0 2 0 1 1 1 1 1 0 0 1 1
  0 1 0 0]]


In [313]:
first_news.shape[1]

76

In [314]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 26
This is lowerpart : 41.56921938165305


0.6254627916220946

In [315]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 2
This is lowerpart : 16.97056274847714


0.11785113019775793

In [316]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 34.29285639896449


0.23328473740792174

# ข่าวที่ 27

In [317]:
clean_text = ['หมอระบบประสาทบอกว่า ผลไม่ต่างจากระบบฝังเข็ม มือซ้ายจับหูขวา มือขวาจับหูซ้าย กระตุ้นการทำงานสมองซีกซ้ายและซีกขวาดีสำหรับคนปกติทุกเพศ ทุกวัย คนขี้หลงขี้ลืม Alzheimer แม้แต่ Autistic ก็จะค่อยๆดีขึ้นมือซ้ายจับติ่งหูขวามือขวาจับติ่งหูซ้ายย่อตัวลงหายใจเข้ายืดตัวขึ้นหายใจออกหมอเรียกวิธีนี้ว่า Superbrain Yogaทำวันละ 3 นาทีง่ายมั้ยล่ะ','ตามที่มีการเผยแพร่ในสื่อต่าง ๆ เกี่ยวกับประเด็น ท่าบริหาร SuperBrain Yoga ช่วยกระตุ้นการทำงานของสมองซีกซ้ายและซีกขวาได้ ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลบิดเบือน','ข่าวบิดเบือน ท่าบริหาร SuperBrain Yoga ช่วยกระตุ้นการทำงานของสมองซีกซ้ายและซีกขวาได้','ตามที่มีการเผยแพร่ในสื่อต่าง ๆ เกี่ยวกับประเด็น ท่าบริหาร SuperBrain Yoga ช่วยกระตุ้นการทำงานของสมองซีกซ้ายและซีกขวาได้ ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลบิดเบือน']

**Step 2: Tokenize**

In [318]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['หมอ', 'ระบบ', 'ประสาท', 'ระบบ', 'ฝัง', 'เข็ม', 'มือ', 'ซ้าย', 'หู', 'ขวา', 'มือขวา', 'หู', 'ซ้าย', 'กระตุ้น', 'ทำงาน', 'สมอง', 'ซีก', 'ซ้าย', 'ซีก', 'ขวา', 'ดี', 'สำหรับ', 'คน', 'ปกติ', 'เพศ', 'วัย', 'คน', 'ขี้', 'หลง', 'ขี้', 'ลืม', 'Alzheimer', 'Autistic', 'ดี', 'มือ', 'ซ้าย', 'ติ่งหู', 'ขวา', 'มือขวา', 'ติ่งหู', 'ซ้าย', 'ย่อ', 'ตัว', 'หายใจ', 'ยืด', 'ตัว', 'หายใจ', 'หมอ', 'วิธี', 'Superbrain', 'Yoga', 'ทำ', 'นาที', 'ล่ะ']
tokenized text:
 ['เผย', 'แพร่', 'สื่อ', 'เกี่ยว', 'ประเด็น', 'ท่า', 'บริหาร', 'SuperBrain', 'Yoga', 'กระตุ้น', 'ทำงาน', 'สมอง', 'ซีก', 'ซ้าย', 'ซีก', 'ขวา', 'ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ดำเนิน', 'ตรวจ', 'สอบ', 'ข้อ', 'เท็จ', 'สถาบัน', 'ประสาท', 'วิทยา', 'ประเด็น', 'ข้อมูล', 'บิดเบือน']
tokenized text:
 ['ข่าว', 'บิดเบือน', 'ท่า', 'บริหาร', 'SuperBrain', 'Yoga', 'กระตุ้น', 'ทำงาน', 'สมอง', 'ซีก', 'ซ้าย', 'ซีก', 'ขวา']
tokenized text:
 ['เผย', 'แพร่', 'สื่อ', 'เกี่ยว', 'ประเด็น', 'ท่า', 'บริหาร', 'SuperBrain', 'Yoga', 'กระตุ้น', 'ทำงาน', 'สมอ

In [319]:
tokens_list = [split_word(txt) for txt in clean_text]

In [320]:
tokens_list

[['หมอ',
  'ระบบ',
  'ประสาท',
  'ระบบ',
  'ฝัง',
  'เข็ม',
  'มือ',
  'ซ้าย',
  'หู',
  'ขวา',
  'มือขวา',
  'หู',
  'ซ้าย',
  'กระตุ้น',
  'ทำงาน',
  'สมอง',
  'ซีก',
  'ซ้าย',
  'ซีก',
  'ขวา',
  'ดี',
  'สำหรับ',
  'คน',
  'ปกติ',
  'เพศ',
  'วัย',
  'คน',
  'ขี้',
  'หลง',
  'ขี้',
  'ลืม',
  'Alzheimer',
  'Autistic',
  'ดี',
  'มือ',
  'ซ้าย',
  'ติ่งหู',
  'ขวา',
  'มือขวา',
  'ติ่งหู',
  'ซ้าย',
  'ย่อ',
  'ตัว',
  'หายใจ',
  'ยืด',
  'ตัว',
  'หายใจ',
  'หมอ',
  'วิธี',
  'Superbrain',
  'Yoga',
  'ทำ',
  'นาที',
  'ล่ะ'],
 ['เผย',
  'แพร่',
  'สื่อ',
  'เกี่ยว',
  'ประเด็น',
  'ท่า',
  'บริหาร',
  'SuperBrain',
  'Yoga',
  'กระตุ้น',
  'ทำงาน',
  'สมอง',
  'ซีก',
  'ซ้าย',
  'ซีก',
  'ขวา',
  'ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ดำเนิน',
  'ตรวจ',
  'สอบ',
  'ข้อ',
  'เท็จ',
  'สถาบัน',
  'ประสาท',
  'วิทยา',
  'ประเด็น',
  'ข้อมูล',
  'บิดเบือน'],
 ['ข่าว',
  'บิดเบือน',
  'ท่า',
  'บริหาร',
  'SuperBrain',
  'Yoga',
  'กระตุ้น',
  'ทำงาน',
  'สมอง',
  'ซีก',
  'ซ้าย',

**Step 3 + 4a: Bag of words + count word**

In [321]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [322]:
cvec.vocabulary_

{'หมอ': 47,
 'ระบบ': 35,
 'ประสาท': 27,
 'ฝัง': 30,
 'เข็ม': 52,
 'มือ': 31,
 'ซ้าย': 13,
 'หู': 50,
 'ขวา': 6,
 'มือขวา': 32,
 'กระตุ้น': 5,
 'ทำงาน': 21,
 'สมอง': 43,
 'ซีก': 12,
 'ดี': 15,
 'สำหรับ': 45,
 'คน': 11,
 'ปกติ': 26,
 'เพศ': 55,
 'วัย': 38,
 'ขี้': 7,
 'หลง': 48,
 'ลืม': 36,
 'Alzheimer': 0,
 'Autistic': 1,
 'ติ่งหู': 18,
 'ย่อ': 34,
 'ตัว': 17,
 'หายใจ': 49,
 'ยืด': 33,
 'วิธี': 40,
 'Superbrain': 3,
 'Yoga': 4,
 'ทำ': 20,
 'นาที': 23,
 'ล่ะ': 37,
 'เผย': 54,
 'แพร่': 56,
 'สื่อ': 46,
 'เกี่ยว': 51,
 'ประเด็น': 28,
 'ท่า': 22,
 'บริหาร': 24,
 'SuperBrain': 2,
 'ศูนย์': 41,
 'ต้าน': 19,
 'ข่าว': 8,
 'ปลอม': 29,
 'ดำเนิน': 14,
 'ตรวจ': 16,
 'สอบ': 44,
 'ข้อ': 9,
 'เท็จ': 53,
 'สถาบัน': 42,
 'วิทยา': 39,
 'ข้อมูล': 10,
 'บิดเบือน': 25}

In [323]:
c_feat[:,:].todense()

matrix([[1, 1, 0, 1, 1, 1, 3, 2, 0, 0, 0, 2, 2, 5, 0, 2, 0, 2, 2, 0, 1,
         1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 1, 0,
         0, 1, 0, 1, 0, 2, 1, 2, 2, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0,
         1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0,
         1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]], dtype=int64)

In [324]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 1 0 1 1 1 3 2 0 0 0 2 2 5 0 2 0 2 2 0 1 1 0 1 0 0 1 1 0 0 1 2 2 1 1 2
  1 1 1 0 1 0 0 1 0 1 0 2 1 2 2 0 1 0 0 1 0]
 [0 0 1 0 1 1 1 0 1 1 1 0 2 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 2 1 0 0 0 0 0 0
  0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1]
 [0 0 1 0 1 1 1 0 1 0 0 0 2 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 1 1 0 1 1 1 0 2 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 2 1 0 0 0 0 0 0
  0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1]]


In [325]:
first_news.shape[1]

57

In [326]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 17
This is lowerpart : 60.33241251599342


0.28177225625601326

In [327]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 16
This is lowerpart : 39.496835316262995


0.40509574683346666

In [328]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 17
This is lowerpart : 60.33241251599342


0.28177225625601326

# ข่าวที่ 28

In [329]:
clean_text = ['การเดินทางไปต่างประเทศ ต้องแสดงวัคซีน พาสปอร์ต ที่ออกโดยรัฐ เท่านั้น','ผู้ที่ฉีดวัคซีนโควิด-19 แล้ว หายป่วยจากโรคโควิด หรือมีผลตรวจโควิดเป็นลบ เพื่อที่จะสามารถเดินทางท่องเที่ยวในยุโรปได้ส่วน สหราชอาณาจักรเอง ก็กำลังจะออกวัคซีนพาสปอร์ตเช่นกัน เพื่อใช้ในการเดินทางท่องเที่ยวในอังกฤษ ยุโรป และ ประเทศที่อังกฤษอนุญาตให้เดินทางไปกลับได้โดยไม่ต้องกักตัว','"วัคซีนพาสปอร์ต" บัตรผ่านเดินทางในต่างประเทศ เงื่อนไข ขั้นตอนการขอไม่ยุ่งยาก','วัคซีนพาสปอร์ต ใบเบิกทาง เดินทางท่องเที่ยวในต่างประเทศขั้นตอนการเตรียมเอกสาร ยื่นขอวัคซีนพาสปอร์ต ไม่ยุ่งยากอย่างที่คิดหากใครที่ติดตามข่าว พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรี เดินทางเข้ารับการฉีดวัคซีนแอสตราเซเนกา ป้องกันโควิด-19 เข็มที่ 2 เมื่อช่วงเช้าวันนี้ (24 พ.ค.) หลังฉีดวัคซีนเข็มแรกไปเมื่อวันที่ 16 มี.ค.64 จากนั้น อธิบดีกรมควบคุมโรค ได้มอบ "วัคซีนพาสปอร์ต" ให้กับนายกฯ ใช้ในการเดินทางไปต่างประเทศ โดยแนบกับหนังสือเดินทาง เพื่อเป็นการยืนยันว่า ปราศจากเชื้อโควิด และได้รับการฉีดวัคซีนครบแล้ว 2 โดส ซึ่งบางคนคงมีคำถามว่า "วัคซีนพาสปอร์ต" คืออะไร และสามารถใช้เดินทางเข้าประเทศไหนได้บ้าง']

**Step 2: Tokenize**

In [330]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เดิน', 'ประเทศ', 'วัคซีน', 'พาส', 'ปอร์ต', 'รัฐ']
tokenized text:
 ['ฉีด', 'วัคซีน', 'โค', 'วิด', 'หาย', 'ป่วย', 'โรค', 'โค', 'วิด', 'ตรวจ', 'โค', 'วิด', 'ลบ', 'เดิน', 'ท่อง', 'เที่ยว', 'ยุโรป', 'สห', 'ราช', 'อาณาจักร', 'วัคซีน', 'พาส', 'ปอร์ต', 'เดิน', 'ท่อง', 'เที่ยว', 'อังกฤษ', 'ยุโรป', 'ประเทศ', 'อังกฤษ', 'อนุญาต', 'เดิน', 'กักตัว']
tokenized text:
 ['วัคซีน', 'พาส', 'ปอร์ต', 'บัตร', 'เดิน', 'ทางใน', 'ประเทศ', 'เงื่อนไข', 'ตอน', 'ยุ่ง']
tokenized text:
 ['วัคซีน', 'พาส', 'ปอร์ต', 'ใบเบิก', 'เดิน', 'ท่อง', 'เที่ยว', 'ประเทศ', 'ตอน', 'การเต', 'รี', 'ยม', 'เอกสาร', 'ยื่น', 'วัคซีน', 'พาส', 'ปอร์ต', 'ยุ่ง', 'ติดตาม', 'ข่าว', 'พล', 'อ', 'ประยุทธ์', 'จันทร์', 'โอชา', 'นายก', 'รัฐมนตรี', 'เดิน', 'ฉีด', 'วัค', 'ซีน', 'แอ', 'ส', 'ตรา', 'เซ', 'เน', 'กา', 'ป้องกัน', 'โค', 'วิด', 'เข็ม', 'เช้า', 'พ', 'ค', 'ฉีด', 'วัคซีน', 'เข็ม', 'ค', 'อธิบดี', 'กรม', 'ควบคุม', 'โรค', 'มอบ', 'วัคซีน', 'พาส', 'ปอร์ต', 'นา', 'ยกฯ', 'เดิน', 'ประเทศ', 'แนบ', 'หนังสือเดินทาง', 'ปราศจาก', 'เชื้อ',

In [331]:
tokens_list = [split_word(txt) for txt in clean_text]

In [332]:
tokens_list

[['เดิน', 'ประเทศ', 'วัคซีน', 'พาส', 'ปอร์ต', 'รัฐ'],
 ['ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'หาย',
  'ป่วย',
  'โรค',
  'โค',
  'วิด',
  'ตรวจ',
  'โค',
  'วิด',
  'ลบ',
  'เดิน',
  'ท่อง',
  'เที่ยว',
  'ยุโรป',
  'สห',
  'ราช',
  'อาณาจักร',
  'วัคซีน',
  'พาส',
  'ปอร์ต',
  'เดิน',
  'ท่อง',
  'เที่ยว',
  'อังกฤษ',
  'ยุโรป',
  'ประเทศ',
  'อังกฤษ',
  'อนุญาต',
  'เดิน',
  'กักตัว'],
 ['วัคซีน',
  'พาส',
  'ปอร์ต',
  'บัตร',
  'เดิน',
  'ทางใน',
  'ประเทศ',
  'เงื่อนไข',
  'ตอน',
  'ยุ่ง'],
 ['วัคซีน',
  'พาส',
  'ปอร์ต',
  'ใบเบิก',
  'เดิน',
  'ท่อง',
  'เที่ยว',
  'ประเทศ',
  'ตอน',
  'การเต',
  'รี',
  'ยม',
  'เอกสาร',
  'ยื่น',
  'วัคซีน',
  'พาส',
  'ปอร์ต',
  'ยุ่ง',
  'ติดตาม',
  'ข่าว',
  'พล',
  'อ',
  'ประยุทธ์',
  'จันทร์',
  'โอชา',
  'นายก',
  'รัฐมนตรี',
  'เดิน',
  'ฉีด',
  'วัค',
  'ซีน',
  'แอ',
  'ส',
  'ตรา',
  'เซ',
  'เน',
  'กา',
  'ป้องกัน',
  'โค',
  'วิด',
  'เข็ม',
  'เช้า',
  'พ',
  'ค',
  'ฉีด',
  'วัคซีน',
  'เข็ม',
  'ค',
  'อธิบดี',
  'กรม',
  'ควบ

**Step 3 + 4a: Bag of words + count word**

In [333]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [334]:
cvec.vocabulary_

{'เดิน': 58,
 'ประเทศ': 22,
 'วัคซีน': 42,
 'พาส': 29,
 'ปอร์ต': 24,
 'รัฐ': 36,
 'ฉีด': 10,
 'โค': 64,
 'วิด': 43,
 'หาย': 47,
 'ป่วย': 25,
 'โรค': 66,
 'ตรวจ': 12,
 'ลบ': 40,
 'ท่อง': 17,
 'เที่ยว': 59,
 'ยุโรป': 34,
 'สห': 45,
 'ราช': 38,
 'อาณาจักร': 52,
 'อังกฤษ': 51,
 'อนุญาต': 50,
 'กักตัว': 1,
 'บัตร': 20,
 'ทางใน': 16,
 'เงื่อนไข': 54,
 'ตอน': 14,
 'ยุ่ง': 35,
 'ใบเบิก': 68,
 'การเต': 3,
 'รี': 39,
 'ยม': 32,
 'เอกสาร': 61,
 'ยื่น': 33,
 'ติดตาม': 15,
 'ข่าว': 4,
 'พล': 28,
 'อ': 48,
 'ประยุทธ์': 21,
 'จันทร์': 9,
 'โอชา': 67,
 'นายก': 19,
 'รัฐมนตรี': 37,
 'วัค': 41,
 'ซีน': 11,
 'แอ': 63,
 'ส': 44,
 'ตรา': 13,
 'เซ': 57,
 'เน': 60,
 'กา': 2,
 'ป้องกัน': 26,
 'เข็ม': 53,
 'เช้า': 56,
 'พ': 27,
 'ค': 5,
 'อธิบดี': 49,
 'กรม': 0,
 'ควบคุม': 7,
 'มอบ': 30,
 'นา': 18,
 'ยกฯ': 31,
 'แนบ': 62,
 'หนังสือเดินทาง': 46,
 'ปราศจาก': 23,
 'เชื้อ': 55,
 'โดส': 65,
 'คน': 6,
 'คำถาม': 8}

In [335]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0,
         0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0,
         2, 3, 0, 1, 0, 1, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0,
         0, 3, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
         0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
         1, 3, 1, 4, 0, 1, 1, 1, 4, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
         6, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 1, 1, 1, 4, 1, 1, 1, 1,
         1, 2, 1, 1, 1, 1]], dtype=int64)

In [336]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0
  1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 2 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 2 0
  0 0 1 0 1 0 2 3 0 1 0 1 0 0 1 2 1 0 0 0 0 0 3 2 0 0 0 0 3 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 2 1 1 1 1 3 1 0 1 1 1 0 1 1 1 0 1 3 1 4 0 1 1 1 4 1 1 1 1 0 1
  0 1 0 1 0 1 6 2 1 0 1 0 1 1 0 0 0 2 0 1 1 1 4 1 1 1 1 1 2 1 1 1 1]]


In [337]:
first_news.shape[1]

69

In [338]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 8
This is lowerpart : 19.13112646970899


0.4181666987914534

In [339]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 5
This is lowerpart : 7.745966692414833


0.6454972243679029

In [340]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 21
This is lowerpart : 31.17691453623979


0.6735753140545635

# ข่าวที่ 29

In [341]:
clean_text = ["'ติดแอร์' หัวเตียงนอน ทำให้เกิด 'เชื้อราในสมอง'",'เว็บไซต์ศูนย์ต่อต้านข่าวปลอม ประเทศไทย รายงานว่า ตามที่มีการเผยแพร่ข้อมูลประเด็นเรื่องติดแอร์บนหัวเตียงนอน ก่อให้เกิดเชื้อราในสมอง ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา กรมการแพทย์ กระทรวงสาธารณสุข พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลเท็จ',"ข่าวปลอม อย่าแชร์! 'ติดแอร์' หัวเตียงนอน ทำให้เกิด 'เชื้อราในสมอง'",'เช็คแล้ว ข่าวปลอม อย่าแชร์! "ติดแอร์" บนหัวเตียงนอน ก่อให้เกิด "เชื้อราในสมอง" สถาบันประสาทวิทยายัน "ข้อมูลเท็จ"']

**Step 2: Tokenize**

In [342]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ติด', 'แอร์', 'หัว', 'เตียง', 'นอน', 'เชื้อ', 'รา', 'สมอง']
tokenized text:
 ['เว็บไซต์', 'ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ประเทศไทย', 'รายงาน', 'เผย', 'แพร่', 'ข้อมูล', 'ประเด็น', 'เรื่อง', 'ติด', 'แอร์', 'หัว', 'เตียง', 'นอน', 'ก่อ', 'เชื้อ', 'รา', 'สมอง', 'ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ดำเนิน', 'ตรวจ', 'สอบ', 'ข้อ', 'เท็จ', 'สถาบัน', 'ประสาท', 'วิทยา', 'กรม', 'แพทย์', 'กระทรวง', 'สาธารณสุข', 'ประเด็น', 'ข้อมูล', 'เท็จ']
tokenized text:
 ['ข่าว', 'ปลอม', 'อย่า', 'แชร์', 'ติด', 'แอร์', 'หัว', 'เตียง', 'นอน', 'เชื้อ', 'รา', 'สมอง']
tokenized text:
 ['เช็ค', 'ข่าว', 'ปลอม', 'อย่า', 'แชร์', 'ติด', 'แอร์', 'หัว', 'เตียง', 'นอน', 'ก่อ', 'เชื้อ', 'รา', 'สมอง', 'สถาบัน', 'ประสาท', 'วิทยา', 'ยัน', 'ข้อมูล', 'เท็จ']


In [343]:
tokens_list = [split_word(txt) for txt in clean_text]

In [344]:
tokens_list

[['ติด', 'แอร์', 'หัว', 'เตียง', 'นอน', 'เชื้อ', 'รา', 'สมอง'],
 ['เว็บไซต์',
  'ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ประเทศไทย',
  'รายงาน',
  'เผย',
  'แพร่',
  'ข้อมูล',
  'ประเด็น',
  'เรื่อง',
  'ติด',
  'แอร์',
  'หัว',
  'เตียง',
  'นอน',
  'ก่อ',
  'เชื้อ',
  'รา',
  'สมอง',
  'ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ดำเนิน',
  'ตรวจ',
  'สอบ',
  'ข้อ',
  'เท็จ',
  'สถาบัน',
  'ประสาท',
  'วิทยา',
  'กรม',
  'แพทย์',
  'กระทรวง',
  'สาธารณสุข',
  'ประเด็น',
  'ข้อมูล',
  'เท็จ'],
 ['ข่าว',
  'ปลอม',
  'อย่า',
  'แชร์',
  'ติด',
  'แอร์',
  'หัว',
  'เตียง',
  'นอน',
  'เชื้อ',
  'รา',
  'สมอง'],
 ['เช็ค',
  'ข่าว',
  'ปลอม',
  'อย่า',
  'แชร์',
  'ติด',
  'แอร์',
  'หัว',
  'เตียง',
  'นอน',
  'ก่อ',
  'เชื้อ',
  'รา',
  'สมอง',
  'สถาบัน',
  'ประสาท',
  'วิทยา',
  'ยัน',
  'ข้อมูล',
  'เท็จ']]

**Step 3 + 4a: Bag of words + count word**

In [345]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [346]:
cvec.vocabulary_

{'ติด': 8,
 'แอร์': 36,
 'หัว': 24,
 'เตียง': 28,
 'นอน': 10,
 'เชื้อ': 26,
 'รา': 16,
 'สมอง': 21,
 'เว็บไซต์': 32,
 'ศูนย์': 19,
 'ต้าน': 9,
 'ข่าว': 3,
 'ปลอม': 14,
 'ประเทศไทย': 13,
 'รายงาน': 17,
 'เผย': 30,
 'แพร่': 35,
 'ข้อมูล': 5,
 'ประเด็น': 12,
 'เรื่อง': 31,
 'ก่อ': 2,
 'ดำเนิน': 6,
 'ตรวจ': 7,
 'สอบ': 22,
 'ข้อ': 4,
 'เท็จ': 29,
 'สถาบัน': 20,
 'ประสาท': 11,
 'วิทยา': 18,
 'กรม': 0,
 'แพทย์': 34,
 'กระทรวง': 1,
 'สาธารณสุข': 23,
 'อย่า': 25,
 'แชร์': 33,
 'เช็ค': 27,
 'ยัน': 15}

In [347]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 0, 1, 1, 1, 2, 1,
         1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
         1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
         1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [348]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0
  1]
 [1 1 1 2 1 2 1 1 1 2 1 1 2 1 2 0 1 1 1 2 1 1 1 1 1 0 1 0 1 2 1 1 1 0 1 1
  1]
 [0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0
  1]
 [0 0 1 1 0 1 0 0 1 0 1 1 0 0 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0
  1]]


In [349]:
first_news.shape[1]

37

In [350]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 8
This is lowerpart : 20.784609690826528


0.3849001794597505

In [351]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 9.797958971132713


0.8164965809277259

In [352]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 12.649110640673518


0.6324555320336759

# ข่าวที่ 30

In [353]:
clean_text = ['เด็กนักเรียนอายุ 12 ปีขึ้นไป ฉีดวัคซีนป้องกันโรคโควิด-19 หากเกิดอาการไม่พึงประสงค์หลังฉีดวัคซีน สปสช.จัดระบบช่วยเหลือเบื้องต้นดูแล ขอยื่นเรื่องรับเงินช่วยเหลือได้','หากเกิดอาการไม่พึงประสงค์หลังฉีดวัคซีนโควิด-19 ตามแผนงานโครงการที่รัฐจัดให้ฟรี สามารถยื่นเรื่องขอรับเงินช่วยเหลือเบื้องต้นกับสำนักงานหลักประกันสุขภาพแห่งชาติ (สปสช.) ได้ที่ โรงพยาบาลที่ฉีด สำนักงานสาธารณสุขจังหวัด (สสจ.) หรือที่สำนักงาน สปสช.สาขาเขตพื้นที่ทั้ง 13 เขต โดยมีระยะเวลายื่นคำร้องได้ภายใน 2 ปีนับแต่วันที่ทราบความเสียหาย เมื่อมีผู้มายื่นคำร้องแล้ว คณะอนุกรรมการฯ ระดับเขตพื้นที่จะเร่งพิจารณาให้แล้วเสร็จโดยเร็ว ซึ่งในกรณีที่ผู้ยื่นคำร้องไม่เห็นด้วยกับผลการวินิจฉัย ก็มีสิทธิยื่นอุทธรณ์ต่อเลขาธิการ สปสช. ได้ภายใน 30 วันนับแต่วันที่ทราบผลการวินิจฉัย','"นักเรียน" เช็คอาการไม่พึงประสงค์ "วัคซีนโควิด" และหลักเกณฑ์รับเยียวยา','"สปสช." ขานรับนโยบายรัฐบาลฉีด "วัคซีนโควิด-19" ให้กับเด็กนักเรียนอายุ 12 ปีขึ้นไป หากเกิด "อาการไม่พึงประสงค์" หลังฉีดวัคซีน สปสช. จัดระบบช่วยเหลือเบื้องต้นดูแล ขอยื่นเรื่องรับเงินช่วยเหลือได้ ย้ำรายงานผู้เกิดผลข้างเคียงมีไม่มาก']

**Step 2: Tokenize**

In [354]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เด็ก', 'นักเรียน', 'อายุ', 'ปี', 'ฉีด', 'วัคซีน', 'ป้องกัน', 'โรค', 'โค', 'วิด', 'อาการ', 'ประสงค์', 'ฉีด', 'วัคซีน', 'สปสช', 'ระบบ', 'เบื้อง', 'ต้น', 'ดูแล', 'ยื่น', 'เรื่อง', 'เงิน']
tokenized text:
 ['อาการ', 'ประสงค์', 'ฉีด', 'วัคซีน', 'โค', 'วิด', 'แผน', 'งาน', 'โครงการ', 'รัฐ', 'ฟรี', 'ยื่น', 'เรื่อง', 'ขอรับ', 'เงิน', 'เบื้อง', 'ต้น', 'สำนักงาน', 'หลัก', 'ประกัน', 'สุขภาพ', 'ชาติ', 'สปสช', 'โรง', 'พยาบาล', 'ฉีด', 'สำนักงาน', 'สาธารณสุข', 'จังหวัด', 'สสจ', 'สำนักงาน', 'สปสช', 'สาขา', 'เขต', 'พื้นที่', 'เขต', 'เวลา', 'ยื่น', 'คำร้อง', 'ปี', 'ทราบ', 'หาย', 'ยื่น', 'คำร้อง', 'คณะ', 'อนุกร', 'รม', 'การฯ', 'ระดับ', 'เขต', 'พื้นที่', 'เร่ง', 'พิจารณา', 'กรณี', 'ยื่น', 'คำร้อง', 'วินิจฉัย', 'สิทธิ', 'ยื่น', 'อุทธรณ์', 'เลขาธิการ', 'สปสช', 'ทราบ', 'วินิจฉัย']
tokenized text:
 ['นักเรียน', 'เช็ค', 'อาการ', 'ประสงค์', 'วัคซีน', 'โค', 'วิด', 'หลัก', 'เกณฑ์', 'เยียวยา']
tokenized text:
 ['สปสช', 'ขานรับ', 'นโยบาย', 'รัฐบาล', 'ฉีด', 'วัคซีน', 'โค', 'วิด', 'เด็ก', 'นักเรียน'

In [355]:
tokens_list = [split_word(txt) for txt in clean_text]

In [356]:
tokens_list

[['เด็ก',
  'นักเรียน',
  'อายุ',
  'ปี',
  'ฉีด',
  'วัคซีน',
  'ป้องกัน',
  'โรค',
  'โค',
  'วิด',
  'อาการ',
  'ประสงค์',
  'ฉีด',
  'วัคซีน',
  'สปสช',
  'ระบบ',
  'เบื้อง',
  'ต้น',
  'ดูแล',
  'ยื่น',
  'เรื่อง',
  'เงิน'],
 ['อาการ',
  'ประสงค์',
  'ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'แผน',
  'งาน',
  'โครงการ',
  'รัฐ',
  'ฟรี',
  'ยื่น',
  'เรื่อง',
  'ขอรับ',
  'เงิน',
  'เบื้อง',
  'ต้น',
  'สำนักงาน',
  'หลัก',
  'ประกัน',
  'สุขภาพ',
  'ชาติ',
  'สปสช',
  'โรง',
  'พยาบาล',
  'ฉีด',
  'สำนักงาน',
  'สาธารณสุข',
  'จังหวัด',
  'สสจ',
  'สำนักงาน',
  'สปสช',
  'สาขา',
  'เขต',
  'พื้นที่',
  'เขต',
  'เวลา',
  'ยื่น',
  'คำร้อง',
  'ปี',
  'ทราบ',
  'หาย',
  'ยื่น',
  'คำร้อง',
  'คณะ',
  'อนุกร',
  'รม',
  'การฯ',
  'ระดับ',
  'เขต',
  'พื้นที่',
  'เร่ง',
  'พิจารณา',
  'กรณี',
  'ยื่น',
  'คำร้อง',
  'วินิจฉัย',
  'สิทธิ',
  'ยื่น',
  'อุทธรณ์',
  'เลขาธิการ',
  'สปสช',
  'ทราบ',
  'วินิจฉัย'],
 ['นักเรียน',
  'เช็ค',
  'อาการ',
  'ประสงค์',
  'วัคซีน',
  'โค',
  'วิด'

**Step 3 + 4a: Bag of words + count word**

In [357]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [358]:
cvec.vocabulary_

{'เด็ก': 52,
 'นักเรียน': 13,
 'อายุ': 45,
 'ปี': 17,
 'ฉีด': 8,
 'วัคซีน': 31,
 'ป้องกัน': 18,
 'โรค': 62,
 'โค': 60,
 'วิด': 32,
 'อาการ': 44,
 'ประสงค์': 16,
 'สปสช': 34,
 'ระบบ': 27,
 'เบื้อง': 53,
 'ต้น': 11,
 'ดูแล': 10,
 'ยื่น': 23,
 'เรื่อง': 55,
 'เงิน': 50,
 'แผน': 59,
 'งาน': 6,
 'โครงการ': 61,
 'รัฐ': 28,
 'ฟรี': 22,
 'ขอรับ': 2,
 'สำนักงาน': 38,
 'หลัก': 41,
 'ประกัน': 15,
 'สุขภาพ': 40,
 'ชาติ': 9,
 'โรง': 63,
 'พยาบาล': 19,
 'สาธารณสุข': 37,
 'จังหวัด': 7,
 'สสจ': 35,
 'สาขา': 36,
 'เขต': 48,
 'พื้นที่': 21,
 'เวลา': 58,
 'คำร้อง': 5,
 'ทราบ': 12,
 'หาย': 42,
 'คณะ': 4,
 'อนุกร': 43,
 'รม': 25,
 'การฯ': 1,
 'ระดับ': 26,
 'เร่ง': 56,
 'พิจารณา': 20,
 'กรณี': 0,
 'วินิจฉัย': 33,
 'สิทธิ': 39,
 'อุทธรณ์': 46,
 'เลขาธิการ': 57,
 'เช็ค': 51,
 'เกณฑ์': 47,
 'เยียวยา': 54,
 'ขานรับ': 3,
 'นโยบาย': 14,
 'รัฐบาล': 29,
 'ย้ำ': 24,
 'รายงาน': 30,
 'เคียง': 49}

In [359]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
         0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
         0],
        [1, 1, 1, 0, 1, 3, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 1,
         2, 1, 5, 0, 1, 1, 0, 1, 0, 0, 1, 1, 2, 3, 1, 1, 1, 3, 1, 1, 1,
         1, 1, 1, 0, 1, 0, 3, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
         1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
         0],
        [0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
         0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0]], dtype=int64)

In [360]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 2 0 1 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 2 1 0 1 0
  0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 1 0]
 [1 1 1 0 1 3 1 1 2 1 0 1 2 0 0 1 1 1 0 1 1 2 1 5 0 1 1 0 1 0 0 1 1 2 3 1
  1 1 3 1 1 1 1 1 1 0 1 0 3 0 1 0 0 1 0 1 1 1 1 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
  0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 2 0 1 1 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 0 1 1 2 1 0 2 0
  0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0]]


In [361]:
first_news.shape[1]

64

In [362]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 23
This is lowerpart : 54.91812087098392


0.41880529841930714

In [363]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 16.1245154965971


0.43412157106222965

In [364]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 25
This is lowerpart : 29.291637031753616


0.8534859274986487

# ข่าวที่ 31

In [365]:
clean_text = ['มีการส่งต่อในกลุ่มไลน์ให้พึงระวัง ผู้ไม่ประสงค์ดีนำเอาตะปูไปใส่ไว้ในลูกส้มแป้น','อ.เจษฎา นักวิชาการอิสระ แจงชัด แชร์เตือนภัย ‘ส้มตะปู’ แท้จริงเกิดในอินโดฯวันนี้ (12 ม.ค.) ล่าสุด อ.เจษฎา เด่นดวงบริพันธ์ ผู้ช่วยศาสตราจารย์ภาควิชาชีววิทยา คณะวิทยาศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย ได้ออกมาแสดงความเห็นต่อเรื่องดังกล่าวว่า“เรื่องส้มสอดไส้ตะปู เป็นเรื่องในอินโดนีเซีย”','เฉลยแล้ว! แชร์เตือนภัย ‘ส้มตะปู’ แท้จริงเกิดในอินโดฯ','อ.เจษฎา นักวิชาการอิสระ แจงชัด แชร์เตือนภัย ‘ส้มตะปู’ แท้จริงเกิดในอินโดฯ วันนี้ (12 ม.ค.) ล่าสุด อ.เจษฎา เด่นดวงบริพันธ์ ผู้ช่วยศาสตราจารย์ภาควิชาชีววิทยา คณะวิทยาศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย ได้ออกมาแสดงความเห็นต่อเรื่องดังกล่าวว่า“เรื่องส้มสอดไส้ตะปู เป็นเรื่องในอินโดนีเซีย”กะแล้วว่าเป็นแค่ข่าวลือ ไม่ใช่เรื่องจริงในไทย เห็นมีแต่รูปกับลือกันว่าโดนกันแถวประจวบฯ ไม่เห็นมีข่าวจริง ๆ เป็นทางการว่าใครโดนไปบ้าง แต่ยังไงก็ระวัง ๆ กันไว้ก็ดีครับ แชร์กันกระหน่ำขนาดนี้ มิจฉาชีพได้ไอเดียทำมาหากินเพิ่มขึ้น ปล. แล้วเวลาจะขับรถหลบส้มหรือของอะไรที่หล่นพื้นอยู่ ดูรถข้าง ๆ ด้วยนะครับ เดี๋ยวพลาดชนกันอีก']

**Step 2: Tokenize**

In [366]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ไลน์', 'ระวัง', 'ประสงค์', 'ดี', 'ตะปู', 'ใส่', 'ลูก', 'ส้ม', 'แป้น']
tokenized text:
 ['อ', 'เจษฎา', 'วิชาการ', 'อิสระ', 'แจง', 'ชัด', 'แชร์', 'เตือน', 'ภัย', 'ส้ม', 'ตะปู', 'อิน', 'โดฯ', 'ม', 'ค', 'อ', 'เจษฎา', 'เด่น', 'ดวง', 'บริพันธ์', 'ศาสตราจารย์', 'วิชา', 'ชีววิทยา', 'คณะ', 'วิทยาศาสตร์', 'จุฬาลงกรณ์', 'มหาวิทยาลัย', 'เรื่อง', 'เรื่อง', 'ส้ม', 'สอดไส้', 'ตะปู', 'เรื่อง', 'อินโดนีเซีย']
tokenized text:
 ['เฉลย', 'แชร์', 'เตือน', 'ภัย', 'ส้ม', 'ตะปู', 'อิน', 'โดฯ']
tokenized text:
 ['อ', 'เจษฎา', 'วิชาการ', 'อิสระ', 'แจง', 'ชัด', 'แชร์', 'เตือน', 'ภัย', 'ส้ม', 'ตะปู', 'อิน', 'โดฯ', 'ม', 'ค', 'อ', 'เจษฎา', 'เด่น', 'ดวง', 'บริพันธ์', 'ศาสตราจารย์', 'วิชา', 'ชีววิทยา', 'คณะ', 'วิทยาศาสตร์', 'จุฬาลงกรณ์', 'มหาวิทยาลัย', 'เรื่อง', 'เรื่อง', 'ส้ม', 'สอดไส้', 'ตะปู', 'เรื่อง', 'อินโดนีเซีย', 'กะ', 'ข่าว', 'ลือ', 'เรื่อง', 'ไทย', 'รูป', 'ลือ', 'โดน', 'แถว', 'ประ', 'จวบฯ', 'ข่าว', 'ทางการ', 'โดน', 'ระวัง', 'ดี', 'แชร์', 'กระหน่ำ', 'ขนาด', 'มิจฉาชีพ', 'ไอ', 'เดีย', 'ทำ', 

In [367]:
tokens_list = [split_word(txt) for txt in clean_text]

In [368]:
tokens_list

[['ไลน์', 'ระวัง', 'ประสงค์', 'ดี', 'ตะปู', 'ใส่', 'ลูก', 'ส้ม', 'แป้น'],
 ['อ',
  'เจษฎา',
  'วิชาการ',
  'อิสระ',
  'แจง',
  'ชัด',
  'แชร์',
  'เตือน',
  'ภัย',
  'ส้ม',
  'ตะปู',
  'อิน',
  'โดฯ',
  'ม',
  'ค',
  'อ',
  'เจษฎา',
  'เด่น',
  'ดวง',
  'บริพันธ์',
  'ศาสตราจารย์',
  'วิชา',
  'ชีววิทยา',
  'คณะ',
  'วิทยาศาสตร์',
  'จุฬาลงกรณ์',
  'มหาวิทยาลัย',
  'เรื่อง',
  'เรื่อง',
  'ส้ม',
  'สอดไส้',
  'ตะปู',
  'เรื่อง',
  'อินโดนีเซีย'],
 ['เฉลย', 'แชร์', 'เตือน', 'ภัย', 'ส้ม', 'ตะปู', 'อิน', 'โดฯ'],
 ['อ',
  'เจษฎา',
  'วิชาการ',
  'อิสระ',
  'แจง',
  'ชัด',
  'แชร์',
  'เตือน',
  'ภัย',
  'ส้ม',
  'ตะปู',
  'อิน',
  'โดฯ',
  'ม',
  'ค',
  'อ',
  'เจษฎา',
  'เด่น',
  'ดวง',
  'บริพันธ์',
  'ศาสตราจารย์',
  'วิชา',
  'ชีววิทยา',
  'คณะ',
  'วิทยาศาสตร์',
  'จุฬาลงกรณ์',
  'มหาวิทยาลัย',
  'เรื่อง',
  'เรื่อง',
  'ส้ม',
  'สอดไส้',
  'ตะปู',
  'เรื่อง',
  'อินโดนีเซีย',
  'กะ',
  'ข่าว',
  'ลือ',
  'เรื่อง',
  'ไทย',
  'รูป',
  'ลือ',
  'โดน',
  'แถว',
  'ประ',
  'จวบฯ',
  'ข่า

**Step 3 + 4a: Bag of words + count word**

In [369]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [370]:
cvec.vocabulary_

{'ไลน์': 62,
 'ระวัง': 29,
 'ประสงค์': 20,
 'ดี': 13,
 'ตะปู': 15,
 'ใส่': 60,
 'ลูก': 32,
 'ส้ม': 38,
 'แป้น': 57,
 'อ': 42,
 'เจษฎา': 46,
 'วิชาการ': 34,
 'อิสระ': 45,
 'แจง': 54,
 'ชัด': 10,
 'แชร์': 55,
 'เตือน': 51,
 'ภัย': 24,
 'อิน': 43,
 'โดฯ': 59,
 'ม': 25,
 'ค': 5,
 'เด่น': 50,
 'ดวง': 12,
 'บริพันธ์': 18,
 'ศาสตราจารย์': 36,
 'วิชา': 33,
 'ชีววิทยา': 11,
 'คณะ': 6,
 'วิทยาศาสตร์': 35,
 'จุฬาลงกรณ์': 8,
 'มหาวิทยาลัย': 26,
 'เรื่อง': 52,
 'สอดไส้': 37,
 'อินโดนีเซีย': 44,
 'เฉลย': 47,
 'กะ': 1,
 'ข่าว': 4,
 'ลือ': 31,
 'ไทย': 61,
 'รูป': 30,
 'โดน': 58,
 'แถว': 56,
 'ประ': 19,
 'จวบฯ': 7,
 'ทางการ': 16,
 'กระหน่ำ': 0,
 'ขนาด': 2,
 'มิจฉาชีพ': 27,
 'ไอ': 63,
 'เดีย': 48,
 'ทำ': 17,
 'หากิน': 41,
 'ปล': 21,
 'เวลา': 53,
 'ขับ': 3,
 'รถ': 28,
 'หลบ': 39,
 'หล่น': 40,
 'พื้น': 23,
 'ดู': 14,
 'เดี๋ยว': 49,
 'พลาด': 22,
 'ชน': 9}

In [371]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
         0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0,
         0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0,
         2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 3, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
         0],
        [1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0,
         1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1,
         2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 4, 1, 1, 2, 1, 0, 2, 1, 0, 1, 0,
         1]], dtype=int64)

In [372]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0]
 [0 0 0 0 0 1 1 0 1 0 1 1 1 0 0 2 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1
  1 1 2 0 0 0 2 1 1 1 2 0 0 0 1 1 3 0 1 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0]
 [1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 0 1 1 1 1 1 1 1 2 1 1 2 0 1 1 1
  1 1 3 1 1 1 2 1 1 1 2 0 1 1 1 1 4 1 1 2 1 0 2 1 0 1 0 1]]


In [373]:
first_news.shape[1]

64

In [374]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 4
This is lowerpart : 20.784609690826528


0.19245008972987526

In [375]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 2
This is lowerpart : 8.485281374238571


0.2357022603955158

In [376]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 30.740852297878796


0.2277100170213244

# ข่าวที่ 32

In [377]:
clean_text = ['เด็กนักเรียนอายุ 12 ปีขึ้นไป ฉีดวัคซีนป้องกันโรคโควิด-19 หากเกิดอาการไม่พึงประสงค์หลังฉีดวัคซีน สปสช.จัดระบบช่วยเหลือเบื้องต้นดูแล ขอยื่นเรื่องรับเงินช่วยเหลือได้','ตามที่มีการนำเสนอข่าว เกี่ยวกับประเด็นเรื่อง นักเรียนที่ได้รับผลข้างเคียงหลังฉีดวัคซีนโควิด-19 สามารถยื่นเรื่องรับเงินช่วยเหลือได้ สูงสุดไม่เกิน 4 แสนบาท ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สำนักงานหลักประกันสุขภาพแห่งชาติ กระทรวงสาธารณสุข พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลจริง','นักเรียนที่ได้รับผลข้างเคียงหลังฉีดวัคซีนโควิด-19 สามารถยื่นเรื่องรับเงินช่วยเหลือได้ สูงสุดไม่เกิน 4 แสนบาท จริงหรือ?','ตามที่มีการนำเสนอข่าว เกี่ยวกับประเด็นเรื่อง นักเรียนที่ได้รับผลข้างเคียงหลังฉีดวัคซีนโควิด-19 สามารถยื่นเรื่องรับเงินช่วยเหลือได้ สูงสุดไม่เกิน 4 แสนบาท ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สำนักงานหลักประกันสุขภาพแห่งชาติ กระทรวงสาธารณสุข พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลจริง']

**Step 2: Tokenize**

In [378]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['เด็ก', 'นักเรียน', 'อายุ', 'ปี', 'ฉีด', 'วัคซีน', 'ป้องกัน', 'โรค', 'โค', 'วิด', 'อาการ', 'ประสงค์', 'ฉีด', 'วัคซีน', 'สปสช', 'ระบบ', 'เบื้อง', 'ต้น', 'ดูแล', 'ยื่น', 'เรื่อง', 'เงิน']
tokenized text:
 ['การนำ', 'เสนอ', 'ข่าว', 'เกี่ยว', 'ประเด็น', 'เรื่อง', 'นักเรียน', 'เคียง', 'ฉีด', 'วัคซีน', 'โค', 'วิด', 'ยื่น', 'เรื่อง', 'เงิน', 'แสน', 'บาท', 'ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ดำเนิน', 'ตรวจ', 'สอบ', 'ข้อ', 'เท็จ', 'สำนักงาน', 'หลัก', 'ประกัน', 'สุขภาพ', 'ชาติ', 'กระทรวง', 'สาธารณสุข', 'ประเด็น', 'ข้อมูล']
tokenized text:
 ['นักเรียน', 'เคียง', 'ฉีด', 'วัคซีน', 'โค', 'วิด', 'ยื่น', 'เรื่อง', 'เงิน', 'แสน', 'บาท']
tokenized text:
 ['การนำ', 'เสนอ', 'ข่าว', 'เกี่ยว', 'ประเด็น', 'เรื่อง', 'นักเรียน', 'เคียง', 'ฉีด', 'วัคซีน', 'โค', 'วิด', 'ยื่น', 'เรื่อง', 'เงิน', 'แสน', 'บาท', 'ศูนย์', 'ต้าน', 'ข่าว', 'ปลอม', 'ดำเนิน', 'ตรวจ', 'สอบ', 'ข้อ', 'เท็จ', 'สำนักงาน', 'หลัก', 'ประกัน', 'สุขภาพ', 'ชาติ', 'กระทรวง', 'สาธารณสุข', 'ประเด็น', 'ข้อมูล']


In [379]:
tokens_list = [split_word(txt) for txt in clean_text]

In [380]:
tokens_list

[['เด็ก',
  'นักเรียน',
  'อายุ',
  'ปี',
  'ฉีด',
  'วัคซีน',
  'ป้องกัน',
  'โรค',
  'โค',
  'วิด',
  'อาการ',
  'ประสงค์',
  'ฉีด',
  'วัคซีน',
  'สปสช',
  'ระบบ',
  'เบื้อง',
  'ต้น',
  'ดูแล',
  'ยื่น',
  'เรื่อง',
  'เงิน'],
 ['การนำ',
  'เสนอ',
  'ข่าว',
  'เกี่ยว',
  'ประเด็น',
  'เรื่อง',
  'นักเรียน',
  'เคียง',
  'ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'ยื่น',
  'เรื่อง',
  'เงิน',
  'แสน',
  'บาท',
  'ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ดำเนิน',
  'ตรวจ',
  'สอบ',
  'ข้อ',
  'เท็จ',
  'สำนักงาน',
  'หลัก',
  'ประกัน',
  'สุขภาพ',
  'ชาติ',
  'กระทรวง',
  'สาธารณสุข',
  'ประเด็น',
  'ข้อมูล'],
 ['นักเรียน',
  'เคียง',
  'ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'ยื่น',
  'เรื่อง',
  'เงิน',
  'แสน',
  'บาท'],
 ['การนำ',
  'เสนอ',
  'ข่าว',
  'เกี่ยว',
  'ประเด็น',
  'เรื่อง',
  'นักเรียน',
  'เคียง',
  'ฉีด',
  'วัคซีน',
  'โค',
  'วิด',
  'ยื่น',
  'เรื่อง',
  'เงิน',
  'แสน',
  'บาท',
  'ศูนย์',
  'ต้าน',
  'ข่าว',
  'ปลอม',
  'ดำเนิน',
  'ตรวจ',
  'สอบ',
  'ข้อ',
  'เท็จ',

**Step 3 + 4a: Bag of words + count word**

In [381]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [382]:
cvec.vocabulary_

{'เด็ก': 36,
 'นักเรียน': 12,
 'อายุ': 32,
 'ปี': 18,
 'ฉีด': 5,
 'วัคซีน': 22,
 'ป้องกัน': 19,
 'โรค': 43,
 'โค': 42,
 'วิด': 23,
 'อาการ': 31,
 'ประสงค์': 15,
 'สปสช': 25,
 'ระบบ': 21,
 'เบื้อง': 38,
 'ต้น': 10,
 'ดูแล': 8,
 'ยื่น': 20,
 'เรื่อง': 39,
 'เงิน': 35,
 'การนำ': 1,
 'เสนอ': 40,
 'ข่าว': 2,
 'เกี่ยว': 33,
 'ประเด็น': 16,
 'เคียง': 34,
 'แสน': 41,
 'บาท': 13,
 'ศูนย์': 24,
 'ต้าน': 11,
 'ปลอม': 17,
 'ดำเนิน': 7,
 'ตรวจ': 9,
 'สอบ': 26,
 'ข้อ': 3,
 'เท็จ': 37,
 'สำนักงาน': 28,
 'หลัก': 30,
 'ประกัน': 14,
 'สุขภาพ': 29,
 'ชาติ': 6,
 'กระทรวง': 0,
 'สาธารณสุข': 27,
 'ข้อมูล': 4}

In [383]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
         1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
         1, 1],
        [1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1,
         0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 1,
         1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
         0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
         1, 0],
        [1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1,
         0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 1,
         1, 0]], dtype=int64)

In [384]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 2 0 0 1 0 1 0 1 0 0 1 0 0 1 1 1 1 2 1 0 1 0 0 0 0 0 1 1 0 0 1
  1 0 1 1 0 0 1 1]
 [1 1 2 1 1 1 1 1 0 1 0 1 1 1 1 0 2 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1
  0 1 0 2 1 1 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1
  0 0 0 1 0 1 1 0]
 [1 1 2 1 1 1 1 1 0 1 0 1 1 1 1 0 2 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1
  0 1 0 2 1 1 1 0]]


In [385]:
first_news.shape[1]

44

In [386]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 11
This is lowerpart : 32.64965543462901


0.3369101405074289

In [387]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 10
This is lowerpart : 16.911534525287763


0.5913123959890826

In [388]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 11
This is lowerpart : 32.64965543462901


0.3369101405074289

# ข่าวที่ 33

In [389]:
clean_text = ['ทางรัฐมีการเรียกเงินคืนจากโครงการต่างๆ เช่นโครงการคนละครึ่ง โครงการเราชนะ ซึ่งโดนทั้งร้านค้าและลูกค้าที่แสกน','พ่อค้าแม่ค้าหลายรายที่เข้าร่วมโครงการรัฐไม่ว่าจะเป็น โครงการเราชนะ โครงการ ม.33 รวมถึงโครงการคนละครึ่ง ได้รับหนังสือแจ้งจากการกระทรวงการคลังว่าต้องทำการคืนเงินให้กับรัฐบาล เนื่องจากไม่ได้ปฏิบัติตามเงื่อนไข โดยผู้ที่ได้รับหนังสือดังกล่าวสามารถติดต่อกับกระทรวงการคลังเพื่อขออุทธรณ์ และชี้แจงข้อเท็จจริง','ร้านค้า"เราชนะ-คนละครึ่ง"โวยถูกเรียกเงินคืนเหตุผิดเงื่อนไข"แลกเงินสด"','ร้านค้าโครงการรัฐ"เราชนะ-คนละครึ่ง" โวยถูก"สศค."เรียกเงินคืนอ้างเหตุไม่ปฏิบัติตามเงื่อนไข ส่วนใหญ่เป็นร้านรับแลกเงินสด- ออนไลน์ สูงสุดเรียกคืน 17 ล้านบาท ด้านนายสามารถ อดีตผช.รมต.ยุติธรรม เตรียมนำผู้ประกอบการร้องทุกข์ต่อนายกฯที่ทำเนียบ 12 ต.ค. นี้']

**Step 2: Tokenize**

In [390]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['รัฐ', 'เงิน', 'คืน', 'โครงการ', 'โครงการ', 'คนละ', 'ครึ่ง', 'โครงการ', 'ชนะ', 'โดน', 'ร้าน', 'ค้า', 'ลูกค้า', 'แส', 'กน']
tokenized text:
 ['พ่อค้า', 'แม่ค้า', 'โครงการ', 'รัฐ', 'โครงการ', 'ชนะ', 'โครงการ', 'ม', 'โครงการ', 'คนละ', 'ครึ่ง', 'หนังสือ', 'แจ้ง', 'การก', 'ระ', 'ทรวง', 'คลัง', 'ทำการ', 'คืน', 'เงิน', 'รัฐบาล', 'เงื่อนไข', 'หนังสือ', 'ติดต่อ', 'กระทรวง', 'คลัง', 'อุทธรณ์', 'ชี้แจง', 'ข้อ', 'เท็จ']
tokenized text:
 ['ร้าน', 'ค้า', 'ชนะ', 'คนละ', 'ครึ่ง', 'โวย', 'เงิน', 'คืน', 'เงื่อนไข', 'แลก', 'เงินสด']
tokenized text:
 ['ร้าน', 'ค้า', 'โครงการ', 'รัฐ', 'ชนะ', 'คนละ', 'ครึ่ง', 'โวย', 'สศค', 'เงิน', 'คืน', 'อ้าง', 'เงื่อนไข', 'ร้าน', 'แลก', 'เงินสด', 'ออนไลน์', 'คืน', 'ล้าน', 'บาท', 'ผช', 'รมต', 'ยุติธรรม', 'เตรียม', 'ร้อง', 'ทุกข์', 'นา', 'ยกฯ', 'ทำเนียบ', 'ต', 'ค']


In [391]:
tokens_list = [split_word(txt) for txt in clean_text]

In [392]:
tokens_list

[['รัฐ',
  'เงิน',
  'คืน',
  'โครงการ',
  'โครงการ',
  'คนละ',
  'ครึ่ง',
  'โครงการ',
  'ชนะ',
  'โดน',
  'ร้าน',
  'ค้า',
  'ลูกค้า',
  'แส',
  'กน'],
 ['พ่อค้า',
  'แม่ค้า',
  'โครงการ',
  'รัฐ',
  'โครงการ',
  'ชนะ',
  'โครงการ',
  'ม',
  'โครงการ',
  'คนละ',
  'ครึ่ง',
  'หนังสือ',
  'แจ้ง',
  'การก',
  'ระ',
  'ทรวง',
  'คลัง',
  'ทำการ',
  'คืน',
  'เงิน',
  'รัฐบาล',
  'เงื่อนไข',
  'หนังสือ',
  'ติดต่อ',
  'กระทรวง',
  'คลัง',
  'อุทธรณ์',
  'ชี้แจง',
  'ข้อ',
  'เท็จ'],
 ['ร้าน',
  'ค้า',
  'ชนะ',
  'คนละ',
  'ครึ่ง',
  'โวย',
  'เงิน',
  'คืน',
  'เงื่อนไข',
  'แลก',
  'เงินสด'],
 ['ร้าน',
  'ค้า',
  'โครงการ',
  'รัฐ',
  'ชนะ',
  'คนละ',
  'ครึ่ง',
  'โวย',
  'สศค',
  'เงิน',
  'คืน',
  'อ้าง',
  'เงื่อนไข',
  'ร้าน',
  'แลก',
  'เงินสด',
  'ออนไลน์',
  'คืน',
  'ล้าน',
  'บาท',
  'ผช',
  'รมต',
  'ยุติธรรม',
  'เตรียม',
  'ร้อง',
  'ทุกข์',
  'นา',
  'ยกฯ',
  'ทำเนียบ',
  'ต',
  'ค']]

**Step 3 + 4a: Bag of words + count word**

In [393]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [394]:
cvec.vocabulary_

{'รัฐ': 27,
 'เงิน': 38,
 'คืน': 8,
 'โครงการ': 47,
 'คนละ': 5,
 'ครึ่ง': 6,
 'ชนะ': 10,
 'โดน': 48,
 'ร้าน': 30,
 'ค้า': 9,
 'ลูกค้า': 31,
 'แส': 46,
 'กน': 0,
 'พ่อค้า': 21,
 'แม่ค้า': 44,
 'ม': 22,
 'หนังสือ': 34,
 'แจ้ง': 43,
 'การก': 2,
 'ระ': 26,
 'ทรวง': 14,
 'คลัง': 7,
 'ทำการ': 15,
 'รัฐบาล': 28,
 'เงื่อนไข': 40,
 'ติดต่อ': 13,
 'กระทรวง': 1,
 'อุทธรณ์': 36,
 'ชี้แจง': 11,
 'ข้อ': 3,
 'เท็จ': 42,
 'โวย': 49,
 'แลก': 45,
 'เงินสด': 39,
 'สศค': 33,
 'อ้าง': 37,
 'ออนไลน์': 35,
 'ล้าน': 32,
 'บาท': 19,
 'ผช': 20,
 'รมต': 25,
 'ยุติธรรม': 24,
 'เตรียม': 41,
 'ร้อง': 29,
 'ทุกข์': 17,
 'นา': 18,
 'ยกฯ': 23,
 'ทำเนียบ': 16,
 'ต': 12,
 'ค': 4}

In [395]:
c_feat[:,:].todense()

matrix([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 1, 3, 1, 0],
        [0, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
         1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 1, 0,
         1, 1, 1, 0, 0, 4, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
         0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
         0, 0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
         0, 0, 0, 1, 0, 1, 0, 1]], dtype=int64)

In [396]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 1 3 1 0]
 [0 1 1 1 0 1 1 2 1 0 1 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 2 0
  1 0 1 0 1 0 1 1 1 0 0 4 0 0]
 [0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 1 1 1 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 1 1 1 0 2 1 1 0 1 0 0 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 2 0 1 1 0 1
  0 1 1 1 1 1 0 0 0 1 0 1 0 1]]


In [397]:
first_news.shape[1]

50

In [398]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 18
This is lowerpart : 31.080540535840104


0.5791405068790082

In [399]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 15.198684153570662


0.46056618647183833

In [400]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 13
This is lowerpart : 27.110883423451916


0.4795122238161564

# ข่าวที่ 34

In [401]:
clean_text = ['นักวิทย์พบโรคติดเชื้ออุบัติใหม่ซุ่มเงียบในญี่ปุ่น สามารถแพร่สู่มนุษย์','คณะนักวิทยาศาสตร์ญี่ปุ่น ค้นพบไวรัสตัวหนึ่งที่ไม่เป็นที่รู้จักมาก่อน ซึ่งมีความสามารถแพร่เชื้อสู่มนุษย์ และจากการศึกษายังชี้ว่า ไวรัสอุบัติใหม่ซึ่งเรียกชื่อกันว่า "ไวรัสเยโซ (Yezo virus)" ตัวนี้ มีความสัมพันธ์กับเชื้อโรคที่ก่อให้เกิดไข้เลือดออกไครเมียน-คองโก (Crimean-Congo hemorrhagic fever) และเชื้อโรคแกะไนโรบี (Nairobi sheep)','นักวิทย์พบโรคติดเชื้ออุบัติใหม่ซุ่มเงียบในญี่ปุ่น เกิดจากถูก‘เห็บ’กัดแล้วแพร่ไวรัสร้ายเข้าสู่มนุษย์','คณะนักวิทยาศาสตร์ญี่ปุ่น ค้นพบไวรัสตัวหนึ่งที่ไม่เป็นที่รู้จักมาก่อน ซึ่งมีความสามารถแพร่เชื้อสู่มนุษย์ และจากการศึกษายังชี้ว่า ไวรัสอุบัติใหม่ซึ่งเรียกชื่อกันว่า "ไวรัสเยโซ (Yezo virus)" ตัวนี้ มีความสัมพันธ์กับเชื้อโรคที่ก่อให้เกิดไข้เลือดออกไครเมียน-คองโก (Crimean-Congo hemorrhagic fever) และเชื้อโรคแกะไนโรบี (Nairobi sheep)']

**Step 2: Tokenize**

In [402]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['วิทย์', 'โรค', 'ติด', 'เชื้อ', 'อุบัติ', 'ซุ่ม', 'เงียบ', 'ญี่ปุ่น', 'แพร่', 'มนุษย์']
tokenized text:
 ['คณะ', 'วิทยาศาสตร์', 'ญี่ปุ่น', 'ค้น', 'ไวรัส', 'ตัว', 'รู้จัก', 'แพร่', 'เชื้อ', 'มนุษย์', 'ศึกษา', 'ชี้', 'ไวรัส', 'อุบัติ', 'ชื่อ', 'ไว', 'รัส', 'เย', 'โซ', 'Yezo', 'virus', 'ตัว', 'สัมพันธ์', 'เชื้อ', 'โรค', 'ก่อ', 'ไข้', 'เลือด', 'ไค', 'ร', 'เมีย', 'น', 'คองโก', 'Crimean', 'Congo', 'hemorrhagic', 'fever', 'เชื้อ', 'โรค', 'แกะ', 'ไนโรบี', 'Nairobi', 'sheep']
tokenized text:
 ['วิทย์', 'โรค', 'ติด', 'เชื้อ', 'อุบัติ', 'ซุ่ม', 'เงียบ', 'ญี่ปุ่น', 'เห็บ', 'กัด', 'แพร่', 'ไวรัส', 'ร้าย', 'มนุษย์']
tokenized text:
 ['คณะ', 'วิทยาศาสตร์', 'ญี่ปุ่น', 'ค้น', 'ไวรัส', 'ตัว', 'รู้จัก', 'แพร่', 'เชื้อ', 'มนุษย์', 'ศึกษา', 'ชี้', 'ไวรัส', 'อุบัติ', 'ชื่อ', 'ไว', 'รัส', 'เย', 'โซ', 'Yezo', 'virus', 'ตัว', 'สัมพันธ์', 'เชื้อ', 'โรค', 'ก่อ', 'ไข้', 'เลือด', 'ไค', 'ร', 'เมีย', 'น', 'คองโก', 'Crimean', 'Congo', 'hemorrhagic', 'fever', 'เชื้อ', 'โรค', 'แกะ', 'ไนโรบี', 'Nairobi

In [403]:
tokens_list = [split_word(txt) for txt in clean_text]

In [404]:
tokens_list

[['วิทย์',
  'โรค',
  'ติด',
  'เชื้อ',
  'อุบัติ',
  'ซุ่ม',
  'เงียบ',
  'ญี่ปุ่น',
  'แพร่',
  'มนุษย์'],
 ['คณะ',
  'วิทยาศาสตร์',
  'ญี่ปุ่น',
  'ค้น',
  'ไวรัส',
  'ตัว',
  'รู้จัก',
  'แพร่',
  'เชื้อ',
  'มนุษย์',
  'ศึกษา',
  'ชี้',
  'ไวรัส',
  'อุบัติ',
  'ชื่อ',
  'ไว',
  'รัส',
  'เย',
  'โซ',
  'Yezo',
  'virus',
  'ตัว',
  'สัมพันธ์',
  'เชื้อ',
  'โรค',
  'ก่อ',
  'ไข้',
  'เลือด',
  'ไค',
  'ร',
  'เมีย',
  'น',
  'คองโก',
  'Crimean',
  'Congo',
  'hemorrhagic',
  'fever',
  'เชื้อ',
  'โรค',
  'แกะ',
  'ไนโรบี',
  'Nairobi',
  'sheep'],
 ['วิทย์',
  'โรค',
  'ติด',
  'เชื้อ',
  'อุบัติ',
  'ซุ่ม',
  'เงียบ',
  'ญี่ปุ่น',
  'เห็บ',
  'กัด',
  'แพร่',
  'ไวรัส',
  'ร้าย',
  'มนุษย์'],
 ['คณะ',
  'วิทยาศาสตร์',
  'ญี่ปุ่น',
  'ค้น',
  'ไวรัส',
  'ตัว',
  'รู้จัก',
  'แพร่',
  'เชื้อ',
  'มนุษย์',
  'ศึกษา',
  'ชี้',
  'ไวรัส',
  'อุบัติ',
  'ชื่อ',
  'ไว',
  'รัส',
  'เย',
  'โซ',
  'Yezo',
  'virus',
  'ตัว',
  'สัมพันธ์',
  'เชื้อ',
  'โรค',
  'ก่อ',
  'ไข้',
  'เลือด

**Step 3 + 4a: Bag of words + count word**

In [405]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [406]:
cvec.vocabulary_

{'วิทย์': 26,
 'โรค': 39,
 'ติด': 18,
 'เชื้อ': 31,
 'อุบัติ': 29,
 'ซุ่ม': 15,
 'เงียบ': 30,
 'ญี่ปุ่น': 16,
 'แพร่': 37,
 'มนุษย์': 20,
 'คณะ': 10,
 'วิทยาศาสตร์': 25,
 'ค้น': 12,
 'ไวรัส': 44,
 'ตัว': 17,
 'รู้จัก': 23,
 'ศึกษา': 27,
 'ชี้': 13,
 'ชื่อ': 14,
 'ไว': 43,
 'รัส': 22,
 'เย': 33,
 'โซ': 38,
 'Yezo': 3,
 'virus': 7,
 'สัมพันธ์': 28,
 'ก่อ': 9,
 'ไข้': 40,
 'เลือด': 34,
 'ไค': 41,
 'ร': 21,
 'เมีย': 32,
 'น': 19,
 'คองโก': 11,
 'Crimean': 1,
 'Congo': 0,
 'hemorrhagic': 5,
 'fever': 4,
 'แกะ': 36,
 'ไนโรบี': 42,
 'Nairobi': 2,
 'sheep': 6,
 'เห็บ': 35,
 'กัด': 8,
 'ร้าย': 24}

In [407]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
         0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1,
         1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 3, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1,
         1, 1, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
         0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1,
         1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 3, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1,
         1, 1, 2]], dtype=int64)

In [408]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0
  0 1 0 1 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 2 0 1 1 1 1 1 0 1 0 1 1 1 0 3 1 1 1 0
  1 1 1 2 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1
  0 1 0 1 0 0 0 0 1]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 2 0 1 1 1 1 1 0 1 0 1 1 1 0 3 1 1 1 0
  1 1 1 2 1 1 1 1 2]]


In [409]:
first_news.shape[1]

45

In [410]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 9
This is lowerpart : 23.45207879911715


0.38376128944009874

In [411]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 10
This is lowerpart : 11.832159566199232


0.8451542547285166

In [412]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 9
This is lowerpart : 23.45207879911715


0.38376128944009874

# ข่าวที่ 35

In [413]:
clean_text = ['Noneทางรัฐมีการเรียกเงินคืนจากโครงการต่างๆ เช่นโครงการคนละครึ่ง โครงการเราชนะ ซึ่งโดนทั้งร้านค้าและลูกค้าที่แสกน','','"วัคซีนพาสปอร์ต" บัตรผ่านเดินทางในต่างประเทศ เงื่อนไข ขั้นตอนการขอไม่ยุ่งยาก','']

**Step 2: Tokenize**

In [414]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['None', 'รัฐ', 'เงิน', 'คืน', 'โครงการ', 'โครงการ', 'คนละ', 'ครึ่ง', 'โครงการ', 'ชนะ', 'โดน', 'ร้าน', 'ค้า', 'ลูกค้า', 'แส', 'กน']
tokenized text:
 []
tokenized text:
 ['วัคซีน', 'พาส', 'ปอร์ต', 'บัตร', 'เดิน', 'ทางใน', 'ประเทศ', 'เงื่อนไข', 'ตอน', 'ยุ่ง']
tokenized text:
 []


In [415]:
tokens_list = [split_word(txt) for txt in clean_text]

In [416]:
tokens_list

[['None',
  'รัฐ',
  'เงิน',
  'คืน',
  'โครงการ',
  'โครงการ',
  'คนละ',
  'ครึ่ง',
  'โครงการ',
  'ชนะ',
  'โดน',
  'ร้าน',
  'ค้า',
  'ลูกค้า',
  'แส',
  'กน'],
 [],
 ['วัคซีน',
  'พาส',
  'ปอร์ต',
  'บัตร',
  'เดิน',
  'ทางใน',
  'ประเทศ',
  'เงื่อนไข',
  'ตอน',
  'ยุ่ง'],
 []]

**Step 3 + 4a: Bag of words + count word**

In [417]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [418]:
cvec.vocabulary_

{'None': 1,
 'รัฐ': 15,
 'เงิน': 19,
 'คืน': 5,
 'โครงการ': 23,
 'คนละ': 3,
 'ครึ่ง': 4,
 'ชนะ': 7,
 'โดน': 24,
 'ร้าน': 16,
 'ค้า': 6,
 'ลูกค้า': 17,
 'แส': 22,
 'กน': 2,
 '': 0,
 'วัคซีน': 18,
 'พาส': 13,
 'ปอร์ต': 12,
 'บัตร': 10,
 'เดิน': 21,
 'ทางใน': 9,
 'ประเทศ': 11,
 'เงื่อนไข': 20,
 'ตอน': 8,
 'ยุ่ง': 14}

In [419]:
c_feat[:,:].todense()

matrix([[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
         0, 1, 3, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
         1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], dtype=int64)

In [420]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 3 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 1 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [421]:
first_news.shape[1]

25

In [422]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 0
This is lowerpart : 4.69041575982343


0.0

In [423]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 0
This is lowerpart : 14.832396974191328


0.0

In [424]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 0
This is lowerpart : 4.69041575982343


0.0

------------------------------------   Develop Part --------------------------------------------

**Step 3 + 4b: Bag of words + tf-idf**

In [425]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
t_feat = tvec.fit_transform(tokens_list_j)

In [426]:
t_feat[:,:5].todense()

matrix([[0.        , 0.21320072, 0.21320072, 0.21320072, 0.21320072],
        [1.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        , 0.        , 0.        ]])

In [427]:
print(len(tvec.idf_),len(tvec.vocabulary_))

25 25


In [428]:
c_feat[:,:5].todense()

matrix([[0, 1, 1, 1, 1],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]], dtype=int64)